In [2]:
import pandas as pd
from am_notebooks import *

from source.utils.associate import TOP_AM_DIR
from source.utils.associate import adjust_am_names as adjust_assoc_columns
from source.utils.general import HIT_TABLES_DIR, timestamp_today
from source.utils.sample import sample_pickle as sp

VERBOSE = True # ! MAKE SURE THIS IS WHAT YOU WANT

REFILTER_NEG = False
K = 8
BK = max(K+2, 10)
N_EX_PER_BIGRAM = 99
BIGRAM_F_FLOOR = 50
ADV_F_FLOOR = 5000
DATE = timestamp_today()

TAG = 'NEQ'
METRIC_PRIORITY = METRIC_PRIORITY_DICT[TAG]
TOP_AM_TAG_DIR = TOP_AM_DIR / TAG
TAG_TOP_STR = f'{TAG}-Top{K}'
OUT_DIR = TOP_AM_TAG_DIR / TAG_TOP_STR

FOCUS = adjust_am_names(FOCUS_DICT[TAG]['polar'])
FOCUS_MEANS = [f'mean_{c}' for c in FOCUS]
SET_FOCUS = [f'{c}_SET' for c in FOCUS]
MIR_FOCUS = [f'{c}_MIR' for c in FOCUS]
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_colwidth", 70)

In [3]:
# * parquet paths
NEG_HITS_PATH = HIT_TABLES_DIR / 'RBdirect'/'ALL-RBdirect_final.parq'
PRESAMPLED_PARQ = find_most_recent_top_am(
    date_str=DATE, data_dir=OUT_DIR, suffix='.parq',
    undated_stem=f'{TAG_TOP_STR}adv_sample-hits_')

In [4]:
adv_am = seek_top_adv_am(
    date_str=DATE, adv_floor=ADV_F_FLOOR, tag_top_dir=OUT_DIR)
adv_am = adjust_assoc_columns(adv_am).convert_dtypes()
if VERBOSE:

    md_frame_code(r"""for selector, selection in (('*_SET', SET_FOCUS), 
                            ('*_MIR', MIR_FOCUS), 
                            ('mean_*', FOCUS_MEANS)):
    nb_show_table(adv_am.filter(selection[:-3]), 
                  title=f'### `{selector}` columns for loaded top adverbs')""")

    for selector, selection in (('*_SET', SET_FOCUS),
                                ('*_MIR', MIR_FOCUS),
                                ('mean_*', FOCUS_MEANS)):

        nb_show_table(adv_am.filter(selection[:-3]),
                      title=f'### `{selector}` columns for loaded top adverbs')

> Loaded top adv AM table from  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/NEQ-Top8/NEQ-Top8_NEG-ADV_combined-5000.2024-08-05.csv`

```python
for selector, selection in (('*_SET', SET_FOCUS), 
                            ('*_MIR', MIR_FOCUS), 
                            ('mean_*', FOCUS_MEANS)):
    nb_show_table(adv_am.filter(selection[:-3]), 
                  title=f'### `{selector}` columns for loaded top adverbs')
```


### `*_SET` columns for loaded top adverbs

|                  |   `f_SET` |   `dP1_SET` |   `LRC_SET` |   `P1_SET` |   `G2_SET` | `l1_SET`   |   `f1_SET` |   `f2_SET` |   `N_SET` |   `exp_f_SET` |   `unexp_f_SET` |   `unexp_r_SET` |   `dP2_SET` |   `P2_SET` |
|:-----------------|----------:|------------:|------------:|-----------:|-----------:|:-----------|-----------:|-----------:|----------:|--------------:|----------------:|----------------:|------------:|-----------:|
| **necessarily**  |    42,595 |        0.50 |        6.65 |       0.99 |  55,995

In [5]:
if VERBOSE:
    md_frame_code("""print('## Comparing metrics across data selections\n')
for metric_cue in METRIC_PRIORITY + ['^[_%]*f', 'f2', '^exp_', 'unexp']:
    print(f'### Displaying Juxtaposed Values for `{repr(metric_cue)}`')
    md_frame_code(f'compare_datasets(adv_am, {repr(metric_cue)})')
    compare_datasets(adv_am, metric_cue)""")

    print('## Comparing metrics across data selections\n')
    for metric_cue in METRIC_PRIORITY + ['^[_%]*f', 'f2', '^exp_', 'unexp']:
        print(f'### Displaying Juxtaposed Values for `{repr(metric_cue)}`')
        md_frame_code(f'compare_datasets(adv_am, {repr(metric_cue)})')
        compare_datasets(adv_am, metric_cue)


```python
print('## Comparing metrics across data selections
')
for metric_cue in METRIC_PRIORITY + ['^[_%]*f', 'f2', '^exp_', 'unexp']:
    print(f'### Displaying Juxtaposed Values for `{repr(metric_cue)}`')
    md_frame_code(f'compare_datasets(adv_am, {repr(metric_cue)})')
    compare_datasets(adv_am, metric_cue)
```

## Comparing metrics across data selections

### Displaying Juxtaposed Values for `'LRC'`

```python
compare_datasets(adv_am, 'LRC')
```

Top 5 by descending `LRC_SET`

|                 |   `LRC_SET` |   `LRC_MIR` |   `mean_LRC` |
|:----------------|------------:|------------:|-------------:|
| **necessarily** |         6.7 |         3.8 |          5.2 |
| **that**        |         6.3 |         3.8 |          5.0 |
| **exactly**     |         6.0 |         3.1 |          4.5 |
| **immediately** |         4.9 |         0.6 |          2.7 |
| **yet**         |         4.6 |         1.0 |          2.8 |

Top 5 by descending `LRC_MIR`

|                 |   `LRC_SET` |  

In [6]:
if VERBOSE:
    md_frame_code("""TOP_ADV, adv_am = pin_top_adv(
        adv_am, select_col=(
            adv_am.filter([f'mean_{m}' for m in METRIC_PRIORITY])
            .columns.to_list()
        ))""")
TOP_ADV, adv_am = pin_top_adv(
    adv_am, select_col=(
        adv_am.filter([f'mean_{m}' for m in METRIC_PRIORITY])
        .columns.to_list()
    ))


```python
TOP_ADV, adv_am = pin_top_adv(
        adv_am, select_col=(
            adv_am.filter([f'mean_{m}' for m in METRIC_PRIORITY])
            .columns.to_list()
        ))
```

## Top Adverb Selection, as ranked by descending `['mean_LRC', 'mean_P1', 'mean_G2', 'mean_P2']`

|        | `adv`        |   `mean_LRC` |   `mean_P1` |   `mean_G2` |   `mean_P2` |
|:-------|:-------------|-------------:|------------:|------------:|------------:|
| **1**  | necessarily  |        5.237 |       0.981 |  28,552.081 |       0.008 |
| **2**  | that         |        5.012 |       0.969 | 109,485.814 |       0.033 |
| **3**  | exactly      |        4.529 |       0.965 |  28,291.906 |       0.008 |
| **4**  | any          |        4.245 |       0.968 |   8,731.768 |       0.004 |
| **5**  | remotely     |        3.288 |       0.932 |   3,538.326 |       0.004 |
| **6**  | yet          |        2.793 |       0.872 |  29,285.224 |       0.009 |
| **7**  | immediately  |        2.744 |       0.835 |

In [7]:
undated_stem = f'{TAG_TOP_STR}_NEG-ADV-{ADV_F_FLOOR}_Top{BK}-bigrams-{BIGRAM_F_FLOOR}'
top_polar_bigram_am_csv = find_most_recent_top_am(DATE, data_dir=OUT_DIR, undated_stem=undated_stem)
if VERBOSE:
    print(f'\n* Loading from `{top_polar_bigram_am_csv}`')


* Loading from `/share/compling/projects/sanpi/results/top_AM/NEQ/NEQ-Top8/NEQ-Top8_NEG-ADV-5000_Top10-bigrams-50.2024-08-05.csv`


In [8]:
polar_bigram_am = adjust_assoc_columns(
    pd.read_csv(top_polar_bigram_am_csv)
    .set_index('key')
    # > not strictly necessary (loaded table should already satisfy this) but just in case...
    .filter(regex=r'|'.join([f'~{a}_' for a in TOP_ADV]), axis=0)
).filter(items=FOCUS).convert_dtypes()

In [9]:
if VERBOSE:
    md_frame_code("""sample = polar_bigram_am.sample(3)
sample.update(sample.select_dtypes(include='float').round(4))
nb_show_table(sample, transpose=True)""")

    sample = polar_bigram_am.sample(3)
    sample.update(sample.select_dtypes(include='float').round(4))
    nb_show_table(sample, transpose=True)


```python
sample = polar_bigram_am.sample(3)
sample.update(sample.select_dtypes(include='float').round(4))
nb_show_table(sample, transpose=True)
```


|                   | `NEGany~necessarily_enough`   | `NEGany~that_different`   | `NEGany~terribly_good`   |
|:------------------|:------------------------------|:--------------------------|:-------------------------|
| **`f`**           | 279                           | 6534                      | 332                      |
| **`dP1`**         | 0.5                           | 0.4972                    | 0.4852                   |
| **`LRC`**         | 3.9325                        | 6.7723                    | 3.4082                   |
| **`P1`**          | 1.0                           | 0.9966                    | 0.9852                   |
| **`G2`**          | 386.7884                      | 8800.6354                 | 415.1662                 |
| **`l1`**          | NEGATED                       | NEGATED                   | NEG

In [10]:
polar_bigram_am = polar_bigram_am.sort_values(METRIC_PRIORITY, ascending=False)
if VERBOSE:
    md_frame_code(
'''save_top_bigrams_overall_md(polar_bigram_am, 
                            overall_k=int(BK/2 * (K+1)), 
                            out_dir=OUT_DIR,
                            bigram_floor=BIGRAM_F_FLOOR, 
                            adv_floor=ADV_F_FLOOR, 
                            suppress=not VERBOSE)'''
    )

save_top_bigrams_overall_md(polar_bigram_am, 
                            overall_k=int(BK/2 * (K+1)), 
                            out_dir=OUT_DIR,
                            bigram_floor=BIGRAM_F_FLOOR, 
                            adv_floor=ADV_F_FLOOR, 
                            suppress=not VERBOSE)


```python
save_top_bigrams_overall_md(polar_bigram_am, 
                            overall_k=int(BK/2 * (K+1)), 
                            out_dir=OUT_DIR,
                            bigram_floor=BIGRAM_F_FLOOR, 
                            adv_floor=ADV_F_FLOOR, 
                            suppress=not VERBOSE)
```


|                                       |    `f` |   `dP1` |   `LRC` |   `P1` |      `G2` | `l1`    | `l2`                       |      `f1` |   `f2` |       `N` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` | `adv`       |   `adv_total` | `adj`          |   `adj_total` |   `odds_r_disc` |   `t` |   `MI` |
|:--------------------------------------|-------:|--------:|--------:|-------:|----------:|:--------|:---------------------------|----------:|-------:|----------:|----------:|------------:|------------:|--------:|-------:|:------------|--------------:|:---------------|--------------:|----------------:|------:|-------:|
| **NEGany~yet_clear**          

### Load Negated Hit Table

In [ ]:
if NEG_HITS_PATH.suffix.startswith('.parq'):
    neg_hits = pd.read_parquet(NEG_HITS_PATH,
                               filters=[('adv_form_lower', 'in', TOP_ADV)])
elif 'pkl' in NEG_HITS_PATH.suffixes:
    neg_hits = pd.read_pickle(NEG_HITS_PATH)

    neg_hits = neg_hits.loc[(neg_hits.adv_lemma.isin(TOP_ADV))
                            | (neg_hits.adv_form_lower.isin(TOP_ADV)), :]


neg_hits = (
    neg_hits.filter(
        regex=r'^[nab].*lower|text|token|g_head|(adv|neg)_lemma')
    .drop_duplicates(['text_window', 'bigram_lower']))


neg_hits = (
    neg_hits.filter(
        regex=r'^[nab].*lower|text|token|g_head|(adv|neg)_lemma')
    .drop_duplicates(['text_window', 'bigram_lower']))

if VERBOSE:
    print(neg_hits.neg_lemma.value_counts()
          .to_frame('subtotal in loaded negated hits sample')
          .to_markdown(intfmt=','))
    nb_show_table(neg_hits.sample(2), transpose=True)

| neg_lemma   |   subtotal in loaded negated hits sample |
|:------------|-----------------------------------------:|
| not         |                                  119,395 |
| nothing     |                                    2,476 |
| never       |                                    1,809 |
| none        |                                    1,064 |
| nor         |                                      553 |
| neither     |                                      427 |
| hardly      |                                      325 |
| no          |                                      218 |
| nobody      |                                      216 |
| rarely      |                                      151 |
| without     |                                      103 |
| ain't       |                                       79 |
| few         |                                       57 |
| barely      |                                       24 |
| scarcely    |                                       20 |
| aint        |                                       12 |
| seldom      |                                       10 |

|                       | `pcc_eng_20_002.4074_x0022529_24:25-27-28`                                                                                                                                                        | `pcc_eng_10_071.4197_x1138397_04:5-6-7`                                                                                                                                                                                                                         |
|:----------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`sent_text`**       | *We can replicate this to a decent degree in captivity, and not being a far-ranging animal that again puts a bonus into captivity not being particularly harmful or restrictive to the animal.*   | *Admittedly, they're not particularly inconspicuous about it, given that they are both GIANT and in possession of a decently hefty marketing budget that allows them to plaster websites with megaton banner ads and titanic, lavishly produced trailers.*      |
| **`hit_text`**        | *not being particularly harmful or restrictive to the*                                                                                                                                            | *not particularly inconspicuous about it , given*                                                                                                                                                                                                               |
| **`text_window`**     | *puts a bonus into captivity not being particularly harmful or restrictive to the animal .*                                                                                                       | *admittedly , they 're not particularly inconspicuous about it , given that they*                                                                                                                                                                               |
| **`token_str`**       | *We can replicate this to a decent degree in captivity , and not being a far-ranging animal that again puts a bonus into captivity not being particularly harmful or restrictive to the animal .* | *Admittedly , they 're not particularly inconspicuous about it , given that they are both GIANT and in possession of a decently hefty marketing budget that allows them to plaster websites with megaton banner ads and titanic , lavishly produced trailers .* |
| **`neg_head`**        | ADJ                                                                                                                                                                                               | ADJ                                                                                                                                                                                                                                                             |
| **`neg_lemma`**       | not                                                                                                                                                                                               | not                                                                                                                                                                                                                                                             |
| **`adv_lemma`**       | particularly                                                                                                                                                                                      | particularly                                                                                                                                                                                                                                                    |
| **`adv_form_lower`**  | particularly                                                                                                                                                                                      | particularly                                                                                                                                                                                                                                                    |
| **`adj_form_lower`**  | harmful                                                                                                                                                                                           | inconspicuous                                                                                                                                                                                                                                                   |
| **`bigram_lower`**    | particularly_harmful                                                                                                                                                                              | particularly_inconspicuous                                                                                                                                                                                                                                      |
| **`neg_form_lower`**  | not                                                                                                                                                                                               | not                                                                                                                                                                                                                                                             |
| **`all_forms_lower`** | not_particularly_harmful                                                                                                                                                                          | not_particularly_inconspicuous                                                                                                                                                                                                                                  |



In [ ]:
neg_hits = clarify_neg_categories(neg_hits, verbose=VERBOSE)
if 'all_forms_lower' not in neg_hits.columns:
    # sourcery skip: use-fstring-for-concatenation
    neg_hits['all_forms_lower'] = (
        neg_hits.neg_form_lower.astype('string')
        + '_'
        + neg_hits.bigram_lower.astype('string')
    ).astype('category')

Updated `neg_lemma` counts with "ain(')t" merged with "not"

| neg_lemma   |   count |
|:------------|--------:|
| not         | 119,486 |
| nothing     |   2,476 |
| never       |   1,809 |
| none        |   1,064 |
| nor         |     553 |
| neither     |     427 |
| hardly      |     325 |
| no          |     218 |
| nobody      |     216 |
| rarely      |     151 |
| without     |     103 |
| few         |      57 |
| barely      |      24 |
| scarcely    |      20 |
| seldom      |      10 |


In [13]:
if PRESAMPLED_PARQ.exists():
    all_hits = pd.read_parquet(PRESAMPLED_PARQ, engine='pyarrow')
else: 
    all_hits = pd.read_parquet(str(NEG_HITS_PATH).replace('RBdirect', 'not-RBdirect'), 
                               filters=[('adv_form_lower', 'in', TOP_ADV)])

In [14]:
pos_hits = all_hits.copy().loc[all_hits.polarity == 'pos', :]
if VERBOSE:
    print('Confirming "Positive" Enforcement Accuracy.... 👀')
    seek_errant_negations(pos_hits)

Confirming "Positive" Enforcement Accuracy.... 👀
🎉 No errant negations found! ✨✅


In [ ]:
if VERBOSE:
    for p, d in all_hits.filter(
        ['polarity', 'all_forms_lower', 'text_window']
    ).assign(
        all_forms_lower=all_hits.all_forms_lower.str.replace('_', ' ')
    ).groupby('polarity', observed=True):
        
        print(f'\n#### sample of `polarity=={p}` hits\n')
        nb_show_table(d.sample(5).sort_values('all_forms_lower').filter(['all_forms_lower', 'text_window']))


#### sample of `polarity==neg` hits


|                                               | `all_forms_lower`       | `text_window`                                                                          |
|:----------------------------------------------|:------------------------|:---------------------------------------------------------------------------------------|
| **pcc_eng_22_088.5143_x1414554_6:2-4-5**      | *n't particularly cute* | *are n't they particularly cute when worn casually with a twist*                       |
| **pcc_eng_22_007.8494_x0110588_183:18-19-20** | *n't that concerned*    | *in the 1990s , they were n't that concerned about rupert murdoch or general electric* |
| **pcc_eng_24_102.7696_x1646461_13:15-16-17**  | *n't that major*        | *released as uploads as they are n't that major a change - and are sometimes*          |
| **pcc_eng_28_066.2942_x1056440_18:2-3-4**     | *not exactly new*       | *- not exactly new , but did come around with*                                         |
| **pcc_eng_05_002.6677_x0027179_40:4-5-6**     | *not exactly true*      | *well that 's not exactly true i would n't eat any weird*                              |


#### sample of `polarity==pos` hits


|                                            | `all_forms_lower`              | `text_window`                                                                               |
|:-------------------------------------------|:-------------------------------|:--------------------------------------------------------------------------------------------|
| **pcc_eng_08_041.2382_x0651236_35:14-15**  | *(+) particularly fine*        | *, there in numbers but also in particularly fine voice .*                                  |
| **pcc_eng_09_045.8555_x0725768_074:11-12** | *(+) particularly fine*        | *in your aunt 's sitting room is particularly fine . "*                                     |
| **pcc_eng_15_010.2219_x0148930_13:5-6**    | *(+) particularly happy*       | *colageo has michael ryder particularly happy with the outcome of the series*               |
| **pcc_eng_17_014.0395_x0210817_097:6-7**   | *(+) particularly significant* | *an early impact study shows particularly significant levels of parent support and mindset* |
| **pcc_eng_14_025.0275_x0388196_78:1-2**    | *(+) terribly blind*           | *terribly blind .*                                                                          |



In [ ]:
if VERBOSE:
    nb_show_table(neg_hits.loc[neg_hits.adv_lemma.astype('string')
                            != neg_hits.adv_form_lower.astype('string')
                            ].filter(['adv_lemma', 'adv_form_lower', 'text_window']).sample(4))
    nb_show_table(neg_hits.filter(like='adv_').loc[
        neg_hits.adv_lemma.astype(
            'string') != neg_hits.adv_form_lower.astype('string')
    ].value_counts().to_frame().reset_index())


|                                              | `adv_lemma`   | `adv_form_lower`   | `text_window`                                                                                                       |
|:---------------------------------------------|:--------------|:-------------------|:--------------------------------------------------------------------------------------------------------------------|
| **pcc_eng_26_083.9681_x1341685_08:19-20-21** | long          | longer             | *for the students ( it was not longer necessary to check the showcases of the*                                      |
| **pcc_eng_10_045.5536_x0720858_42:36-45-46** | long          | longer             | *the First Amendment is nothing more than a quaint 18th Century notion no longer applicable to the narrow-minded ,* |
| **pcc_eng_06_025.9504_x0403798_10:09-10-11** | long          | longer             | *i was freaking out about is not longer stuck to the rest of my thoughts*                                           |
| **pcc_eng_00_008.3566_x0118730_25:16-18-19** | long          | longer             | *defined by what it is not no longer imperial , no longer soviet , no*                                              |


|       | `adv_lemma`   | `adv_form_lower`   |   `count` |
|:------|:--------------|:-------------------|----------:|
| **1** | long          | longer             |       210 |



In [ ]:
if VERBOSE:
    print(neg_hits.adv_form_lower.value_counts()
          .to_frame('Tokens in loaded *negated* sample')
          .to_markdown(floatfmt=',.0f', intfmt=','))
    for pol, pol_hits in all_hits.groupby('polarity', observed=True):
        print(f'\n ### `{pol}` adv subtotals\n')
        print(pol_hits.adv_form_lower.value_counts()
              .to_frame('Tokens in loaded sample')
              .to_markdown(floatfmt=',.0f', intfmt=','))

| adv_form_lower   |   Tokens in loaded *negated* sample |
|:-----------------|------------------------------------:|
| that             |                              45,346 |
| particularly     |                              15,526 |
| immediately      |                              14,700 |
| yet              |                              14,546 |
| exactly          |                              12,302 |
| necessarily      |                              11,693 |
| terribly         |                               4,876 |
| any              |                               4,248 |
| ever             |                               1,701 |
| remotely         |                               1,622 |
| longer           |                                 379 |

 ### `neg` adv subtotals

| adv_form_lower   |   Tokens in loaded sample |
|:-----------------|--------------------------:|
| that             |                    45,345 |
| particularly     |                    15,526 |
| immediately      |                    14,700 |
| yet              |                    14,546 |
| exactly          |                    12,302 |
| necessarily      |                    11,692 |
| terribly         |                     4,876 |
| any              |                     4,248 |
| ever             |                     1,701 |
| remotely         |                     1,622 |
| longer           |                       379 |

 ### `pos` adv subtotals

| adv_form_lower   |   Tokens in loaded sample |
|:-----------------|--------------------------:|
| particularly     |                    21,153 |
| ever             |                     4,915 |
| yet              |                     1,910 |
| terribly         |                     1,849 |
| that             |                     1,844 |
| immediately      |                     1,625 |
| any              |                       781 |
| exactly          |                       538 |
| remotely         |                       448 |
| longer           |                       437 |
| necessarily      |                       321 |


In [ ]:
if VERBOSE:
    rare_forms = neg_hits.neg_form_lower.value_counts().nsmallest(6).index
    # nb_show_table(neg_hits.loc[neg_hits.neg_form_lower.isin(rare_forms), :].sort_values('neg_form_lower').filter(regex=r'bigram|neg|text'))
    nb_show_table(neg_hits.filter(['all_forms_lower','text_window'])
                  .assign(
                      text_window=italic(embolden(
                          neg_hits.text_window, 
                          bold_regex=r'\b('+r'|'.join(rare_forms)+r')\b')))
                  .loc[neg_hits.neg_form_lower.isin(rare_forms), :]
                  .sort_values('all_forms_lower')
                  )


|                                               | `all_forms_lower`               | `text_window`                                                                                                 |
|:----------------------------------------------|:--------------------------------|:--------------------------------------------------------------------------------------------------------------|
| **pcc_eng_04_001.4775_x0007747_3:4-6-7**      | fewer_exactly_solvable          | *in 2d far __``fewer``__ are exactly solvable , the simplest being a rectangle*                               |
| **pcc_eng_02_047.2547_x0748214_04:11-13-14**  | n_longer_safe                   | *travelers , the roads are __``n``__ o longer safe for us to travel .*                                        |
| **pcc_eng_17_078.8556_x1258350_19:2-3-4**     | n`t_that_beautiful              | *is __``n``__`t that beautiful and all you have to do*                                                        |
| **pcc_eng_20_080.0820_x1277728_116:20-23-24** | nobodies_that_big               | *away from the story __``nobodies``__ dick is that big and if it was having a*                                |
| **pcc_eng_18_002.1227_x0018259_110:3-4-5**    | nt_that_much                    | *This is __``nt``__ that much of a hardship .*                                                                |
| **pcc_eng_06_073.4511_x1171981_07:13-14-15**  | seldom_any_cheaper              | *nowadays as the goods there are __``seldom``__ any cheaper than may find in a good*                          |
| **pcc_eng_13_033.0482_x0518168_054:20-21-22** | seldom_any_stronger             | *: the weapons the generated were __``seldom``__ any stronger than regular weapons , and tended*              |
| **pcc_eng_00_012.2130_x0181116_61:09-10-11**  | seldom_ever_careful             | *, goodness , those children are __``seldom``__ ever careful , " she mused .*                                 |
| **pcc_eng_14_089.1449_x1424729_051:3-5-6**    | seldom_ever_fearful             | *max was __``seldom``__ if ever fearful , not even during the mystery*                                        |
| **pcc_eng_02_009.2911_x0133924_53:10-12-13**  | seldom_ever_forthcoming         | *why questions of life are __``seldom``__ if ever forthcoming from god .*                                     |
| **pcc_eng_03_034.7677_x0547039_21:08-10-11**  | seldom_ever_sexualized          | *same time , alice is __``seldom``__ if ever sexualized -- she gives off more of*                             |
| **pcc_eng_24_022.5354_x0347911_012:32-33-34** | seldom_immediately_popular      | *which is really creative -- and __``seldom``__ immediately popular -- should apply themselves sedulously to* |
| **pcc_eng_val_2.01260_x18257_071:5-6-7**      | seldom_particularly_stimulating | *even recorded lectures are __``seldom``__ particularly stimulating unless delivered by a gifted speaker*     |
| **pcc_eng_26_004.6713_x0059186_165:3-5-6**    | seldom_that_serious             | *Things are __``seldom``__ really that serious .*                                                             |
| **pcc_eng_01_103.1159_x1650317_068:11-12-13** | seldom_that_simple              | *, end- of- life decisions are __``seldom``__ that simple .*                                                  |



In [ ]:
if VERBOSE:
    _sample = neg_hits.loc[
        ((neg_hits.neg_form_lower != "n't")
         & (neg_hits.neg_lemma.astype('string')
            != neg_hits.neg_form_lower.astype('string'))),
        ['neg_lemma', 'neg_form_lower', 'text_window']
    ].sample(10).sort_values('neg_form_lower')

    nb_show_table(
        _sample.assign(
            text_window=italic(embolden(
                _sample.text_window,
                bold_regex=r'(\b'+r'\b|\b'.join(_sample.neg_form_lower)+r')\b'))))


|                                               | `neg_lemma`   | `neg_form_lower`   | `text_window`                                                                                          |
|:----------------------------------------------|:--------------|:-------------------|:-------------------------------------------------------------------------------------------------------|
| **pcc_eng_24_027.2903_x0425051_063:10-11-12** | not           | ain't              | *the lower 48 , alaska just __``ain't``__ that great .*                                                |
| **pcc_eng_23_031.0729_x0485493_07:11-12-13**  | not           | ain't              | *think it 's natural , it __``ain't``__ ever natural ; you 'd like to be*                              |
| **pcc_eng_17_077.5733_x1237401_18:26-27-28**  | not           | ain't              | *interests and become friends - aww __``ain't``__ that great !*                                        |
| **pcc_eng_10_028.7595_x0448752_056:08-09-10** | not           | ain't              | *great trepidation , because this ink __``ain't``__ particularly cheap , i diluted it with de-ionised* |
| **pcc_eng_14_092.1583_x1473622_088:3-4-5**    | not           | ain't              | *it really __``ain't``__ that bad .*                                                                   |
| **pcc_eng_04_075.9094_x1209755_083:4-5-6**    | not           | ain't              | *honestly , it __``ain't``__ that much of a secret when you 're*                                       |
| **pcc_eng_01_102.1319_x1634470_23:6-7-8**     | not           | aint               | *trust me the pats secondary __``aint``__ that good .*                                                 |
| **pcc_eng_14_095.0942_x1521024_12:16-17-18**  | not           | aint               | *the last seven years , it __``aint``__ that impressive .*                                             |
| **pcc_eng_02_035.3300_x0555650_137:4-5-6**    | not           | nit                | *although they were __``nit``__ particularly successful on radio , the group continued*                |
| **pcc_eng_23_004.9083_x0062990_035:22-23-24** | not           | nit                | *or videotape , but that are __``nit``__ necessarily audible to the human ear -- these*                |



In [ ]:
if VERBOSE:
    _sample = neg_hits.loc[
        ((neg_hits.neg_lemma != "not")
         & (neg_hits.neg_lemma.astype('string') != neg_hits.neg_form_lower.astype('string'))),
        ['neg_lemma', 'neg_form_lower', 'text_window']].sort_values('neg_form_lower')

    nb_show_table(
        _sample.assign(
            text_window=italic(embolden(
                _sample.text_window,
                bold_regex=r'(\b'+r'\b|\b'.join(_sample.neg_form_lower)+r')\b'))))


|                                               | `neg_lemma`   | `neg_form_lower`   | `text_window`                                                                   |
|:----------------------------------------------|:--------------|:-------------------|:--------------------------------------------------------------------------------|
| **pcc_eng_04_001.4775_x0007747_3:4-6-7**      | few           | fewer              | *in 2d far __``fewer``__ are exactly solvable , the simplest being a rectangle* |
| **pcc_eng_20_080.0820_x1277728_116:20-23-24** | nobody        | nobodies           | *away from the story __``nobodies``__ dick is that big and if it was having a*  |



In [ ]:
print(f'# {BK} Most Negative Bigrams for each of the {K} Most Negative Adverbs\n')

for rank, adverb in enumerate(adv_am.index, start=1):
    print(f'\n## {rank}. *{adverb}*')
    populate_adv_dir(
        adverb,
        bigram_am=polar_bigram_am,
        neg_hits_df=neg_hits,
        pos_hits_df=pos_hits,
        data_dir=TOP_AM_TAG_DIR, 
        rank_by=METRIC_PRIORITY,
        n_bigrams=BK, n_ex=N_EX_PER_BIGRAM,
        verbose=VERBOSE)

```python
print(f'# {BK} Most Negative Bigrams for each of the {K} Most Negative Adverbs\n')

for rank, adverb in enumerate(adv_am.index, start=1):
    print(f'\n## {rank}. *{adverb}*')
    populate_adv_dir(
        adverb,
        bigram_am=polar_bigram_am,
        neg_hits_df=neg_hits,
        pos_hits_df=pos_hits,
        data_dir=TOP_AM_TAG_DIR, 
        rank_by=METRIC_PRIORITY,
        n_bigrams=BK, n_ex=N_EX_PER_BIGRAM,
        verbose=VERBOSE)
```

# 10 Most Negative Bigrams for each of the 8 Most Negative Adverbs


## 1. *necessarily*

|                                       |   `f` |   `dP1` |   `LRC` |   `P1` |     `G2` |   `f2` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `adj_total` |
|:--------------------------------------|------:|--------:|--------:|-------:|---------:|-------:|----------:|------------:|------------:|--------:|-------:|--------------:|
| **NEGany~necessarily_true**           | 3,232 |    0.50 |    6.34 |   1.00 | 4,360.41 |  3,242 |  1,621.00 |    1,611.00 |        0.50 |    0.00 |   0.00 |        35,146 |
| **NEGany~necessarily_better**         | 1,887 |    0.50 |    6.33 |   1.00 | 2,587.87 |  1,889 |    944.50 |      942.50 |        0.50 |    0.00 |   0.00 |        49,936 |
| **NEGany~necessarily_indicative**     | 1,389 |    0.50 |    6.29 |   1.00 | 1,925.87 |  1,389 |    694.50 |      694.50 |        0.50 |    0.00 |   0.00 |         2,319 |
| **NEGany~necessarily_bad**            | 2,059 |    0.50 |    5.75 |   1.00 | 2,751.69 |  2,068 |  1,034.00 |    1,025.00 |        0.50 |    0.00 |   0.00 |       119,777 |
| **NEGany~necessarily_easy**           |   909 |    0.50 |    5.67 |   1.00 | 1,260.27 |    909 |    454.50 |      454.50 |        0.50 |    0.00 |   0.00 |       109,374 |
| **NEGany~necessarily_wrong**          |   762 |    0.50 |    5.18 |   1.00 | 1,042.56 |    763 |    381.50 |      380.50 |        0.50 |    0.00 |   0.00 |        21,208 |
| **NEGany~necessarily_representative** |   487 |    0.50 |    4.76 |   1.00 |   675.16 |    487 |    243.50 |      243.50 |        0.50 |    0.00 |   0.00 |         2,559 |
| **NEGany~necessarily_surprising**     |   340 |    0.50 |    4.23 |   1.00 |   471.36 |    340 |    170.00 |      170.00 |        0.50 |    0.00 |   0.00 |        18,886 |
| **NEGany~necessarily_enough**         |   279 |    0.50 |    3.93 |   1.00 |   386.79 |    279 |    139.50 |      139.50 |        0.50 |    0.00 |   0.00 |        27,343 |
| **NEGmir~necessarily_wrong**          |   211 |    0.50 |    3.63 |   1.00 |   292.58 |    211 |    105.50 |      105.50 |        0.50 |    0.00 |   0.00 |         8,526 |
| **NEGany~necessarily_happy**          |   224 |    0.50 |    3.60 |   1.00 |   310.54 |    224 |    112.00 |      112.00 |        0.50 |    0.00 |   0.00 |        47,131 |
| **NEGmir~necessarily_true**           |    53 |    0.50 |    1.47 |   1.00 |    73.48 |     53 |     26.50 |       26.50 |        0.50 |    0.00 |   0.00 |         2,870 |
| **NEGmir~necessarily_bad**            |    50 |    0.50 |    1.37 |   1.00 |    69.32 |     50 |     25.00 |       25.00 |        0.50 |    0.00 |   0.00 |         4,885 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/ALL-necessarily_10mostNEG-bigrams_AMscores.2024-08-07.md`


|                         |       `N` |      `f1` |   `adv_total` |
|:------------------------|----------:|----------:|--------------:|
| **NEGATED_necessarily** | 6,347,362 | 3,173,681 |        42,916 |
| **NEGMIR_necessarily**  |   583,470 |   291,735 |           993 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/necessarily_MarginalFreqs_2024-08-07.md`


### 1. _necessarily true_


#### 1.1. "necessarily true" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                 |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20070106_0112_68:23-24-25`**         | but at the end of the day , your paycheck comes from the paper , which means it is possible -LRB- though not __``necessarily true``__ -RRB- either that you may occasionally consciously or unconsciously censor yourself , or that one day the paper may stop paying you . |
| **`pcc_eng_22_057.7184_x0916795_16:11-12-13`**  | Usually that means warmer nicer weather , but that 's not __``necessarily true``__ here in Colorado .                                                                                                                                                                       |
| **`pcc_eng_04_059.4251_x0943840_09:3-4-5`**     | This was not __``necessarily true``__ .                                                                                                                                                                                                                                     |
| **`pcc_eng_08_073.8235_x1179073_5:3-4-5`**      | It is not __``necessarily true``__ that spending more money on students - based on per-pupil expenditures - translates into smaller class sizes or better student test scores .                                                                                             |
| **`pcc_eng_07_052.4530_x0831739_141:12-13-14`** | Mark escaped the chaos with little apparent psychic damage , something not __``necessarily true``__ of all of his contemporaries .                                                                                                                                          |
| **`pcc_eng_02_006.5961_x0090435_14:09-10-11`**  | But a newly published study reports that 's not __``necessarily true``__ :                                                                                                                                                                                                  |
| **`pcc_eng_00_039.6945_x0625048_36:5-6-7`**     | Even if that 's not __``necessarily true``__ .                                                                                                                                                                                                                              |
| **`pcc_eng_21_019.1689_x0293334_55:11-12-13`**  | A stream is a cloud , but the converse is not __``necessarily true``__ .                                                                                                                                                                                                    |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_true_ex.md`


#### 1.2. "necessarily true" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                    |
|:---------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_085.8305_x1371157_170:4-5`**   | " That ainit __``necessarily true``__ .                                                                                                                        |
| **`pcc_eng_02_058.1432_x0924332_097:8-9`**   | Some people have argued that it is __``necessarily true``__ that anybody who acts sufficiently like a normal human being must also have conscious experience . |
| **`pcc_eng_25_093.4729_x1496556_034:26-27`** | For example , ye might say it was a mistake for a club tae buy such an ' such a player but that is nae __``necessarily true``__ .                              |
| **`pcc_eng_04_022.4879_x0347027_52:6-7`**    | I took this as a __``necessarily true``__ starting point ; as an axiom , if you wish .                                                                         |
| **`pcc_eng_25_049.3712_x0783107_08:21-22`**  | Right now Donald Trump is being looked at as the most divisive personality in politics today , but is that __``necessarily true``__ ?                          |
| **`pcc_eng_07_050.3684_x0798080_3:8-9`**     | Colonel Muammar Gaddafi proves that this is __``necessarily true``__ .                                                                                         |
| **`pcc_eng_25_047.8675_x0758685_060:3-4`**   | This is __``necessarily true``__ , for since all gifts are made nontaxable , there can be no such thing under the statute as a taxable gift .                  |
| **`pcc_eng_09_014.9779_x0226569_27:3-4`**    | Is that __``necessarily true``__ ?                                                                                                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/pos-necessarily/pos-necessarily_true_ex.md`


### 2. _necessarily better_


#### 2.1. "necessarily better" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                          |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20000831_0184_35:29-30-31`**        | despite the unprecedented economic growth of the last eight years , Gore is trying to tap into the discontent of `` working families '' _ those who are not __``necessarily better``__ off than they were when the Clinton-Gore administration took office in 1993 . |
| **`pcc_eng_18_030.2896_x0474091_02:12-13-14`** | " People are starting to realize that newer and bigger is not __``necessarily better``__ , " he says gazing over the roof-tops .                                                                                                                                     |
| **`pcc_eng_12_002.0690_x0017287_27:18-19-20`** | Our culture is in a constant state of making things faster , bigger , louder , though not __``necessarily better``__ .                                                                                                                                               |
| **`pcc_eng_15_043.1671_x0681767_1:19-20-21`**  | It 's been said that when a baby is born , a bomb goes off ; things are n't __``necessarily better``__ .                                                                                                                                                             |
| **`pcc_eng_21_070.2449_x1118970_19:3-4-5`**    | New is not __``necessarily better``__ .                                                                                                                                                                                                                              |
| **`pcc_eng_18_094.0572_x1507176_22:1-2-3`**    | Not __``necessarily better``__ or worse , but it 's different , " says owner Ergin Iren .                                                                                                                                                                            |
| **`pcc_eng_03_040.5486_x0640854_57:4-5-6`**    | Random Forests is not __``necessarily better``__ than the decision trees algorithm in all scenarios .                                                                                                                                                                |
| **`pcc_eng_09_036.5131_x0574888_23:6-7-8`**    | However , while it is not __``necessarily better``__ than its predecessor , Frozen II still has a lot to offer both children and adults alike , especially with a voice cast as charismatic as the one we are lucky enough to have in Frozen II .                    |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_better_ex.md`


#### 2.2. "necessarily better" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|:---------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_10_081.7489_x1304892_05:3-4`**    | Is it __``necessarily better``__ , though ?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| **`pcc_eng_12_044.0091_x0695348_008:37-38`** | Again , it is still more useless to inquire whether there is any essential connection between the two inequalities ; for this would be only asking , in other words , whether those who command are __``necessarily better``__ than those who obey , and if strength of body or of mind , wisdom or virtue are always found in particular individuals , in proportion to their power or wealth : a question fit perhaps to be discussed by slaves in the hearing of their masters , but highly unbecoming to reasonable and free men in search of the truth . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/pos-necessarily/pos-necessarily_better_ex.md`


### 3. _necessarily indicative_


#### 3.1. "necessarily indicative" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                            |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_19_086.0859_x1374989_46:09-10-11`**  | I realize that its only one lap and not __``necessarily indicative``__ of race pace , but still ??                                                                                                                                                     |
| **`pcc_eng_18_011.6927_x0173116_4:14-15-16`**   | Any references to performance are for illustrative purposes only ; past performance is not __``necessarily indicative``__ of future results .                                                                                                          |
| **`pcc_eng_19_019.3740_x0296477_45:11-12-13`**  | The scrutiny of Trevino 's birth certificates is neither new nor __``necessarily indicative``__ of any wrongdoing .                                                                                                                                    |
| **`pcc_eng_16_054.9764_x0873528_004:12-13-14`** | The historical results set forth in this discussion and analysis are not __``necessarily indicative``__ of trends with respect to any actual or projected future financial performance .                                                               |
| **`pcc_eng_16_056.2970_x0895041_14:22-23-24`**  | You 've got to win one - on- one battles and be better in the areas of the game that are n't __``necessarily indicative``__ of skill . "                                                                                                               |
| **`pcc_eng_21_074.0496_x1180527_062:33-36-37`** | The pro forma information has been included for comparative purposes and is not indicative of results of operations of the Company had the acquisitions occurred as of January 1 , 2019 , nor is it __``necessarily indicative``__ of future results . |
| **`pcc_eng_04_055.4557_x0879520_18:11-12-13`**  | Clapper cautioned that " just being in a database is not __``necessarily indicative``__ of current nefarious behavior . "                                                                                                                              |
| **`pcc_eng_18_088.3062_x1413836_27:6-7-8`**     | Current results and trends are not __``necessarily indicative``__ of future results that we may achieve .                                                                                                                                              |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_indicative_ex.md`


#### 3.2. "necessarily indicative" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*necessarily indicative*" found in **positive** hits.

### 4. _necessarily bad_


#### 4.1. "necessarily bad" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                                               |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_098.1901_x1572511_36:12-13-14`**  | If somebody has a different perspective on it , it 's not __``necessarily bad``__ .                                                                                                                                                                                                                                                                                                       |
| **`pcc_eng_26_008.2907_x0117661_33:5-6-7`**     | Cheap HD cameras are n't __``necessarily bad``__ , but bear in mind that cameras which record onto SD card will have memory restrictions which will limit the amount of recordings your camera can capture .                                                                                                                                                                              |
| **`apw_eng_20020313_1621_21:08-09-10`**         | `` In the longer term that 's not __``necessarily bad``__ for Compaq , '' said Lehman Brothers analyst Dan Niles , noting that Compaq and Texas rival Dell Computer Corp. would have develop even more dominant market share in the PC business if HP scaled back .                                                                                                                       |
| **`pcc_eng_05_033.0308_x0518867_36:08-09-10`**  | A large number of high-quality corporates is n't __``necessarily bad``__ , but at least for now , it does n't appear to offer much diversification from Treasuries , either .                                                                                                                                                                                                             |
| **`pcc_eng_29_008.8856_x0127429_16:13-14-15`**  | " We Bring the Shake " is n't that notable , though not __``necessarily bad``__ , but ultimately a throwaway .                                                                                                                                                                                                                                                                            |
| **`pcc_eng_02_033.3793_x0524075_19:1-5-6`**     | Not that feminism is __``necessarily bad``__ , just that I think it influenced the author 's interpretation of things to a degree .                                                                                                                                                                                                                                                       |
| **`pcc_eng_17_070.7100_x1126357_23:7-8-9`**     | Outsourcing duties to private contractors is not __``necessarily bad``__ .                                                                                                                                                                                                                                                                                                                |
| **`pcc_eng_07_108.10815_x1748320_08:20-21-22`** | I went there on a date once and it was a good date but the food was disappointing ( not __``necessarily bad``__ , just disappointing , we got the " margherita " which was of the common reinterpreted variety that is really a white pie with tomatoes ) , and then the boy turned out later to be both a bit of a jerk and rather disappointing , so I did n't really want to go back . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_bad_ex.md`


#### 4.2. "necessarily bad" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                                                               |
|:---------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_069.5665_x1108979_59:23-24`**  | The film Lars and the Real Girl ( 2007 ) provides a thoughtful answer to the question of whether mental disorders are __``necessarily bad``__ .                                                                                                           |
| **`pcc_eng_27_063.3870_x1008588_22:18-19`**  | The Antitrust Paradox ( 1978 ) , both of which attacked the idea that big business is __``necessarily bad``__ .                                                                                                                                           |
| **`pcc_eng_03_080.3072_x1284348_038:34-35`** | And really , some of the things that I found out for the first time -stories of alcoholism , violence - both physical and emotional , infidelity - are these kinds of secrets __``necessarily bad``__ ?                                                   |
| **`pcc_eng_25_100.6160_x1612141_09:4-5`**    | Are these all __``necessarily bad``__ things ?                                                                                                                                                                                                            |
| **`pcc_eng_14_041.5377_x0655020_33:3-4`**    | It was __``necessarily bad``__ , but it was clear that the band was still honing its craft .                                                                                                                                                              |
| **`pcc_eng_22_025.4697_x0394616_085:5-6`**   | Since personalized dictatorships are __``necessarily bad``__ at admitting fault -- for nothing can be permitted to damage the myth of the omnipotent leader -- China will also likely become less adept at correcting mistakes once it makes them .       |
| **`pcc_eng_27_039.8796_x0628309_07:26-27`**  | The result is that secession is now thought of in Europe as a neutral act to be evaluated on its merits rather than as something __``necessarily bad``__ as Lincoln classified it when he identified it with anarchy and a threat to government as such . |
| **`pcc_eng_07_038.4700_x0605912_018:5-6`**   | ( But is this __``necessarily bad``__ ?                                                                                                                                                                                                                   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/pos-necessarily/pos-necessarily_bad_ex.md`


### 5. _necessarily easy_


#### 5.1. "necessarily easy" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                  |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_10_026.5436_x0412757_04:08-09-10`**  | Yet , in France , it is not __``necessarily easy``__ to find good quality , tasty products that are conducive to a plant- based diet , even in a big city like Paris .                                                                                       |
| **`pcc_eng_02_031.3274_x0490753_08:15-16-17`**  | " Sometimes the process of democratic reform and having a democratic political system is n't __``necessarily easy``__ , but at the end of the day in our view , that is the best way forward , " State Department spokesman Sean Mc Cormack said yesterday . |
| **`pcc_eng_20_038.5242_x0606424_03:12-13-14`**  | Finding that perfect job that matches your skills and interest is n't __``necessarily easy``__ unless you are provided with several options .                                                                                                                |
| **`pcc_eng_10_083.1058_x1326818_156:15-16-17`** | These service dogs are n't perfect , and living with and training them is n't __``necessarily easy``__ .                                                                                                                                                     |
| **`pcc_eng_10_023.5277_x0363897_47:4-5-6`**     | Fresh flowers are n't __``necessarily easy``__ for vendors to come by , either , and plenty of businesses look for them - from wholesalers to small business owners who want to spruce up their space .                                                      |
| **`pcc_eng_19_075.1399_x1197725_11:4-5-6`**     | Playing bagpipes is n't __``necessarily easy``__ , in fact it 's fairly difficult , even for bag pipe veterans of the Drouthy Ducks , that have been playing for years .                                                                                     |
| **`pcc_eng_09_032.1962_x0505042_259:3-4-5`**    | It is not __``necessarily easy``__ because it does require a decision on the part of the counselor , a decision that " my feelings do n't matter , you matter .                                                                                              |
| **`pcc_eng_28_072.1923_x1151612_20:28-30-31`**  | Speech is able to take the good with the bad and is grateful he 's had time to take a step back - even though it has n't been __``necessarily easy``__ .                                                                                                     |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_easy_ex.md`


#### 5.2. "necessarily easy" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*necessarily easy*" found in **positive** hits.

### 6. _necessarily wrong_


#### 6.1. "necessarily wrong" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                          |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_025.6302_x0398157_25:3-4-5`**    | There 's nothing __``necessarily wrong``__ with that strategy , but if everything 's like that then you 're going to lose certain kinds of writing . "                                               |
| **`pcc_eng_12_035.1517_x0552464_30:7-8-9`**    | The committee said these cross-subsidies were not __``necessarily wrong``__ , but made it difficult to make comparisons on charges .                                                                 |
| **`pcc_eng_23_081.0315_x1293147_23:4-5-6`**    | While there 's nothing __``necessarily wrong``__ with choosing to be listed on an app like Weedmaps , the ideal route for building your business and your brand will always be developing your own . |
| **`pcc_eng_23_034.0648_x0533840_09:4-5-6`**    | ( They 're not __``necessarily wrong``__ . )                                                                                                                                                         |
| **`pcc_eng_15_105.6844_x1692271_034:4-5-6`**   | But Pollan 's not __``necessarily wrong``__ .                                                                                                                                                        |
| **`pcc_eng_16_054.2745_x0862245_12:10-11-12`** | And the thing is , the Johnson thesis is n't __``necessarily wrong``__ .                                                                                                                             |
| **`pcc_eng_12_009.5130_x0137705_28:4-5-6`**    | The principle is not __``necessarily wrong``__ although obviously it would be the big companies which would benefit more than smaller ones .                                                         |
| **`pcc_eng_16_058.1103_x0924560_025:5-6-7`**   | Not necessarily right and not __``necessarily wrong``__ but very interesting .                                                                                                                       |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_wrong_ex.md`


#### 6.2. "necessarily wrong" under _positive_ polarity


|                                    | `token_str`                                                                                                   |
|:-----------------------------------|:--------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19951006_0083_33:4-5`** | `` Is that __``necessarily wrong``__ ? You can argue both points in terms of employees and Italian industry . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/pos-necessarily/pos-necessarily_wrong_ex.md`


### 7. _necessarily representative_


#### 7.1. "necessarily representative" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                          |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_25_088.4163_x1414686_12:16-17-18`**  | These sites were established for a variety of reasons , but the data collected is not __``necessarily representative``__ of the whole country .                                                                                                                                                                                                                                                                      |
| **`pcc_eng_03_045.3218_x0718004_6:3-4-5`**      | It is not __``necessarily representative``__ of where they will actually be drafted .                                                                                                                                                                                                                                                                                                                                |
| **`nyt_eng_20070312_0121_37:21-22-23`**         | they also acknowledged that they did not have information on suicide attempts , and that the group they studied was not __``necessarily representative``__ of the general population .                                                                                                                                                                                                                               |
| **`pcc_eng_27_027.8893_x0434352_034:20-21-22`** | The league system could dramatically increase the number of rated rounds in a year at a single course , not __``necessarily representative``__ of how a player plays overall ( just how they play on that one course ) .                                                                                                                                                                                             |
| **`pcc_eng_01_095.8247_x1533243_2:10-11-12`**   | The opinions and advocacy expressed on this site are not __``necessarily representative``__ of NRW 's views , or shared by NRW or its staff , including MCZ staff .                                                                                                                                                                                                                                                  |
| **`pcc_eng_03_009.9521_x0144759_35:09-10-11`**  | So the apparent reaction at that meeting was not __``necessarily representative``__ of any sort of consensus among Wilkinsburg residents .                                                                                                                                                                                                                                                                           |
| **`pcc_eng_26_034.6047_x0543237_1:6-7-8`**      | The above two images are NOT __``necessarily representative``__ of the same location along the fence , nor can we be sure that the gardeners did n't make significant changes over time , but the comparison does tend to suggest that this slide was taken early in the lifespan of the Cliff House , likely near the end of the initial construction in 1896 , rather than during one of the subsequent remodels . |
| **`pcc_eng_23_046.1591_x0729668_26:08-09-10`**  | Particularly since those vocal constituent opponents are not __``necessarily representative``__ of the public .                                                                                                                                                                                                                                                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_representative_ex.md`


#### 7.2. "necessarily representative" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*necessarily representative*" found in **positive** hits.

### 8. _necessarily surprising_


#### 8.1. "necessarily surprising" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                            |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_21_060.4235_x0960662_19:3-4-5`**    | It 's not __``necessarily surprising``__ , but as a feminist , as a parent , as a media critic , it feels bleak .                                                                                                                      |
| **`pcc_eng_20_008.4348_x0119852_16:4-5-6`**    | Odd , but not __``necessarily surprising``__ , because cat litter is big business .                                                                                                                                                    |
| **`pcc_eng_05_008.2342_x0117417_19:5-6-7`**    | Although the finding is not __``necessarily surprising``__ to many microbiologists , it does have implications for the search for life across the universe .                                                                           |
| **`pcc_eng_21_071.5938_x1140806_14:6-7-8`**    | But while economic downturns are n't __``necessarily surprising``__ , that does n't make them any easier to deal with .                                                                                                                |
| **`pcc_eng_14_086.1705_x1376679_13:3-4-5`**    | This is n't __``necessarily surprising``__ considering the great size and spread of the bat population , says Yan Xiang , a professor of virology at the University of Texas .                                                         |
| **`pcc_eng_25_045.5510_x0721204_15:11-12-13`** | The former first-round pick 's transformation into a leader is n't __``necessarily surprising``__ .                                                                                                                                    |
| **`pcc_eng_21_097.0581_x1551942_13:22-23-24`** | Combine large numbers of young birds not getting enough to eat with cold temperatures and rough seas , and it 's not __``necessarily surprising``__ that so many are ending up dead or dying on the beaches of the Pacific Northwest . |
| **`pcc_eng_12_065.1866_x1037557_03:17-18-19`** | In Cuba , some who heard about it from state TV called it objectionable , if not __``necessarily surprising``__ .                                                                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_surprising_ex.md`


#### 8.2. "necessarily surprising" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*necessarily surprising*" found in **positive** hits.

### 9. _necessarily enough_


#### 9.1. "necessarily enough" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_001.2721_x0004439_07:34-35-36`** | These are staggering figures , and one might wonder why the Obama administration would allow such a fiscal debacle to tarnish the celebratory passage of a law that has encountered some , but not __``necessarily enough``__ , congressional resistance . |
| **`pcc_eng_19_043.9392_x0693268_47:27-28-29`** | With budgetary restraints limiting the number of dancers Kain can accommodate within the higher - paid ranks of first and second soloist , merit alone is not __``necessarily enough``__ to earn promotion .                                               |
| **`pcc_eng_21_025.8582_x0401768_10:08-09-10`** | In our experience the immunity above is not __``necessarily enough``__ to guarantee that the removals vehicle remains ticketless .                                                                                                                         |
| **`pcc_eng_25_004.1995_x0052297_06:7-8-9`**    | Just paying the planned premium is not __``necessarily enough``__ to keep the coverage healthy .                                                                                                                                                           |
| **`pcc_eng_24_105.5328_x1691256_51:23-24-25`** | There is a major disconnect between major unemployment statistics and the rest of the economy , where even having a job is not __``necessarily enough``__ to support the expected standard of living .                                                     |
| **`pcc_eng_27_002.5799_x0025423_20:16-17-18`** | As we 'll learn later , just rolling out your website in another language is n't __``necessarily enough``__ to make you rank , even if you rank very well in your existing languages .                                                                     |
| **`pcc_eng_05_043.0386_x0680517_13:08-09-10`** | Merely getting information to the public is not __``necessarily enough``__ for professional recognition .                                                                                                                                                  |
| **`pcc_eng_18_092.5680_x1482825_07:4-5-6`**    | All this is n't __``necessarily enough``__ , however , to turn enemies into friends .                                                                                                                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_enough_ex.md`


#### 9.2. "necessarily enough" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*necessarily enough*" found in **positive** hits.

### 10. _necessarily happy_


#### 10.1. "necessarily happy" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_065.4243_x1041377_026:30-31-32`** | The final chapter is told from the perspective of one of the " new people " , and we are suddenly back into more familiar narrative territory - yet not __``necessarily happy``__ to be there , among this species with such capacity for violence . |
| **`pcc_eng_27_022.3080_x0344492_036:24-25-26`** | A runner - up for the MVC Freshman of the Year last season , Rice has not kept it secret that he is n't __``necessarily happy``__ at Drake , but that he is n't going to transfer ... at least not yet .                                             |
| **`pcc_eng_04_100.4058_x1605983_15:19-20-21`**  | But Asena Roy of Lyndeborough , who had n't heard about the new law , said she was n't __``necessarily happy``__ about it .                                                                                                                          |
| **`pcc_eng_21_015.9097_x0240632_14:09-10-11`**  | I was n't miserable , but I was n't __``necessarily happy``__ either .                                                                                                                                                                               |
| **`pcc_eng_06_024.6405_x0382728_32:4-5-6`**     | Both teams are n't __``necessarily happy``__ but they 're content that one team does n't have an advantage over another . "                                                                                                                          |
| **`pcc_eng_28_011.0948_x0163259_7:3-4-5`**      | Neitzsche was n't __``necessarily happy``__ about this .                                                                                                                                                                                             |
| **`pcc_eng_01_103.3519_x1654043_082:10-11-12`** | All the gods looked some what interested , but not __``necessarily happy``__ .                                                                                                                                                                       |
| **`pcc_eng_14_096.9560_x1551300_28:10-11-12`**  | " To be 100 % honest ... I was n't __``necessarily happy``__ with ' Thank Me Later . '                                                                                                                                                               |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_happy_ex.md`


#### 10.2. "necessarily happy" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*necessarily happy*" found in **positive** hits.

### Saving *`necessarily`* Samples

 > in `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/`

1. *necessarily true*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_true_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/pos-necessarily/pos-necessarily_true_99ex~10.csv"`

2. *necessarily better*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_better_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/pos-necessarily/pos-necessarily_better_99ex~2.csv"`

3. *necessarily indicative*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_indicative_99ex~99.csv"`

4. *necessarily bad*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_bad_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/pos-necessarily/pos-necessarily_bad_99ex~9.csv"`

5. *necessarily easy*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_easy_99ex~99.csv"`

6. *necessarily wrong*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_wrong_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/pos-necessarily/pos-necessarily_wrong_99ex~1.csv"`

7. *necessarily representative*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_representative_99ex~99.csv"`

8. *necessarily surprising*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_surprising_99ex~90.csv"`

9. *necessarily enough*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_enough_99ex~77.csv"`

10. *necessarily happy*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/necessarily/neg-necessarily/neg-necessarily_happy_99ex~79.csv"`


## 2. *that*

|                            |    `f` |   `dP1` |   `LRC` |   `P1` |      `G2` |   `f2` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `adj_total` |
|:---------------------------|-------:|--------:|--------:|-------:|----------:|-------:|----------:|------------:|------------:|--------:|-------:|--------------:|
| **NEGany~that_hard**       |  9,948 |    0.50 |    7.63 |   1.00 | 13,590.74 |  9,964 |  4,982.00 |    4,966.00 |        0.50 |    0.00 |   0.00 |        45,416 |
| **NEGany~that_great**      | 11,032 |    0.50 |    7.49 |   1.00 | 15,014.60 | 11,055 |  5,527.50 |    5,504.50 |        0.50 |    0.00 |   0.00 |        45,537 |
| **NEGany~that_different**  |  6,534 |    0.50 |    6.77 |   1.00 |  8,800.64 |  6,556 |  3,278.00 |    3,256.00 |        0.50 |    0.00 |   0.00 |        80,019 |
| **NEGany~that_difficult**  |  5,560 |    0.50 |    6.54 |   1.00 |  7,455.62 |  5,582 |  2,791.00 |    2,769.00 |        0.50 |    0.00 |   0.00 |        61,518 |
| **NEGany~that_bad**        | 16,474 |    0.49 |    6.29 |   0.99 | 21,498.88 | 16,609 |  8,304.50 |    8,169.50 |        0.50 |    0.01 |   0.01 |       119,777 |
| **NEGany~that_common**     |  1,207 |    0.50 |    6.08 |   1.00 |  1,673.49 |  1,207 |    603.50 |      603.50 |        0.50 |    0.00 |   0.00 |        34,621 |
| **NEGany~that_familiar**   |  1,126 |    0.50 |    5.98 |   1.00 |  1,561.17 |  1,126 |    563.00 |      563.00 |        0.50 |    0.00 |   0.00 |        19,650 |
| **NEGany~that_uncommon**   |    802 |    0.50 |    5.49 |   1.00 |  1,111.91 |    802 |    401.00 |      401.00 |        0.50 |    0.00 |   0.00 |         3,193 |
| **NEGany~that_impressed**  |    681 |    0.50 |    5.25 |   1.00 |    944.14 |    681 |    340.50 |      340.50 |        0.50 |    0.00 |   0.00 |        12,269 |
| **NEGany~that_sweet**      |    392 |    0.50 |    4.44 |   1.00 |    543.45 |    392 |    196.00 |      196.00 |        0.50 |    0.00 |   0.00 |        13,492 |
| **NEGmir~that_simple**     |    474 |    0.47 |    3.41 |   0.97 |    555.63 |    487 |    243.50 |      230.50 |        0.49 |    0.00 |   0.00 |         7,580 |
| **NEGmir~that_easy**       |    450 |    0.47 |    3.28 |   0.97 |    517.97 |    464 |    232.00 |      218.00 |        0.48 |    0.00 |   0.00 |         7,897 |
| **NEGmir~that_great**      |    286 |    0.48 |    3.17 |   0.98 |    346.44 |    292 |    146.00 |      140.00 |        0.49 |    0.00 |   0.00 |         2,147 |
| **NEGmir~that_good**       |    447 |    0.43 |    2.56 |   0.93 |    429.34 |    479 |    239.50 |      207.50 |        0.46 |    0.00 |   0.00 |        13,484 |
| **NEGmir~that_big**        |    113 |    0.47 |    1.95 |   0.97 |    127.35 |    117 |     58.50 |       54.50 |        0.48 |    0.00 |   0.00 |         3,083 |
| **NEGmir~that_interested** |     62 |    0.50 |    1.73 |   1.00 |     85.96 |     62 |     31.00 |       31.00 |        0.50 |    0.00 |   0.00 |         2,778 |
| **NEGmir~that_hard**       |     59 |    0.50 |    1.64 |   1.00 |     81.80 |     59 |     29.50 |       29.50 |        0.50 |    0.00 |   0.00 |         2,124 |
| **NEGmir~that_close**      |     60 |    0.48 |    1.40 |   0.98 |     74.36 |     61 |     30.50 |       29.50 |        0.49 |    0.00 |   0.00 |         4,972 |
| **NEGmir~that_difficult**  |     52 |    0.48 |    1.16 |   0.98 |     63.56 |     53 |     26.50 |       25.50 |        0.49 |    0.00 |   0.00 |         4,891 |
| **NEGmir~that_high**       |     41 |    0.50 |    1.03 |   1.00 |     56.84 |     41 |     20.50 |       20.50 |        0.50 |    0.00 |   0.00 |         2,195 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/ALL-that_10mostNEG-bigrams_AMscores.2024-08-07.md`


|                  |       `N` |      `f1` |   `adv_total` |
|:-----------------|----------:|----------:|--------------:|
| **NEGATED_that** | 6,347,362 | 3,173,681 |       166,680 |
| **NEGMIR_that**  |   583,470 |   291,735 |         4,538 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/that_MarginalFreqs_2024-08-07.md`


### 1. _that hard_


#### 1.1. "that hard" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                        |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_27_052.4449_x0831453_05:15-17-18`** | It is all about the sauces , and cooking and making great sauces is n't really __``that hard``__ to do , but it does take a bit of practice and understanding to get it right .    |
| **`pcc_eng_28_015.5136_x0235018_04:18-19-20`** | Being able to create code issues and refactorings with the code the user was typing was really not __``that hard``__ to do .                                                       |
| **`pcc_eng_27_004.2888_x0052775_26:20-22-23`** | These two things are a bit on the technical side when it comes to making video but they are not really __``that hard``__ to understand .                                           |
| **`pcc_eng_03_004.9233_x0063399_156:3-4-5`**   | It 's not __``that hard``__ to know what should be done .                                                                                                                          |
| **`pcc_eng_08_085.9741_x1375671_06:5-6-7`**    | Well , it 's not __``that hard``__ to imagine , right ?                                                                                                                            |
| **`pcc_eng_22_003.9607_x0048039_66:18-19-20`** | But once the fun is gone , once your confidence has been completely undermined , it 's not __``that hard``__ to imagine .                                                          |
| **`pcc_eng_21_091.8142_x1467580_84:30-32-33`** | The thing is , apart from the standard difficulties of running a bed and breakfast ( which is , I 'm sure , enormously challenging ) , this would n't be __``that hard``__ to do . |
| **`pcc_eng_25_047.0611_x0745751_33:4-5-6`**    | " It 's not __``that hard``__ to capture them .                                                                                                                                    |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_hard_ex.md`


#### 1.2. "that hard" under _positive_ polarity


|                                             | `token_str`                                                                                                                      |
|:--------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_038.4420_x0605162_09:10-11`** | I 'm just saying , next time you get __``that hard``__ to reach itch , ... Continue Reading -                                    |
| **`pcc_eng_10_083.7585_x1337376_52:12-13`** | Maybe I 'm just feeling defensive , but is it really __``that hard``__ ?                                                         |
| **`pcc_eng_01_051.2006_x0811185_245:5-6`**  | Will It Really Be __``That Hard``__ To Raise Up And Take                                                                         |
| **`pcc_eng_15_013.1409_x0195867_58:4-5`**   | Is it really __``THAT hard``__ to improve on that ?                                                                              |
| **`pcc_eng_28_054.5422_x0866313_69:5-6`**   | And are brownies really __``that hard``__ to make ?                                                                              |
| **`pcc_eng_26_040.2774_x0635124_23:4-5`**   | Is it really __``that hard``__ for Hollywood to wrap their heads around a zombie story that is n't a shitty action-horror film ? |
| **`pcc_eng_16_001.9183_x0015037_35:4-5`**   | Is it really __``that hard``__ to get up there ?                                                                                 |
| **`pcc_eng_18_100.0808_x1604915_11:25-26`** | I could see her rock hard nipples just waiting to be sucked , my cock felt like it was going to burst it was __``that hard``__ . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_hard_ex.md`


### 2. _that great_


#### 2.1. "that great" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                              |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_004.3211_x0053357_009:3-4-5`**   | It 's not __``that great``__ , but I suppose it could be worse .                                                                                                                                                                                                                                                                                         |
| **`pcc_eng_26_001.0120_x0000197_26:7-8-9`**    | I find that most people are n't __``that great``__ about going back to the Book Crossingwebsite and adding a journal entry , but it 's still nice to know that my already - read books are out there getting read again .                                                                                                                                |
| **`nyt_eng_19990429_0227_24:08-09-10`**        | `` My experience with California wines is n't __``that great``__ , '' he said .                                                                                                                                                                                                                                                                          |
| **`pcc_eng_12_006.1248_x0082565_12:56-57-58`** | We enjoyed a fun day of apple picking in Hood River last weekend so I had lots to choose from and ended up using four varieties , super sweet fuji's , tart granny smith's , flavorful spitzenburgs and winter banana apples which were perfect for apple butter because they became kind of mealy and were n't __``that great``__ to eat on their own . |
| **`pcc_eng_02_044.7622_x0707930_13:12-13-14`** | Passport photos , like driver 's license photos , usually are n't __``that great``__ .                                                                                                                                                                                                                                                                   |
| **`pcc_eng_13_083.5661_x1334825_11:19-21-22`** | Even a lot of the name players , the ones you 've seen have some success , are n't always __``that great``__ .                                                                                                                                                                                                                                           |
| **`pcc_eng_03_017.2123_x0262355_29:7-8-9`**    | But then again , it 's not __``that great``__ of a pic .                                                                                                                                                                                                                                                                                                 |
| **`pcc_eng_15_091.7115_x1466305_12:13-14-15`** | The mounting surface contact area for the M performance rear spoiler is not __``that great``__ , and I really think you should use the betalink adhesive here .                                                                                                                                                                                          |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_great_ex.md`


#### 2.2. "that great" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                        |
|:--------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_016.7923_x0254740_03:36-37`** | While I am sure it 's nice to be memorialized , I have to wonder if having your name be the embodiment of the sound of a giant , gas-filled fat guy exploding is all __``that great``__ .                                          |
| **`nyt_eng_19960720_0230_50:15-16`**        | what Wheatley may be able to do someday is scary ; his potential is __``that great``__ .                                                                                                                                           |
| **`pcc_eng_27_069.7593_x1111689_48:28-29`** | " There are a lot of people who 've had these insanely great experiences , whether they were really __``that great``__ or whether they remember them being __``that great``__ because it was 25 years ago , " the bartender says . |
| **`pcc_eng_28_018.5850_x0284349_48:17-18`** | And by the way , Tom Hardy told me to tell you if home had been __``that great``__ , he would n't have left .                                                                                                                      |
| **`pcc_eng_18_105.7846_x1697656_21:6-7`**   | Is working from home really __``that great``__ ?                                                                                                                                                                                   |
| **`apw_eng_20030122_0578_2:14-15`**         | `` Brian Lara is a world class player ... that can really make __``that great``__ a difference for us , '' Harper said as the camp moved into its second week .                                                                    |
| **`pcc_eng_13_105.1847_x1683596_27:25-26`** | I wanted to place an order right away , but my brain started telling me to dig deeper and see if they 're really __``that great``__ .                                                                                              |
| **`pcc_eng_15_023.2000_x0358751_05:11-12`** | Honestly , I doubted that the toys or treats were __``that great``__ of quality .                                                                                                                                                  |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_great_ex.md`


### 3. _that different_


#### 3.1. "that different" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                            |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_099.9083_x1598832_14:4-6-7`**     | That 's actually not all __``that different``__ than how Legacy goes in Mt G currently .                                                                                                               |
| **`pcc_eng_19_014.8379_x0223479_159:17-18-19`** | But , in terms of other key points the conversation we 're having here is really not __``that different``__ than what you see in Scandinavia .                                                         |
| **`pcc_eng_14_035.0717_x0550321_33:20-21-22`**  | Finally , computational models for functional genomics in higher organisms , particularly for human health and populations are often not __``that different``__ from those for microbial communities . |
| **`pcc_eng_14_087.6551_x1400736_41:33-35-36`**  | Or , to my mind , he was hamming it up a bit , but still , it got me thinking that vis a vis democracy , the role of newspapers is not all __``that different``__ than the role of public libraries .  |
| **`pcc_eng_27_105.5611_x1691378_15:4-5-6`**     | This one is not __``that different``__ from others of its type but there are a few good things to consider .                                                                                           |
| **`pcc_eng_22_008.7142_x0124436_22:16-17-18`**  | For me , the greatest " secret " about contemporary womanhood is that it is not __``that different``__ from contemporary " manhood . "                                                                 |
| **`pcc_eng_12_038.0757_x0599631_17:17-20-21`**  | I mean , the stuff that comes in a nice box all full of stuff ca n't really be __``that different``__ and making your own sounds like such a hassle right ?                                            |
| **`pcc_eng_14_038.7135_x0609224_13:09-11-12`**  | In a sense , our elected representatives are not all __``that different``__ from the North Koreans .                                                                                                   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_different_ex.md`


#### 3.2. "that different" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                   |
|:--------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20020508_0249_45:6-7`**          | but are the new machines __``that different``__ from their predecessors ? The biggest change is that some models now have a touchpad in addition to the red eraserlike TrackPoint button that sits between the G and H keys . |
| **`pcc_eng_08_005.3943_x0071094_46:6-7`**   | - Are tobacco companies really __``that different``__ from alcohol companies or ultra-processed food companies ?                                                                                                              |
| **`pcc_eng_07_045.0507_x0712156_05:4-5`**   | Are they really __``that different``__ from other , more fleet -footed species ?                                                                                                                                              |
| **`pcc_eng_21_001.4588_x0007394_06:09-10`** | Sceptics often query how a brush can be __``that different``__ to using a good cleanser , or even wipes .                                                                                                                     |
| **`pcc_eng_23_051.9611_x0823228_08:13-14`** | " People keep asking us , ' Is this situation really all __``that different``__ or not ? ' "                                                                                                                                  |
| **`pcc_eng_17_053.5434_x0848948_04:5-6`**   | Is a picture really __``that different``__ from a personal note ?                                                                                                                                                             |
| **`pcc_eng_15_012.6223_x0187567_20:15-16`** | What has your experiences with an online audience been like , and how was __``that different``__ than the audience in the gallery ?                                                                                           |
| **`pcc_eng_15_039.7498_x0626692_38:4-5`**   | Is this really __``that different``__ than the previous Liberal government ?                                                                                                                                                  |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_different_ex.md`


### 4. _that difficult_


#### 4.1. "that difficult" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                          |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_009.1415_x0131452_013:17-18-19`** | It remains one of the funniest moments of my editing career , because it really was n't __``that difficult``__ to do .                                                                                               |
| **`pcc_eng_19_026.2798_x0408152_41:4-5-6`**     | And it is not __``that difficult``__ .                                                                                                                                                                               |
| **`pcc_eng_16_058.7796_x0935459_067:4-5-6`**    | Physically it was n't __``that difficult``__ because the training was there .                                                                                                                                        |
| **`pcc_eng_09_085.6899_x1370118_58:12-13-14`**  | Well , you might be surprised to learn that it is n't __``that difficult``__ , and with a bit of persistence and smart thinking , you could soon be living a life of freedom anywhere in the world that you choose . |
| **`pcc_eng_15_019.2631_x0294955_006:11-12-13`** | In that area , looking for a teaching job is not __``that difficult``__ .                                                                                                                                            |
| **`pcc_eng_03_095.7224_x1533648_44:6-7-8`**     | Keeping your audience active is n't __``that difficult``__ .                                                                                                                                                         |
| **`pcc_eng_11_068.3642_x1090448_02:07-09-10`**  | Creating 3D texts in Photoshop may not be __``that difficult``__ , but creating a whole background theme or a story around those texts may be an interesting thing to do .                                           |
| **`pcc_eng_22_088.7841_x1418915_0081:3-4-5`**   | It 's not __``that difficult``__ , right ?                                                                                                                                                                           |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_difficult_ex.md`


#### 4.2. "that difficult" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                   |
|:--------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_01_083.6441_x1336450_11:15-16`** | House Democrats want to pass tighter gun restrictions , but Senate Republicans could make __``that difficult``__ . >>                                                                                                                                         |
| **`pcc_eng_27_070.3360_x1121070_09:24-25`** | I know a lot of people might try to ignore Junior Eurovision , but the 2009 winner , Ralf Mackenbach , might make __``that difficult``__ in the next few years .                                                                                              |
| **`pcc_eng_03_099.7857_x1599376_43:8-9`**   | But a tight mortgage market has made __``that difficult``__ .                                                                                                                                                                                                 |
| **`pcc_eng_00_096.5968_x1546152_49:45-46`** | He originally intended to write the outline for the second film and script it with author Suzanne Collins ( they teamed to do a lot of writing on the first film ) , but the post production schedule on The Hunger Games has made __``that difficult``__ .   |
| **`pcc_eng_26_076.1956_x1215483_31:7-8`**   | Is the situation in France really __``that difficult``__ for entrepreneurs ?                                                                                                                                                                                  |
| **`pcc_eng_08_095.1659_x1524587_20:20-21`** | Ms Morris has endured the full glare of the media spotlight in recent days and admitted she had found __``that difficult``__ .                                                                                                                                |
| **`pcc_eng_09_106.3551_x1704975_04:15-16`** | In the years since , lawmakers on both sides of the aisle have made __``that difficult``__ .                                                                                                                                                                  |
| **`pcc_eng_15_041.6364_x0657084_02:4-5`**   | I 'm finding __``that difficult``__ because , on a typical night , she takes a dump in our master bathroom with the door open , saunters over to our bed with a few open sores on her face from picking her zits , rips a fart , and comes at me for a kiss . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_difficult_ex.md`


### 5. _that bad_


#### 5.1. "that bad" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                           |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_04_108.01487_x1731738_19:10-11-12`** | ( Hint : ABC says the procedure itself is not __``that bad``__ , because you 're under anesthesia . )                                                                                                                                 |
| **`pcc_eng_10_075.2062_x1199345_273:21-22-23`** | Ok , so the kid is actually just scared , and petty , and a kid , and he 's not __``that bad``__ .                                                                                                                                    |
| **`pcc_eng_07_001.8763_x0014121_20:08-09-10`**  | But , in fact , it was n't __``that bad``__ at all . "                                                                                                                                                                                |
| **`pcc_eng_21_102.1612_x1634668_186:09-10-11`** | Maybe Baby was right , getting out was n't __``that bad``__ .                                                                                                                                                                         |
| **`pcc_eng_08_077.0010_x1230510_46:11-12-13`**  | What 's more , for a first attempt it 's not __``that bad``__ at all .                                                                                                                                                                |
| **`pcc_eng_02_046.4046_x0734558_10:12-14-15`**  | But I did n't mind it of course thinking it would n't be __``that bad``__ .                                                                                                                                                           |
| **`pcc_eng_07_029.3755_x0458959_37:11-12-13`**  | We can become convinced using cognitive dissonance that smoking is not __``that bad``__ and our new healthy habits - such as yoga - are not bringing us happiness , but our old habits will because our memories of them were happy . |
| **`apw_eng_20021215_0326_11:11-13-14`**         | `` Now I am all the happier that it is n't all __``that bad``__ .                                                                                                                                                                     |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_bad_ex.md`


#### 5.2. "that bad" under _positive_ polarity


|                                             | `token_str`                                                                                                        |
|:--------------------------------------------|:-------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19950107_0168_18:13-14`**        | but are the 49ers really THAT good , or are the Bears __``that bad``__ ?                                           |
| **`nyt_eng_19960808_0388_5:5-6`**           | can it really be __``that bad``__ out there ?                                                                      |
| **`pcc_eng_04_025.0830_x0389169_07:11-12`** | Are you sure loud squealing and grinding noises are really __``that bad``__ ?                                      |
| **`pcc_eng_00_101.7264_x1629060_16:24-25`** | On Monday evening I did just one hour of work on my PC and felt worse for it all week - it is __``that bad``__ .   |
| **`pcc_eng_12_023.6021_x0365708_13:4-5`**   | Since smoking is __``that bad``__ , why not ban smoking all together ?                                             |
| **`pcc_eng_04_090.5874_x1447182_070:4-5`**  | Are friends really __``that bad``__ of a thing to have ?                                                           |
| **`pcc_eng_17_056.5319_x0897149_14:5-6`**   | If they wanted it __``that bad``__ , they could come take it .                                                     |
| **`pcc_eng_04_064.6762_x1028681_08:4-5`**   | Is it really __``that bad``__ if my 18 - month -old watches a couple of episodes of the Twirlywoos before dinner ? |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_bad_ex.md`


### 6. _that common_


#### 6.1. "that common" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                     |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_006.1420_x0082868_36:4-5-6`**     | Ciguatoxin poisoning is n't __``that common``__ , there are an average of about 40 cases a year reported to the CDC , and people almost always recover from it without permanent aftereffects . |
| **`pcc_eng_16_085.2146_x1363092_371:11-12-13`** | You know just three or four , so it 's not __``that common``__ where jurors are allowed to ask questions .                                                                                      |
| **`pcc_eng_04_101.5123_x1623794_22:10-11-12`**  | It 's not something new , but it 's not __``that common``__ for employees to work remotely or at home , instead of at the office .                                                              |
| **`pcc_eng_18_035.6892_x0561522_03:4-5-6`**     | Hail storms are not __``that common``__ in our area and when we do get a storm , the size of the hail ball is typically less than one - inch in diameter .                                      |
| **`pcc_eng_18_046.0912_x0729748_27:4-5-6`**     | Female Hardhead - not __``that common``__ here .                                                                                                                                                |
| **`pcc_eng_26_036.8791_x0579992_23:20-21-22`**  | He also said the team has seen a lot of children with asthma caused by smoking , something else not __``that common``__ in the United States .                                                  |
| **`pcc_eng_14_007.7571_x0109185_49:7-8-9`**     | " High- volume " requesters are not __``that common``__ .                                                                                                                                       |
| **`pcc_eng_28_041.1269_x0649067_15:3-4-5`**     | It 's not __``that common``__ over here .                                                                                                                                                       |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_common_ex.md`


#### 6.2. "that common" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*that common*" found in **positive** hits.

### 7. _that familiar_


#### 7.1. "that familiar" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_18_006.8766_x0095198_40:41-42-43`** | I 'm still in the process of reading it ( too many other things I have to do , alas ) , but I would definitely encourage anyone interested in regency romance to read this , especially if they are not __``that familiar``__ with the realities of life at that time .                                                                                                                                                                                                                                                                |
| **`pcc_eng_19_047.1919_x0745558_08:7-8-9`**    | Most researchers in the audience were not __``that familiar``__ with the blockchain , but picked up the concepts very quickly and asked lots of insightful questions .                                                                                                                                                                                                                                                                                                                                                                 |
| **`pcc_eng_03_098.3167_x1575639_163:5-6-7`**   | What if you are not __``that familiar``__ with the area ?                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| **`pcc_eng_08_072.2829_x1154162_20:5-6-7`**    | Even if you 're not __``that familiar``__ with art , you probably know what Andy Warhol 's Campbell 's Soup cans look like .                                                                                                                                                                                                                                                                                                                                                                                                           |
| **`pcc_eng_00_037.9442_x0596757_08:36-37-38`** | Our music group is centred around Tim on guitar ( who actually is in charge of the music services here at St. Michaels ) , Allyson on leading vocals - especially useful if we are not __``that familiar``__ with a new song ! , and Chris when he is taking the service , on Piano .                                                                                                                                                                                                                                                  |
| **`pcc_eng_08_072.3167_x1154696_11:49-50-51`** | Then , there 's the whole audio deal - I 'm fairly capable when it comes to using i Tunes ( like relocating the i Tunes Library ) , & other media players , etc. , but - my technician , while a nice guy , just is n't __``that familiar``__ w/ these kinds of software , unfortunately , which makes things difficult - i Tunes tells me I do n't have " rights " to do things like relocate the Library , set the " default media player " , etc. , but he does n't always know a whole lot about how to do these kinds of things . |
| **`pcc_eng_04_076.1566_x1213787_07:13-14-15`** | That made them legendary , even to people who at first were n't __``that familiar``__ with their catalog , " said Greg Gonzalez of the young , Grammy - winning Latino - funk fusion band Grupo Fantasma .                                                                                                                                                                                                                                                                                                                             |
| **`pcc_eng_22_083.5293_x1334009_03:22-23-24`** | The names of those competing for playing time and / or just a spot on the 53 - man roster are hardly __``that familiar``__ : Brice Butler , J.J. Nelson , Chad Williams , Greg Little , Christian Kirk and Trent Sherfield , among others .                                                                                                                                                                                                                                                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_familiar_ex.md`


#### 7.2. "that familiar" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*that familiar*" found in **positive** hits.

### 8. _that uncommon_


#### 8.1. "that uncommon" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                       |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19981021_0449_14:4-5-6`**            | `` It 's not __``that uncommon``__ . ''                                                                                                                                                                                                                                                                                                           |
| **`pcc_eng_08_047.1282_x0746667_043:53-54-55`** | " I interviewed an older student who has her own recovery community off campus , but she said that when you are younger and you 're realizing you have addiction issues , it is really easy to blend in or get away with dangerously addictive behavior because things like binge drinking are n't __``that uncommon``__ on campus , " she said . |
| **`pcc_eng_24_071.1969_x1135485_037:6-7-8`**    | Perhaps such a practice was not __``that uncommon``__ in Japan , especially during the Middle Ages .                                                                                                                                                                                                                                              |
| **`pcc_eng_02_097.0985_x1553718_009:11-12-13`** | The golden retriever has light colored eyes , which is n't __``that uncommon``__ .                                                                                                                                                                                                                                                                |
| **`pcc_eng_25_043.9642_x0695622_28:11-12-13`**  | After additional research , I found that it 's actually not __``that uncommon``__ .                                                                                                                                                                                                                                                               |
| **`pcc_eng_17_001.9533_x0015379_06:5-6-7`**     | However , it is n't __``that uncommon``__ for a defective product to actually be a defective medical device .                                                                                                                                                                                                                                     |
| **`pcc_eng_25_084.7209_x1355003_04:15-17-18`**  | Dinosaur tracks ranging from single or partial imprints to trackways of consecutive prints are not all __``that uncommon``__ in the area .                                                                                                                                                                                                        |
| **`pcc_eng_18_008.9083_x0127940_11:19-20-21`**  | Early morning FBI arrests for those who threaten national security based on wiretaps and email interceptions are now not __``that uncommon``__ , but you can walk the streets confident that you are not on camera the whole time , and the innocent seem to have relatively little to fear .                                                     |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_uncommon_ex.md`


#### 8.2. "that uncommon" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*that uncommon*" found in **positive** hits.

### 9. _that impressed_


#### 9.1. "that impressed" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                       |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_006.7749_x0093157_281:4-5-6`**    | " I was n't __``that impressed``__ with him , and he knew it . "                                                                                                                                                                                                                  |
| **`pcc_eng_05_007.6467_x0107931_32:4-5-6`**     | ( I was n't __``that impressed``__ by their sushi or bentos though .                                                                                                                                                                                                              |
| **`pcc_eng_13_002.8725_x0030044_92:5-6-7`**     | The TV critic was n't __``that impressed``__ with " 24 ' s " new event series , noting that it " does n't in any way reinvent the show . "                                                                                                                                        |
| **`pcc_eng_01_100.4603_x1607747_30:12-13-14`**  | I tried this recipe for mac-n-cheese and quite frankly I was n't __``that impressed``__ .                                                                                                                                                                                         |
| **`pcc_eng_22_084.2291_x1345343_079:5-6-7`**    | Frankly , I 'm not __``that impressed``__ that we keep buying new reading programs , because in the ten years I 've been there , the percentage of kids reading on grade level for both 9th and 10th grades has been relatively static , regardless of the program we 're using . |
| **`pcc_eng_29_086.0294_x1373438_096:18-19-20`** | If you 're thinking what we thought when we first saw this image , you 're probably not __``that impressed``__ because you 've managed a pretty similar photograph while attempting to use the panoramic setting on your i Phone camera while the dog was around .                |
| **`pcc_eng_17_006.1973_x0084068_133:08-09-10`** | First bite , and honestly I was n't __``that impressed``__ .                                                                                                                                                                                                                      |
| **`pcc_eng_27_027.9819_x0435828_25:10-12-13`**  | Playing this game on a emulator you probably would not be __``that impressed``__ .                                                                                                                                                                                                |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_impressed_ex.md`


#### 9.2. "that impressed" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*that impressed*" found in **positive** hits.

### 10. _that sweet_


#### 10.1. "that sweet" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                            |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_032.9392_x0516029_132:3-4-5`**   | - Is n't __``that sweet``__ !                                                                                                                                          |
| **`pcc_eng_07_106.0288_x1697593_16:4-6-7`**    | And this is n't even __``that sweet``__ , really .                                                                                                                     |
| **`pcc_eng_03_090.8626_x1455152_185:2-3-4`**   | Is n't __``that sweet``__ ?!                                                                                                                                           |
| **`pcc_eng_04_004.0777_x0049904_13:6-7-8`**    | Sweet girls ( well , not __``that sweet``__ ) with wild hair and long , adolescent limbs .                                                                             |
| **`pcc_eng_05_032.0542_x0503098_47:2-3-4`**    | Is n't __``that sweet``__ that a leader we should be so lucky to have has such kindness in his heart for such a cuddly little killer ?                                 |
| **`nyt_eng_20070403_0207_17:12-14-15`**        | sweet potatoes , without the benefit of brown sugar , are n't really __``that sweet``__ -- and they get gummy on the palate .                                          |
| **`pcc_eng_08_042.9278_x0678513_02:27-28-29`** | Kief is infused into oil for the cake and cannabutter is used for a deliciously easy medicated frosting for a double dose THC cupcake that is not __``that sweet``__ . |
| **`pcc_eng_23_098.0599_x1568749_20:3-4-5`**    | ( Was n't __``that sweet``__ of him ? ? )                                                                                                                              |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_sweet_ex.md`


#### 10.2. "that sweet" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*that sweet*" found in **positive** hits.

### 11. _that simple_


#### 11.1. "that simple" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                      |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_067.7597_x1078856_07:15-17-18`**  | Even if this is the correct interpretation of the election results , it may not be __``that simple``__ .                                                                                                                         |
| **`pcc_eng_13_002.4450_x0023215_04:16-17-18`**  | All the clues in this puzzle leads to this result , I know it 's not __``that simple``__ , but clearly the benifits the USA will get through the continuity of wars in the middle - east is much much more when there 's peace . |
| **`pcc_eng_04_053.5250_x0848509_12:17-18-19`**  | We pretend that our political competition is between Republicans and Democrats , however , it is not __``that simple``__ .                                                                                                       |
| **`pcc_eng_21_078.2869_x1249011_32:5-6-7`**     | Unfortunately many decisions are not __``that simple``__ !                                                                                                                                                                       |
| **`pcc_eng_08_002.5502_x0025206_07:6-8-9`**     | Of course , things ca n't be __``that simple``__ .                                                                                                                                                                               |
| **`pcc_eng_22_005.7331_x0076455_04:16-17-18`**  | But the caveat is determination of price of any commodity / product / service is not __``that simple``__ .                                                                                                                       |
| **`pcc_eng_21_097.4817_x1558783_015:08-10-11`** | I tried to explain that it was n't quite __``that simple``__ but is it just __``that simple``__ ?                                                                                                                                |
| **`pcc_eng_11_064.5245_x1028102_14:19-21-22`**  | But the math behind it -- and the conclusions Wolf and his team draw from it -- may not be __``that simple``__ .                                                                                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_simple_ex.md`


#### 11.2. "that simple" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                 |
|:--------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_10_085.4154_x1364373_21:5-6`**   | Could forgiveness really be __``that simple``__ ?                                                                                                                                                           |
| **`pcc_eng_07_039.3016_x0619389_034:4-5`**  | It could be __``that simple``__ .                                                                                                                                                                           |
| **`nyt_eng_20030109_0235_62:4-5`**          | it was just __``that simple``__ , he said .                                                                                                                                                                 |
| **`nyt_eng_19970709_0950_9:4-5`**           | it 's really __``that simple``__ .                                                                                                                                                                          |
| **`pcc_eng_00_077.1063_x1230375_01:5-6`**   | if only life were __``that simple``__                                                                                                                                                                       |
| **`pcc_eng_05_011.9332_x0177280_23:35-36`** | What is more , we are available for you throughout the city and regardless of your current location you can contact us to arrange the details of your exceptional date - it is really __``that simple``__ ! |
| **`nyt_eng_19960523_0291_20:3-4`**          | it 's __``that simple``__ , '' Gingrich said in the Sept. 18 issue of `` Time '' magazine .                                                                                                                 |
| **`pcc_eng_18_104.4262_x1675683_27:5-6`**   | Yes , it 's __``that simple``__ really .                                                                                                                                                                    |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_simple_ex.md`


### 12. _that easy_


#### 12.1. "that easy" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                       |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20050319_0197_96:11-12-13`**         | i think it will get done , but it 's not __``that easy``__ , and completing it is going to require a good deal of work . ''                                                                                                                                                                                                                       |
| **`pcc_eng_28_011.8739_x0176131_73:18-19-20`**  | And it does n't mean that you 're not trying as hard , but it 's just not __``that easy``__ .                                                                                                                                                                                                                                                     |
| **`pcc_eng_24_104.6213_x1676600_03:3-4-5`**     | It 's not __``that easy``__ at all !                                                                                                                                                                                                                                                                                                              |
| **`pcc_eng_23_088.3629_x1411827_142:32-33-34`** | When you 're getting associated with a business that can offer quality providers at affordable cost , you are going to realize that junk removal about completion of your job was never __``that easy``__ and that affordable .                                                                                                                   |
| **`pcc_eng_21_020.9455_x0322185_605:3-7-8`**    | Of course nothing is ever " __``that easy``__ " so I anticipate some kind of hiccup .                                                                                                                                                                                                                                                             |
| **`pcc_eng_06_106.9826_x1714476_4:12-13-14`**   | You r stuck with him or her in ur heart , not __``that easy``__ to be replaced .                                                                                                                                                                                                                                                                  |
| **`pcc_eng_00_033.0072_x0517286_15:3-4-5`**     | It 's not __``that easy``__ though , as when a sound chimes while doing the riddle ( except for the pilot which was a bit different ) a set of gates on the race course slowly start moving towards - yes , you guessed it , a pack of excited hounds , who will race to tear that backpack off them before they can make it to the finish line . |
| **`pcc_eng_07_023.3850_x0361975_05:6-7-8`**     | Riding for several hours is not __``that easy``__ , so you should be proud of being in safe hands .                                                                                                                                                                                                                                               |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_easy_ex.md`


#### 12.2. "that easy" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                |
|:--------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_026.6395_x0414138_09:27-28`** | Simply click on this link , and register , vote for THE CANDY SHOW under the category Best TV series supporting Aboriginal Music , it 's __``that easy``__ .               |
| **`pcc_eng_06_082.5343_x1318565_06:6-7`**   | If only the choice were __``that easy``__ .                                                                                                                                |
| **`pcc_eng_23_083.6484_x1335628_18:3-4`**   | It 's __``that easy``__ and it 's that important .                                                                                                                         |
| **`pcc_eng_25_064.9015_x1035176_023:7-8`**  | Do you think it 'll be __``that easy``__ ?                                                                                                                                 |
| **`pcc_eng_27_028.9791_x0451755_11:5-6`**   | Scallion pancakes really are __``that easy``__ to make home .                                                                                                              |
| **`pcc_eng_02_068.2741_x1088149_22:7-8`**   | Every dollar counts and it 's __``that easy``__ .                                                                                                                          |
| **`pcc_eng_16_095.8676_x1535682_29:5-6`**   | Yes , it is __``that easy``__ !                                                                                                                                            |
| **`nyt_eng_20070928_0183_52:24-25`**        | `` Obviously making $ 30,000 to $ 50,000 is an opportunity everyone would like to get in on , but if it were __``that easy``__ , everyone would be doing it , '' he said . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_easy_ex.md`


### 13. _that good_


#### 13.1. "that good" under _negative_ polarity


|                                                  | `token_str`                                                                                                                                                                                                             |
|:-------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_07_059.0822_x0938727_46:14-15-16`**   | If you paid attention in chemestry in school you know that it is not __``that good``__ of an idea to mix acids and bases as this will lead to a fierce and foggy reaction .                                             |
| **`pcc_eng_23_008.7735_x0125487_094:08-09-10`**  | As a society , we 're often not __``that good``__ at allowing people to withdraw for periods of time to lick their wounds .                                                                                             |
| **`pcc_eng_23_080.5657_x1285642_25:12-13-14`**   | Makes me feel as if either A ) 286 titles were not __``that good``__ ( which is not the case ) or B )                                                                                                                   |
| **`pcc_eng_14_007.6200_x0107004_0683:14-16-17`** | Wish I could count on 'em being stupid , but our luck 's never been __``that good``__ .                                                                                                                                 |
| **`pcc_eng_02_006.1980_x0084057_4:14-15-16`**    | Cooked it once or twice when we were in Australia but response was not __``that good``__ .                                                                                                                              |
| **`pcc_eng_25_009.8187_x0142865_22:27-28-29`**   | I have been using this tent also as a single person tent , and I have been very satisfied , especially when the weather conditions are not __``that good``__ and I have to stay in the tent watching the rain to fall . |
| **`pcc_eng_09_082.9873_x1326322_18:27-28-29`**   | He was 31 at the time of the trade , had proven he could be quite stubborn when dealing with non-premiere franchises and he 's just not __``that good``__ any more .                                                    |
| **`pcc_eng_19_085.9590_x1372919_21:07-09-10`**   | Try giving up something that may not be __``that good``__ for you and replacing it with something healthier !                                                                                                           |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_good_ex.md`


#### 13.2. "that good" under _positive_ polarity


|                                              | `token_str`                                                                                                                                 |
|:---------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20000110_0180_22:8-9`**           | if plastic surgery can make her look __``that good``__ , I figure it could make me look like Cindy Crawford or Kathie Lee , at least .      |
| **`nyt_eng_19960219_0276_27:5-6`**           | `` When you 're __``that good``__ , you always think that you can do it .                                                                   |
| **`pcc_eng_25_065.2953_x1041585_02:16-17`**  | Some people wait up to three hours to eat their delicious pizza ; it is __``that good``__ !                                                 |
| **`nyt_eng_20060502_0101_52:3-4`**           | it 's __``that good``__ .                                                                                                                   |
| **`pcc_eng_17_045.8002_x0723668_015:20-21`** | Unfortunately , conspiracy theories are best served shadowy and far-reaching -- this is too targeted a conspiracy to be __``that good``__ . |
| **`pcc_eng_07_015.5828_x0236011_48:4-5`**    | He should be __``that good``__ .                                                                                                            |
| **`nyt_eng_20100906_0077_11:24-25`**         | `` Of all the shows we 've done , the thing I hear the most is , ` Were those vegan cupcakes really __``that good``__ ? '' ' Nelson said .  |
| **`pcc_eng_14_108.08563_x1744925_10:4-5`**   | The Rizzo is __``that good``__ !                                                                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_good_ex.md`


### 14. _that big_


#### 14.1. "that big" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                            |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_25_044.5414_x0704936_093:5-6-7`**    | And it 's really not __``that big``__ of a deal for me to be blogging about it .                                                                                                                                                                       |
| **`pcc_eng_15_099.6417_x1594177_34:17-18-19`**  | " Some of my patients will say to me , ' Well , 5 % is not __``that big``__ ; I 'll take the risk , ' " says Boolbol , who was not involved in either of the new reviews .                                                                             |
| **`pcc_eng_test_1.3001_x04824_32:5-6-7`**       | 8 . I am not __``that big``__ on online social networking .                                                                                                                                                                                            |
| **`pcc_eng_27_021.5105_x0331653_090:20-21-22`** | I also realized that , if that was the worst possible situation I could imagine , it probably was n't __``that big``__ of a deal , and I should just do it .                                                                                           |
| **`pcc_eng_02_081.8633_x1307487_05:33-35-36`**  | A nonpartisan state panel of tax and economic experts on Tuesday gave its blessing to significantly improved revenue projections which suggest that Maine 's $ 1 billion - plus budget gap may not be __``that big``__ after all .                     |
| **`pcc_eng_29_002.5888_x0025571_26:13-14-15`**  | That was n't going to happen and in 1980 the controversy was n't __``that big``__ !                                                                                                                                                                    |
| **`pcc_eng_22_081.7667_x1305464_22:22-23-24`**  | Dakota L. Wood , a retired Marine lieutenant colonel , said the size of the cuts ordered by Gates were " not __``that big``__ of a deal . "                                                                                                            |
| **`pcc_eng_13_031.5648_x0494224_22:10-11-12`**  | Talk that China will sell our Treasuries is actually not __``that big``__ of a deal as they only own 5 % of them ( equivalent to 25 % of the Fed 's QE acquisition of bonds , so rather small actually ) and the rest of the world needs to buy them . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_big_ex.md`


#### 14.2. "that big" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                         |
|:---------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_049.4828_x0783457_077:09-10`** | I often thought , if only I was __``that big``__ , I could be that tough .                                                                                                          |
| **`pcc_eng_07_093.6488_x1497339_13:7-8`**    | Is foregoing wedding night sex actually __``that big``__ of a deal for your relationship ?                                                                                          |
| **`apw_eng_20010606_1078_28:3-4`**           | a stone __``that big``__ , that valuable , the government figured , must have been found on government land .                                                                       |
| **`nyt_eng_20060226_0030_13:5-6`**           | but is it really __``that big``__ ? '' And they stopped using gas-burning logs in their fireplace .                                                                                 |
| **`pcc_eng_15_023.7457_x0367622_15:4-5`**    | When you are __``that big``__ , you know you are __``that big``__ , and I just was not motivated to really do anything about it except just accept that " it was the way it was " . |
| **`nyt_eng_19980609_0269_48:14-15`**         | we took a zoo tour on the double-decker bus _ the place is __``that big``__ .                                                                                                       |
| **`nyt_eng_20050221_0208_16:5-6`**           | Las Vegas has grown __``that big``__ .                                                                                                                                              |
| **`nyt_eng_20060609_0267_32:5-6`**           | `` When you get __``that big``__ , it means each individual horse is getting less attention , and I think that shows up this time of year . ''                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_big_ex.md`


### 15. _that interested_


#### 15.1. "that interested" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                   |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_005.4067_x0070914_105:10-13-14`** | Despite the previous 2400 words of this essay , nobody is really __``that interested``__ in the finer details of how their emails get to them .                                               |
| **`nyt_eng_20071116_0064_4:26-27-28`**          | `` We 've tried to avoid a lot of the self-promoting aspects of it -- -LRB- BEGIN BRACKET -RRB- we 're -LRB- END BRACKET -RRB- not __``that interested``__ in it . ''                         |
| **`pcc_eng_07_027.9923_x0436517_154:4-5-6`**    | Apple probably is n't __``that interested``__ in doing something like that because they do n't see the type market as something that 's going to make them billions and billions of dollars . |
| **`pcc_eng_25_036.9817_x0582423_052:14-15-16`** | When I come to church after a hard week , I 'm just not __``that interested``__ in hearing about all the controversial things Brigham Young said .                                            |
| **`pcc_eng_10_079.3196_x1265894_39:3-4-5`**     | People are n't __``that interested``__ in a team that has no chance to win a championship -- even if the team plays hard and overachieves .                                                   |
| **`nyt_eng_19971204_0462_34:11-12-13`**         | until recently , America 's Big Three auto makers were n't __``that interested``__ in the Japanese market , MacCarthy said .                                                                  |
| **`pcc_eng_13_006.6414_x0091043_035:18-19-20`** | SCULLY : What made you decide to write a book about an alien abduction if you 're not __``that interested``__ in the subject yourself ?                                                       |
| **`pcc_eng_04_076.6566_x1221970_013:7-8-9`**    | To be honest , I 'm not __``that interested``__ in stories about your children , and I do n't expect you to be interested in stories about mine .                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_interested_ex.md`


#### 15.2. "that interested" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                         |
|:--------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_29_067.5209_x1074664_09:16-17`** | I 'd rather keep it down to a dull roar and if you 're really __``that interested``__ in finding out how an intellectual genius ' mind works , you can follow along with the jump to learn about such important things as :                                         |
| **`nyt_eng_19950523_0204_26:8-9`**          | `` I thought that if people are __``that interested``__ in writers , then I 'd give people a novel about them .                                                                                                                                                     |
| **`pcc_eng_01_030.6889_x0479882_59:4-5`**   | If you 're __``that interested``__ , or that concerned that Hillcrest may be in violation of Federal law , the feel free to keep looking yourself .                                                                                                                 |
| **`nyt_eng_20051205_0140_5:22-23`**         | the new system is bulky and takes hours to develop an image , so it is unlikely that photographers will be __``that interested``__ in using it : There will not be , in other words , an Adams of the petri dish .                                                  |
| **`pcc_eng_25_089.9320_x1439028_07:13-14`** | " There are other teams that have said : ' We 're __``that interested``__ in a short-term , one - to - two year deal .                                                                                                                                              |
| **`pcc_eng_27_056.9960_x0905257_04:36-37`** | Physician dating service found girl wants questioning his friendship with her , drop subtle hints and comments about dating sites for physicians the needs dating site for physicians of what looking for , chances are __``that interested``__ in signing up for . |
| **`pcc_eng_11_021.9351_x0338669_1:6-7`**    | And if you are really __``that interested``__ , here is a ( fairly ) complete bibliography of David 's creative work online .                                                                                                                                       |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_interested_ex.md`


### 16. _that close_


#### 16.1. "that close" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                    |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_027.4693_x0427950_58:3-5-6`**     | It should never been __``that close``__ but now that I see how many are on the dole , I see they want Santa to keep handing out the " gifts " .                                                                                                                                                                |
| **`pcc_eng_04_070.6575_x1125132_23:19-20-21`**  | That being said , you do n't need to break the bank on this one considering you 're not __``that close``__ with the host .                                                                                                                                                                                     |
| **`nyt_eng_20070315_0228_30:6-7-8`**            | `` Before , it was not __``that close``__ to home -- usually a couple of blocks away , '' Harris said of the violence .                                                                                                                                                                                        |
| **`pcc_eng_07_054.3405_x0862178_12:46-47-48`**  | I remember once he said he gets his blood cells or white cells something along those lines checked often and now I believe maybe because he has hep b or c and I do n't want to be mean and ask him since we are n't __``that close``__ and I know if he does have them I doubt he would just openly tell me . |
| **`pcc_eng_10_073.8001_x1176787_22:16-17-18`**  | But Wilson says she 's staying optimistic about the four remaining nests since they were n't __``that close``__ to hatching .                                                                                                                                                                                  |
| **`pcc_eng_22_085.8779_x1371965_093:13-14-15`** | You see batters complain all the time about called strikes that are not __``that close``__ - perhaps this is just for effect / out of frustration , but it indicates they are often wrong about where a pitch was located .                                                                                    |
| **`apw_eng_20081031_1167_16:28-29-30`**         | MIT political scientist Adam Berinsky predicted the highest levels since 1968 , which he said is still quite impressive given that the polls show this election is not __``that close``__ and fewer people tend to vote when the race is n't tight .                                                           |
| **`pcc_eng_28_072.4622_x1156017_268:12-13-14`** | In fairness , not a bad miss , as he was n't __``that close``__ to goal .                                                                                                                                                                                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_close_ex.md`


#### 16.2. "that close" under _positive_ polarity


|                                      | `token_str`                                                                                                                                                                                               |
|:-------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_20040823_0034_50:32-33`** | Boston , with 12 wins in 15 games , moved within 5 1\/2 games of the New York Yankees in the AL East , the first time the Red Sox have been __``that close``__ since June 28 .                            |
| **`apw_eng_19991030_0171_58:3-4`**   | it was __``that close``__ , '' -- Navy safety Chris Lepore after a 28-24 loss to Notre Dame .                                                                                                             |
| **`nyt_eng_20000102_0088_65:4-5`**   | `` I was __``that close``__ to a nervous breakdown , '' he said , making the universal gesture for teeny with his thumb and index finger .                                                                |
| **`nyt_eng_19990426_0163_26:6-7`**   | she allowed people to be __``that close``__ to her . ''                                                                                                                                                   |
| **`nyt_eng_19971003_0052_24:4-5`**   | when you 're __``that close``__ it 's just a reaction .                                                                                                                                                   |
| **`apw_eng_19960723_1021_23:29-30`** | `` To be perfectly honest , in some of the bigger events I 've been in , such as the Olympics in ' 84 , when people are __``that close``__ and we 're packed together , I was n't really comfortable . '' |
| **`apw_eng_20030803_0264_12:4-5`**   | it 's been __``that close``__ , '' she said .                                                                                                                                                             |
| **`nyt_eng_19961020_0422_12:3-4`**   | to be __``that close``__ to winning it , and to lose it , -LRB- it -RRB- hurts even more .                                                                                                                |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_close_ex.md`


### 17. _that high_


#### 17.1. "that high" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                           |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_03_006.2072_x0084162_06:15-16-17`** | Hillary corrected the arrogant and self-righteous Aaj TV anchor and said the rent was not __``that high``__ but was in the range of $ 50 million .                                                                                    |
| **`nyt_eng_20061225_0168_26:13-14-15`**        | `` The compulsion to cut rates is much less when rates are not __``that high``__ to begin with . ''                                                                                                                                   |
| **`pcc_eng_23_089.0960_x1423752_70:4-5-6`**    | Your prices are not __``that high``__ , why ?                                                                                                                                                                                         |
| **`pcc_eng_03_045.3148_x0717898_15:10-11-12`** | Neither guy fits the MO and frankly I 'm not __``that high``__ on either of them .                                                                                                                                                    |
| **`pcc_eng_19_073.3431_x1168683_003:7-8-9`**   | But whereas that the expectations were not __``that high``__ for Philadelphia , they came out of the gate like gangbusters by winning their first three games .                                                                       |
| **`pcc_eng_28_018.3153_x0280034_51:22-23-24`** | Anyone new to the community would be advised to take What 's interview , even if their interest in music was n't __``that high``__ , simply because succeeding there pretty much assured entry to almost anywhere they wanted to be . |
| **`pcc_eng_06_100.4766_x1609123_26:10-11-12`** | " Coming out of high school my grades were n't __``that high``__ , but they stuck with me so that 's why I wanted to come here .                                                                                                      |
| **`pcc_eng_17_053.3057_x0845051_30:12-13-14`** | But even the rates of mother - to- child breastfeeding are n't __``that high``__ in the United States .                                                                                                                               |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_high_ex.md`


#### 17.2. "that high" under _positive_ polarity


|                                            | `token_str`                                                                                                                                         |
|:-------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_016.2849_x0247508_6:11-12`** | As mentioned in the interview , the average price is __``that high``__ for a game that is n't even going to be show in primetime .                  |
| **`pcc_eng_13_064.3426_x1023609_17:4-5`**  | Mine was certainly __``that high``__ when I opened .                                                                                                |
| **`nyt_eng_19990622_0113_82:10-11`**       | LEAD-STUDIES -LRB- Undated -RRB- _ One new study suggests __``that high``__ levels                                                                  |
| **`apw_eng_20050922_1109_14:13-14`**       | but some forecasters said Rita could pound the barrier with waves twice __``that high``__ .                                                         |
| **`nyt_eng_20080328_0149_19:3-4`**         | anyone with __``that high``__ a political profile must also be prepared to face the toughest scrutiny .                                             |
| **`nyt_eng_20091001_0175_13:6-7`**         | moreover , unemployment that remains __``that high``__ , that long , will cast long shadows over America 's future .                                |
| **`pcc_eng_27_077.9347_x1243983_10:3-4`**  | Mussina is __``that high``__ because of consistent excellence .                                                                                     |
| **`apw_eng_19980911_1198_27:24-25`**       | growth will likely be a scrawny 1 percent , while unemployment officially is 8 percent _ labor union statistics say it 's twice __``that high``__ . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_high_ex.md`


### Saving *`that`* Samples

 > in `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/`

1. *that hard*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_hard_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_hard_99ex~14.csv"`

2. *that great*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_great_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_great_99ex~23.csv"`

3. *that different*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_different_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_different_99ex~22.csv"`

4. *that difficult*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_difficult_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_difficult_99ex~22.csv"`

5. *that bad*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_bad_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_bad_99ex~99.csv"`

6. *that common*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_common_99ex~99.csv"`

7. *that familiar*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_familiar_99ex~99.csv"`

8. *that uncommon*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_uncommon_99ex~99.csv"`

9. *that impressed*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_impressed_99ex~99.csv"`

10. *that sweet*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_sweet_99ex~98.csv"`

11. *that simple*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_simple_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_simple_99ex~98.csv"`

12. *that easy*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_easy_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_easy_99ex~99.csv"`

13. *that good*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_good_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_good_99ex~99.csv"`

14. *that big*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_big_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_big_99ex~36.csv"`

15. *that interested*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_interested_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_interested_99ex~7.csv"`

16. *that close*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_close_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_close_99ex~29.csv"`

17. *that high*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/neg-that/neg-that_high_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/that/pos-that/pos-that_high_99ex~30.csv"`


## 3. *exactly*

|                               |   `f` |   `dP1` |   `LRC` |   `P1` |      `G2` |   `f2` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `adj_total` |
|:------------------------------|------:|--------:|--------:|-------:|----------:|-------:|----------:|------------:|------------:|--------:|-------:|--------------:|
| **NEGany~exactly_sure**       | 8,794 |    0.50 |    8.41 |   1.00 | 12,153.52 |  8,797 |  4,398.50 |    4,395.50 |        0.50 |    0.00 |   0.00 |       134,058 |
| **NEGany~exactly_clear**      | 1,746 |    0.50 |    6.38 |   1.00 |  2,405.41 |  1,747 |    873.50 |      872.50 |        0.50 |    0.00 |   0.00 |        83,958 |
| **NEGany~exactly_new**        | 1,371 |    0.50 |    6.03 |   1.00 |  1,885.84 |  1,372 |    686.00 |      685.00 |        0.50 |    0.00 |   0.00 |        21,548 |
| **NEGany~exactly_easy**       | 1,066 |    0.50 |    5.90 |   1.00 |  1,477.97 |  1,066 |    533.00 |      533.00 |        0.50 |    0.00 |   0.00 |       109,374 |
| **NEGany~exactly_true**       | 1,447 |    0.49 |    5.24 |   0.99 |  1,909.27 |  1,456 |    728.00 |      719.00 |        0.50 |    0.00 |   0.00 |        35,146 |
| **NEGany~exactly_surprising** |   440 |    0.50 |    4.61 |   1.00 |    610.00 |    440 |    220.00 |      220.00 |        0.50 |    0.00 |   0.00 |        18,886 |
| **NEGany~exactly_happy**      |   439 |    0.50 |    4.60 |   1.00 |    608.61 |    439 |    219.50 |      219.50 |        0.50 |    0.00 |   0.00 |        47,131 |
| **NEGany~exactly_great**      |   308 |    0.50 |    4.08 |   1.00 |    426.99 |    308 |    154.00 |      154.00 |        0.50 |    0.00 |   0.00 |        45,537 |
| **NEGany~exactly_subtle**     |   263 |    0.50 |    3.84 |   1.00 |    364.61 |    263 |    131.50 |      131.50 |        0.50 |    0.00 |   0.00 |         5,270 |
| **NEGany~exactly_fun**        |   224 |    0.50 |    3.60 |   1.00 |    310.54 |    224 |    112.00 |      112.00 |        0.50 |    0.00 |   0.00 |        19,811 |
| **NEGmir~exactly_sure**       |   148 |    0.50 |    3.10 |   1.00 |    205.21 |    148 |     74.00 |       74.00 |        0.50 |    0.00 |   0.00 |         5,983 |
| **NEGmir~exactly_clear**      |    52 |    0.50 |    1.43 |   1.00 |     72.09 |     52 |     26.00 |       26.00 |        0.50 |    0.00 |   0.00 |         3,331 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/ALL-exactly_10mostNEG-bigrams_AMscores.2024-08-07.md`


|                     |       `N` |      `f1` |   `adv_total` |
|:--------------------|----------:|----------:|--------------:|
| **NEGATED_exactly** | 6,347,362 | 3,173,681 |        44,378 |
| **NEGMIR_exactly**  |   583,470 |   291,735 |           862 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/exactly_MarginalFreqs_2024-08-07.md`


### 1. _exactly sure_


#### 1.1. "exactly sure" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_26_038.5253_x0606776_126:3-4-5`**   | I 'm not __``exactly sure``__ where we 're going to stop for the interval , but roughly looking at it on paper I 'm not in about three quarters of the second half .                                                                                                                                                                                                                                                                                                                                                  |
| **`pcc_eng_20_008.8701_x0127064_10:29-30-31`** | He does remember one instance where he was helping spray wash a vehicle with chlorine and he thinks he might have been splashed , but honestly he 's not __``exactly sure``__ , " Dr. Levy said .                                                                                                                                                                                                                                                                                                                     |
| **`pcc_eng_28_047.9127_x0759041_06:3-4-5`**    | You are n't __``exactly sure``__ what you 're looking for , so just start walking until you find something .                                                                                                                                                                                                                                                                                                                                                                                                          |
| **`pcc_eng_08_040.0566_x0632217_02:3-4-5`**    | I 'm not __``exactly sure``__ what all the bitching and moaning was about ( 56 % at Rottentomaotes - are you kidding me ? ! ) , but from where I sit and view cinema , the new Netflix original movie War Machine , from hot-shot director David Michod ( Animal Kingdom , The Rover ) is a pointed , extremely funny , and often times sad commentary on the war in Afghanistan and how the American military simply could never fully understand the ramifications of doing what they 've done in the middle east . |
| **`pcc_eng_00_014.4143_x0216652_50:1-2-3`**    | Not __``exactly sure``__ if you saw this , however her appropriate eye seems smaller compared to the left eye .                                                                                                                                                                                                                                                                                                                                                                                                       |
| **`pcc_eng_24_072.9455_x1163701_03:22-23-24`** | Not in the sense of , " How good will these be ? " so much as , " I 'm not __``exactly sure``__ what these things do . "                                                                                                                                                                                                                                                                                                                                                                                              |
| **`pcc_eng_11_090.3688_x1446709_074:3-4-5`**   | I 'm not __``exactly sure``__ how they 're saving lives .                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **`pcc_eng_02_017.3657_x0264870_29:3-4-5`**    | I 'm not __``exactly sure``__ when My Japanese Coach will be released in Europe .                                                                                                                                                                                                                                                                                                                                                                                                                                     |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_sure_ex.md`


#### 1.2. "exactly sure" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                          |
|:--------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_051.5735_x0817481_128:3-4`**  | we arent __``exactly sure``__ how it happened , the trailer was not damaged at all so we think that he some how kicked his own hoof with his opposite foot , so hard that it spilt . |
| **`pcc_eng_01_054.9333_x0872024_50:12-13`** | Dowling said , " unlike in China , where they are __``exactly sure``__ what they 're doing . "                                                                                       |
| **`pcc_eng_21_010.6826_x0156293_04:1-2`**   | __``Exactly sure``__ , it is very important go with a design which you exceedingly are certain to get connected to , and this may end in mentioning your favorite attitude .         |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/pos-exactly/pos-exactly_sure_ex.md`


### 2. _exactly clear_


#### 2.1. "exactly clear" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                      |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_009.4972_x0137242_17:09-10-11`** | The cause of the peculiar magnetic field is not __``exactly clear``__ .                                                                                                                                                                                                                                                                          |
| **`pcc_eng_00_031.1490_x0487313_29:1-4-5`**    | Nor is it __``exactly clear``__ what the terms describe .                                                                                                                                                                                                                                                                                        |
| **`pcc_eng_15_035.4801_x0557565_14:3-4-5`**    | It is n't __``exactly clear``__ as to why Apple 's review team revoked the approval or what these " business conflicts " could be but some people are suggesting it has something to do with Apple 's 30 percent App Store fee or it could be the fact that Steam Link allows i OS users access to a different app store , the Steam App Store . |
| **`pcc_eng_24_089.0376_x1423809_1:3-4-5`**     | It is n't __``exactly clear``__ that the above tweet is referring to Papa John's .                                                                                                                                                                                                                                                               |
| **`pcc_eng_13_089.9042_x1436975_25:3-4-5`**    | It is not __``exactly clear``__ why he was there , but multiple reliable sources noted that he appeared to be in good spirits and ...                                                                                                                                                                                                            |
| **`pcc_eng_25_044.0545_x0697097_41:3-4-5`**    | It 's not __``exactly clear``__ what would happen if the United States rolls back its participation in the program .                                                                                                                                                                                                                             |
| **`pcc_eng_11_011.8852_x0175930_155:4-5-6`**   | His role is n't __``exactly clear``__ , but the Royals have struggled to get much production out of second base between the likes of Chris Getz , Elliot Johnson , and Johnny Giavotella .                                                                                                                                                       |
| **`pcc_eng_26_008.2997_x0117795_07:23-24-25`** | Second , you have to delve pretty deep into the abyss of Oregon statutes to find an answer - and it 's not __``exactly clear``__ .                                                                                                                                                                                                               |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_clear_ex.md`


#### 2.2. "exactly clear" under _positive_ polarity


|                                            | `token_str`                                                                                                                           |
|:-------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_26_077.8429_x1242212_045:5-6`** | " Let me be __``exactly clear``__ about what health care reform means to you , " Obama said at one rally in New Jersey in July 2009 . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/pos-exactly/pos-exactly_clear_ex.md`


### 3. _exactly new_


#### 3.1. "exactly new" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                               |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_08_074.0576_x1182945_02:08-09-10`**  | The New Kids on the Block are n't __``exactly new``__ anymore - Donnie , Danny , Joey , Jordan and Jonathan are now all around 40 years old - but New Kids on the Block tickets are still some of the hottest in the entire music world . |
| **`pcc_eng_18_033.7120_x0529541_18:18-19-20`**  | I remember he ended up getting into a brawl with another guy - again , this was n't __``exactly new``__ .                                                                                                                                 |
| **`pcc_eng_05_093.2702_x1492818_21:4-5-6`**     | While this is n't __``exactly new``__ for postgraduate students , undergraduate students will soon be feeling the pain .                                                                                                                  |
| **`pcc_eng_11_089.2805_x1429013_02:4-5-6`**     | Television crossovers are not __``exactly new``__ , and typically come during sweeps week and are advertised as big " events " to try to draw big ratings .                                                                               |
| **`pcc_eng_07_059.1578_x0939996_06:14-15-16`**  | And let 's not forget that , for Microsoft , Saa S is n't __``exactly new``__ .                                                                                                                                                           |
| **`pcc_eng_06_107.9617_x1730227_038:25-26-27`** | The bass sound was largely influence by the likes of Meshuggah and Gojira , which was n't a bad thing per se but was n't __``exactly new``__ .                                                                                            |
| **`pcc_eng_12_067.9784_x1082388_04:4-5-6`**     | 3D printing is n't __``exactly new``__ anymore , but the products people have come up with using this technology continues to surprise us .                                                                                               |
| **`pcc_eng_16_039.5426_x0623916_1:5-6-7`**      | Nice design , but not __``exactly new``__ .                                                                                                                                                                                               |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_new_ex.md`


#### 3.2. "exactly new" under _positive_ polarity


|                                            | `token_str`                                                                                                                          |
|:-------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_06_106.0546_x1699477_007:6-7`** | The newly promoted clubs were __``exactly new``__ : Tatran ( Presov ) and Dukla ( Banska Bystrica ) often played in First Division . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/pos-exactly/pos-exactly_new_ex.md`


### 4. _exactly easy_


#### 4.1. "exactly easy" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                    |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_26_036.8066_x0578811_04:6-7-8`**    | Although convenient , it 's not __``exactly easy``__ to find in the Montreal area .                                                                                            |
| **`pcc_eng_21_092.7837_x1483124_16:7-8-9`**    | Adjusting to the real world is n't __``exactly easy``__ for Quantum Hughes .                                                                                                   |
| **`pcc_eng_28_060.5304_x0963060_10:7-8-9`**    | And , while these jobs are never __``exactly easy``__ ; they 're not as hard as the bathroom and kitchen .                                                                     |
| **`pcc_eng_06_107.1074_x1716444_34:7-8-9`**    | MS . TOWNSEND : It 's not __``exactly easy``__ .                                                                                                                               |
| **`pcc_eng_25_001.3116_x0005121_37:22-23-24`** | The freedom to share what we want , when we want and who we want to share it with , is not __``exactly easy``__ .                                                              |
| **`pcc_eng_12_068.4119_x1089326_07:7-8-9`**    | And most of the issues are n't __``exactly easy``__ to understand .                                                                                                            |
| **`pcc_eng_14_088.1475_x1408683_084:3-4-5`**   | It 's not __``exactly easy``__ to eyeball the difference between a harmless spot on your skin and a cancerous mole .                                                           |
| **`pcc_eng_23_086.9011_x1388426_01:3-4-5`**    | It was n't __``exactly easy``__ on the eyes but Ohio State accomplished the goal turning back Michigan 66 - 55 this afternoon in the Schott to stay in the league title hunt . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_easy_ex.md`


#### 4.2. "exactly easy" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*exactly easy*" found in **positive** hits.

### 5. _exactly true_


#### 5.1. "exactly true" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                    |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_081.2222_x1297767_04:3-4-5`**     | That 's not __``exactly true``__ , as it turns out , as there 's a moment in last year 's Rogue One that sets - up The Last Jedi in one sentence .                                                                                             |
| **`pcc_eng_17_042.2569_x0666416_232:08-09-10`** | The bit about stay- at- homes is n't __``exactly true``__ , because even in my generation , a stay - at- home mom was a luxury .                                                                                                               |
| **`pcc_eng_15_092.5341_x1479384_059:1-2-3`**    | Not __``exactly true``__ either .                                                                                                                                                                                                              |
| **`nyt_eng_20001215_0038_27:3-4-5`**            | that was n't __``exactly true``__ , considering Garrett contacted him Thursday afternoon and the two had more than one conversation .                                                                                                          |
| **`pcc_eng_12_001.8713_x0014057_03:5-6-7`**     | Well , that 's not __``exactly true``__ as you 'll find out later in this post .                                                                                                                                                               |
| **`pcc_eng_05_008.3119_x0118715_069:3-4-5`**    | That is not __``exactly true``__ .                                                                                                                                                                                                             |
| **`pcc_eng_27_009.3339_x0134405_23:13-15-16`**  | Even the way Jesus is described as looking to us , might not be __``exactly true``__ .                                                                                                                                                         |
| **`nyt_eng_20000311_0008_16:36-37-38`**         | the question on whether respondents agreed with the state board 's decision is one example , Holloway said : Pollsters told respondents the state board deleted evolution from the new science standards , which is n't __``exactly true``__ . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_true_ex.md`


#### 5.2. "exactly true" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                             |
|:---------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_01_039.4008_x0620551_017:27-28`** | The emotional core of the episode , the despair of CSM as his stories continue to be rejected , remains true even if the facts are __``exactly true``__ .                               |
| **`pcc_eng_04_108.00946_x1730885_68:4-5`**   | The same is __``exactly true``__ of my mom .                                                                                                                                            |
| **`pcc_eng_16_063.6170_x1013517_17:1-2`**    | __``Exactly true``__ .                                                                                                                                                                  |
| **`pcc_eng_10_096.5608_x1544912_097:8-9`**   | And I think your # 14 is __``exactly true``__ .                                                                                                                                         |
| **`pcc_eng_02_035.0751_x0551549_14:28-29`**  | The pieces show a rind of dry , snow- white mould , the French Cheese issue of Eyewitness Companions calls it " velvety " and that is __``exactly true``__ .                            |
| **`pcc_eng_19_073.9412_x1178295_36:20-21`**  | " In the way that great art can open possibilities and bad art can be deadening , it 's __``exactly true``__ for street art .                                                           |
| **`pcc_eng_20_065.8458_x1047803_26:4-5`**    | His prediction was __``exactly true``__ for my LSI 2.0 liter Toyota Celica engine that they also put in all Chevy Prisms and Toyota Corollas , but only a few get the LSI engine .      |
| **`pcc_eng_26_024.9026_x0386175_117:16-17`** | For it is a custom with us , when we think that we are speaking __``exactly true``__ , never to refuse our testimony ; and if we do this , much more would he who wrote by the Spirit . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/pos-exactly/pos-exactly_true_ex.md`


### 6. _exactly surprising_


#### 6.1. "exactly surprising" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                          |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_086.5269_x1381797_16:4-5-6`**     | The numbers are n't __``exactly surprising``__ , given an earlier report by the HHS showing that those who received a tax credit for purchasing health insurance in the federally run marketplaces pay an average of just $ 82 a month in premium ( after their tax credit is factored in ) across all plan types and all insurers . |
| **`pcc_eng_15_094.5244_x1511622_052:09-10-11`** | Rubber stamping the pre-selected " applicant " is n't __``exactly surprising``__ , and certainly is n't newsworthy .                                                                                                                                                                                                                 |
| **`pcc_eng_29_098.2723_x1571575_03:3-4-5`**     | This is n't __``exactly surprising``__ since :                                                                                                                                                                                                                                                                                       |
| **`pcc_eng_20_031.5448_x0493886_14:3-4-5`**     | It is n't __``exactly surprising``__ that the Orwells would attempt a stunt like this .                                                                                                                                                                                                                                              |
| **`pcc_eng_21_090.6035_x1448191_01:29-30-31`**  | Distracting attention from common sense policies by bringing up wedge social issues has been the strategy of the Republican Party for nearly a generation , so it 's not __``exactly surprising``__ that Rep.                                                                                                                        |
| **`pcc_eng_00_017.1458_x0260749_18:2-3-4`**     | So not __``exactly surprising``__ that Donald Trump gets his facts wrong in an appeal to his supporters .                                                                                                                                                                                                                            |
| **`pcc_eng_06_020.3518_x0313099_017:09-10-11`** | Google 's exit after just 22 months is n't __``exactly surprising``__ .                                                                                                                                                                                                                                                              |
| **`pcc_eng_26_039.1250_x0616448_23:3-4-5`**     | It is n't __``exactly surprising``__ that Pullman would receive angry letters or that they 'd accuse him of impiety or irreverence .                                                                                                                                                                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_surprising_ex.md`


#### 6.2. "exactly surprising" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*exactly surprising*" found in **positive** hits.

### 7. _exactly happy_


#### 7.1. "exactly happy" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                              |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19990602_0347_9:3-4-5`**            | they are not __``exactly happy``__ : Most came here with only the clothes on their backs , driven from their homes by Serbian troops bent on cleansing Kosovo of ethnic Albanians such as themselves .                                                                                   |
| **`pcc_eng_29_004.6955_x0059817_27:15-16-17`** | One of the things I love about this movie is that the ending is n't __``exactly happy``__ , nor does it wrap things up .                                                                                                                                                                 |
| **`pcc_eng_12_002.3203_x0021339_40:31-32-33`** | Just as a final remark to illustrate its importance exclusively as a comparison device : quite recently Oricon decided to hunt down the dedicated livedoor sales boards because they were n't __``exactly happy``__ about their precious reports being shared for free on the internet . |
| **`pcc_eng_14_006.1751_x0083789_073:4-5-6`**   | If you call not __``exactly happy``__ snapping his prize Seeker 's Firebolt 500 in half because of a built up burst of anger mixed recklessly with adrenalin .                                                                                                                           |
| **`pcc_eng_14_006.1751_x0083789_074:5-6-7`**   | I called his actions not __``exactly happy``__ .                                                                                                                                                                                                                                         |
| **`pcc_eng_04_004.5727_x0057898_149:1-2-3`**   | Not __``exactly happy``__ or happening -- it had been woeful -- but unflattering .                                                                                                                                                                                                       |
| **`apw_eng_19980514_1417_20:15-16-17`**        | if your next excursion takes you to Hong Kong , and the missus is not __``exactly happy``__ you 're not taking her along yet again , best check in at the Mandarin Oriental .                                                                                                            |
| **`apw_eng_20030216_0559_16:4-5-6`**           | `` We were n't __``exactly happy``__ with our performance , '' Wagner said .                                                                                                                                                                                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_happy_ex.md`


#### 7.2. "exactly happy" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*exactly happy*" found in **positive** hits.

### 8. _exactly great_


#### 8.1. "exactly great" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                             |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_004.5573_x0057597_02:22-23-24`**  | A typical conviction among the wireless clients wherever is the way that that a signal wireless signal supported or rehashed is not __``exactly great``__ as the first signal and that this strategy can cause a large number of issues particularly if the signal booster is handcrafted however is this all genuine ? |
| **`pcc_eng_12_030.1801_x0472503_01:18-19-20`**  | Some authors are comfortable with software such as Microsoft Word or Apple 's Pages but they are not __``exactly great``__ for authors .                                                                                                                                                                                |
| **`pcc_eng_03_038.3858_x0605532_299:11-12-13`** | When you are 12 years old , your speed is n't __``exactly great``__ .                                                                                                                                                                                                                                                   |
| **`pcc_eng_20_038.6004_x0607685_07:6-7-8`**     | What we saw yesterday was not __``exactly great``__ , or even complete , localization .                                                                                                                                                                                                                                 |
| **`pcc_eng_21_091.9854_x1470331_21:4-5-6`**     | Ride comfort is n't __``exactly great``__ either , and rough roads will send passengers ' teeth chattering .                                                                                                                                                                                                            |
| **`pcc_eng_28_046.4356_x0735209_04:20-21-22`**  | " When you miss some of the chances we did , and are going through a period that is not __``exactly great``__ , we can become a bit too predictable .                                                                                                                                                                   |
| **`pcc_eng_25_041.2122_x0650874_36:6-7-8`**     | Not too bad , but not __``exactly great``__ either .                                                                                                                                                                                                                                                                    |
| **`pcc_eng_15_010.1019_x0146987_46:2-3-4`**     | Certainly not __``exactly great``__ !                                                                                                                                                                                                                                                                                   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_great_ex.md`


#### 8.2. "exactly great" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*exactly great*" found in **positive** hits.

### 9. _exactly subtle_


#### 9.1. "exactly subtle" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                       |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_002.2391_x0020034_09:25-26-27`** | This film takes all the usual 80s tropes that are doing the rounds of late ( yes , that Stranger Things reference above was n't __``exactly subtle``__ was it ? ) but somehow gets away with it .                                                                 |
| **`pcc_eng_04_074.6445_x1189535_14:32-33-34`** | Topical elements reinforce the militaristic cast -- song titles like " War Games " or " Oil Well " and sounds of weapons cocked or shells clinking on the floor are not __``exactly subtle``__ -- but they shade the tone of the project instead of defining it . |
| **`pcc_eng_29_082.1890_x1311301_29:1-2-3`**    | Not __``exactly subtle``__ , but Kelly handles most of it with a light touch , which fails him only in depicting the uptight gym teacher ( Beth Grant ) .                                                                                                         |
| **`pcc_eng_24_104.7698_x1678990_01:09-10-11`** | The latest trojan to hit the block is n't __``exactly subtle``__ -- it sends your GPU on a roaring hunt for bitcoins .                                                                                                                                            |
| **`pcc_eng_00_064.3487_x1024226_08:08-09-10`** | " Coast Is Clear , " while not __``exactly subtle``__ , is further adrift from the dubstep chaos that launched Skrillex .                                                                                                                                         |
| **`pcc_eng_22_055.1459_x0874987_098:1-2-3`**   | Not __``exactly subtle``__ , but I got one of his eyes back in time for him to catch me smile , and return the ceremonial gesture to the Bug as best I could .                                                                                                    |
| **`pcc_eng_01_076.6688_x1223709_19:3-4-5`**    | Religion is n't __``exactly subtle``__ , and that did n't seem like a relevant priority here .                                                                                                                                                                    |
| **`pcc_eng_22_086.6897_x1385094_18:17-18-19`** | He mugs too much for my taste but then again , " The Producers " is n't __``exactly subtle``__ .                                                                                                                                                                  |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_subtle_ex.md`


#### 9.2. "exactly subtle" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*exactly subtle*" found in **positive** hits.

### 10. _exactly fun_


#### 10.1. "exactly fun" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                    |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_007.8789_x0111201_48:6-7-8`**     | The things they dislike are not __``exactly fun``__ to read , but I learn from them and the students seem to appreciate the opportunity to vent .                                                                              |
| **`pcc_eng_21_091.7055_x1465827_50:17-18-19`**  | Yes , it will take persistence , passion and elbow grease and no , it 's not __``exactly fun``__ , but I will tell you this : when the dust settles , and yes , it will settle , there will be better days and easier trades . |
| **`pcc_eng_08_049.5675_x0786125_034:10-11-12`** | Now I know that watching your puppy non-stop is n't __``exactly fun``__ & exciting , but being able to catch them before they have an accident is why this method works so well .                                              |
| **`pcc_eng_28_017.8871_x0273064_23:17-19-20`**  | When asked if the experience had been " fun , " he responded , " While nothing was __``exactly fun``__ , it was exciting to rappel with a full ruck sack and combat load off of the 65 ft tower .                              |
| **`pcc_eng_11_013.8988_x0208488_06:5-6-7`**     | While financial planning is n't __``exactly fun``__ , debt and money woes in the first year of marriage can be even worse .                                                                                                    |
| **`pcc_eng_21_017.0359_x0258848_248:19-20-21`** | I 'm not a fan of this team whatsoever , as they play a rough style that is not __``exactly fun``__ to watch .                                                                                                                 |
| **`pcc_eng_09_044.0674_x0696856_201:09-10-11`** | " The other people in the club were not __``exactly fun``__ , " she said , slowly .                                                                                                                                            |
| **`pcc_eng_16_028.9639_x0452742_38:23-24-25`**  | majority of people who play wow has never had to make difficult choices quickly , never had to do things that were not __``exactly fun``__ , just because they needed to .                                                     |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_fun_ex.md`


#### 10.2. "exactly fun" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*exactly fun*" found in **positive** hits.

### Saving *`exactly`* Samples

 > in `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/`

1. *exactly sure*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_sure_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/pos-exactly/pos-exactly_sure_99ex~3.csv"`

2. *exactly clear*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_clear_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/pos-exactly/pos-exactly_clear_99ex~1.csv"`

3. *exactly new*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_new_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/pos-exactly/pos-exactly_new_99ex~1.csv"`

4. *exactly easy*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_easy_99ex~99.csv"`

5. *exactly true*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_true_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/pos-exactly/pos-exactly_true_99ex~9.csv"`

6. *exactly surprising*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_surprising_99ex~99.csv"`

7. *exactly happy*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_happy_99ex~99.csv"`

8. *exactly great*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_great_99ex~80.csv"`

9. *exactly subtle*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_subtle_99ex~78.csv"`

10. *exactly fun*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/exactly/neg-exactly/neg-exactly_fun_99ex~71.csv"`


## 4. *any*

|                          |   `f` |   `dP1` |   `LRC` |   `P1` |     `G2` |   `f2` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `adj_total` |
|:-------------------------|------:|--------:|--------:|-------:|---------:|-------:|----------:|------------:|------------:|--------:|-------:|--------------:|
| **NEGany~any_happier**   |   828 |    0.50 |    5.13 |   1.00 | 1,122.62 |    830 |    415.00 |      413.00 |        0.50 |    0.00 |   0.00 |         1,909 |
| **NEGany~any_easier**    | 1,594 |    0.48 |    4.45 |   0.98 | 1,952.80 |  1,624 |    812.00 |      782.00 |        0.49 |    0.00 |   0.00 |        12,945 |
| **NEGmir~any_better**    |   380 |    0.48 |    3.59 |   0.98 |   473.47 |    386 |    193.00 |      187.00 |        0.49 |    0.00 |   0.00 |         3,681 |
| **NEGany~any_better**    | 4,719 |    0.44 |    3.51 |   0.94 | 4,676.15 |  5,022 |  2,511.00 |    2,208.00 |        0.47 |    0.00 |   0.00 |        49,936 |
| **NEGany~any_clearer**   |   355 |    0.48 |    3.41 |   0.98 |   439.41 |    361 |    180.50 |      174.50 |        0.49 |    0.00 |   0.00 |         1,037 |
| **NEGany~any_worse**     | 1,686 |    0.44 |    3.20 |   0.94 | 1,658.44 |  1,797 |    898.50 |      787.50 |        0.47 |    0.00 |   0.00 |        11,891 |
| **NEGany~any_younger**   |   255 |    0.48 |    3.12 |   0.98 |   317.76 |    259 |    129.50 |      125.50 |        0.49 |    0.00 |   0.00 |         1,789 |
| **NEGany~any_cheaper**   |   129 |    0.49 |    2.52 |   0.99 |   168.49 |    130 |     65.00 |       64.00 |        0.50 |    0.00 |   0.00 |         3,191 |
| **NEGmir~any_worse**     |    87 |    0.50 |    2.27 |   1.00 |   120.62 |     87 |     43.50 |       43.50 |        0.50 |    0.00 |   0.00 |         1,936 |
| **NEGany~any_nicer**     |    96 |    0.49 |    2.06 |   0.99 |   123.33 |     97 |     48.50 |       47.50 |        0.49 |    0.00 |   0.00 |           630 |
| **NEGmir~any_easier**    |    61 |    0.50 |    1.70 |   1.00 |    84.57 |     61 |     30.50 |       30.50 |        0.50 |    0.00 |   0.00 |           639 |
| **NEGany~any_brighter**  |    63 |    0.50 |    1.62 |   1.00 |    87.34 |     63 |     31.50 |       31.50 |        0.50 |    0.00 |   0.00 |           662 |
| **NEGany~any_wiser**     |    63 |    0.50 |    1.62 |   1.00 |    87.34 |     63 |     31.50 |       31.50 |        0.50 |    0.00 |   0.00 |           377 |
| **NEGmir~any_more**      |    41 |    0.50 |    1.03 |   1.00 |    56.84 |     41 |     20.50 |       20.50 |        0.50 |    0.00 |   0.00 |         2,193 |
| **NEGmir~any_different** |    48 |    0.48 |    1.02 |   0.98 |    58.17 |     49 |     24.50 |       23.50 |        0.49 |    0.00 |   0.00 |         8,657 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/ALL-any_10mostNEG-bigrams_AMscores.2024-08-07.md`


|                 |       `N` |      `f1` |   `adv_total` |
|:----------------|----------:|----------:|--------------:|
| **NEGATED_any** | 6,347,362 | 3,173,681 |        16,176 |
| **NEGMIR_any**  |   583,470 |   291,735 |         1,083 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/any_MarginalFreqs_2024-08-07.md`


### 1. _any happier_


#### 1.1. "any happier" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                      |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_008.1936_x0116018_02:27-30-31`**  | Email OZONE PARK , N.Y. - While Indian Blessing was certainly impressive winning last Saturday 's Breeders ' Cup Juvenile Fillies , trainer Patrick Reynolds could n't have been __``any happier``__ with Backseat Rhythm 's third - place finish in that race . |
| **`pcc_eng_20_049.5464_x0784127_8:4-6-7`**      | And I ca n't be __``any happier``__ to help with that !                                                                                                                                                                                                          |
| **`pcc_eng_07_102.1929_x1635552_23:22-24-25`**  | " This is our third birthday card for our daughter that we have ordered from Tiny Prints , and we could not be __``any happier``__ !                                                                                                                             |
| **`pcc_eng_07_026.8183_x0417542_19:3-5-6`**     | I could n't be __``any happier``__ for Barboursville Vineyards .                                                                                                                                                                                                 |
| **`pcc_eng_20_039.9482_x0629438_02:5-7-8`**     | " Man I could n't be __``any happier``__ right now , " says Constable of his outlook entering the 2018 season .                                                                                                                                                  |
| **`pcc_eng_09_008.3499_x0119087_13:21-23-24`**  | I was dreaming about getting my hands on this dress and when I got it I was ecstatic and could n't be __``any happier``__ .                                                                                                                                      |
| **`pcc_eng_23_037.3661_x0587543_21:08-10-11`**  | People kept telling her that she would n't be __``any happier``__ even with a different body , and that happiness came from " inside " .                                                                                                                         |
| **`pcc_eng_00_033.7294_x0528926_061:12-14-15`** | We just finished getting new windows from LAS and I ca n't be __``any happier``__ .                                                                                                                                                                              |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_happier_ex.md`


#### 1.2. "any happier" under _positive_ polarity


|                                             | `token_str`                                                                                                                              |
|:--------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_21_054.7927_x0869740_492:7-8`**  | Flowchart it if it makes you __``any happier``__ .                                                                                       |
| **`pcc_eng_09_066.2860_x1056139_21:16-17`** | They claim to be fighting to please their gods , but are their gods truly __``any happier``__ now that they have slaughtered thousands ? |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_happier_ex.md`


### 2. _any easier_


#### 2.1. "any easier" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                     |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_001.9368_x0015046_4:07-09-10`**  | But their second time around wo n't be __``any easier``__ as we 've got an exclusive sneak peek from Sunday 's premiere , which features Brendon experiencing the previously mentioned wedgie ! |
| **`pcc_eng_06_097.4540_x1560183_04:7-8-9`**    | Sometimes , closing a ring is not __``any easier``__ than opening it , and these pliers will make this task quick and easy .                                                                    |
| **`pcc_eng_26_087.1239_x1392709_55:18-20-21`** | In a world where wearing earbuds has practically become acceptable every waking hour , this course could n't be __``any easier``__ to use .                                                     |
| **`pcc_eng_09_002.2103_x0019621_13:12-14-15`** | Shopping for your 1935 Plymouth parts at Auto Parts WAY could n't be __``any easier``__ .                                                                                                       |
| **`pcc_eng_14_005.4745_x0072539_06:14-15-16`** | For a new investor , the problem of determining how to invest is n't __``any easier``__ to solve than picking investments .                                                                     |
| **`pcc_eng_21_027.2887_x0424976_11:4-6-7`**    | Your decision could n't be __``any easier``__ , make the choice .                                                                                                                               |
| **`pcc_eng_24_101.0029_x1617928_38:4-6-7`**    | And it wo n't be __``any easier``__ - or different - down the stretch of what should be an intriguing second half of the conference race .                                                      |
| **`pcc_eng_06_029.2768_x0457526_16:3-5-6`**    | It could n't be __``any easier``__ to get involved .                                                                                                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_easier_ex.md`


#### 2.2. "any easier" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                  |
|:--------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_29_097.2910_x1555633_20:4-5`**   | Can it get __``any easier``__ than that ?                                                                                                                                                    |
| **`pcc_eng_01_023.3553_x0361596_04:32-33`** | For anyone who already dreads the idea of trying to sift through a 20 - page Compensation Discussion and Analysis section , the new rules are unlikely to make the task __``any easier``__ . |
| **`pcc_eng_02_056.9188_x0904543_41:4-5`**   | Does it get __``any easier``__ than that ?                                                                                                                                                   |
| **`pcc_eng_02_029.4209_x0460098_08:4-5`**   | Could this be __``any easier``__ ?                                                                                                                                                           |
| **`pcc_eng_24_103.8677_x1664344_49:5-6`**   | If the work were __``any easier``__ , there would n't be any profit in it .                                                                                                                  |
| **`pcc_eng_19_022.7904_x0351899_13:7-8`**   | Anything that can make the process __``any easier``__ is typically embraced by students .                                                                                                    |
| **`pcc_eng_17_101.3464_x1621979_20:4-5`**   | Things cannot go __``any easier``__ than this only here at Slot Fruity casino .                                                                                                              |
| **`pcc_eng_13_041.0796_x0647978_91:14-15`** | I doubted that meeting the red car phantom would make 3 AM 's __``any easier``__ .                                                                                                           |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_easier_ex.md`


### 3. _any better_


#### 3.1. "any better" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                  |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_19_071.6640_x1141356_109:20-22-23`** | Maybe giving Leblebijian a shot is needed as others have made the case for although I suspect he wo n't be __``any better``__ .                                                                                                                                                                                                                                                                                              |
| **`pcc_eng_03_080.3001_x1284222_04:23-25-26`**  | NEW DELHI : Even as India is making headlines with its rising air pollution levels , the water in the country may not be __``any better``__ .                                                                                                                                                                                                                                                                                |
| **`pcc_eng_00_001.5820_x0009403_07:28-32-33`**  | I know now that I should have gone to MCAD and signed up for some live drawing classes , but at the time I was clueless ( not that I 'm __``any better``__ now ) .                                                                                                                                                                                                                                                           |
| **`pcc_eng_00_014.7778_x0222534_09:4-7-8`**     | " It could n't have been __``any better``__ .                                                                                                                                                                                                                                                                                                                                                                                |
| **`pcc_eng_06_101.1271_x1619624_34:32-33-34`**  | Wages begin to stagnate or even drop , until finally an unhappy equilibrium between " this job sucks , I quit " and " I'm disgruntled , but the alternatives are n't __``any better``__ " is found .                                                                                                                                                                                                                         |
| **`apw_eng_20090119_0905_12:08-10-11`**         | but Meissner 's results this season have n't been __``any better``__ , with eighth-place finishes at both Skate America and Cup of Russia .                                                                                                                                                                                                                                                                                  |
| **`pcc_eng_17_101.9677_x1632081_156:46-47-48`** | Basically within 24 hours of landing on Elephant Island he acknowledged , ' Boys , we 're not in a very good spot here ... we 're going to move over there ' and they got over there and they found that it really was n't __``any better``__ and he said ' well boys , then we 've got to get off this island , so here 's what I 'm thinking ' and he started to outline this plan of 6 of them going off in a life boat . |
| **`pcc_eng_13_005.6545_x0075002_095:4-6-7`**    | The timing could n't be __``any better``__ to play a Dallas team that has its own issues as of late .                                                                                                                                                                                                                                                                                                                        |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_better_ex.md`


#### 3.2. "any better" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                           |
|:---------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_083.3891_x1331430_60:11-12`**  | We have to embrace it , because it cannot get __``any better``__ .                                                                                                                                    |
| **`pcc_eng_00_028.8150_x0449447_22:12-13`**  | How exactly can Governor of Poker 2 hack make my game __``any better``__ ?                                                                                                                            |
| **`pcc_eng_21_095.8676_x1532744_039:10-11`** | But do we really get to know Robert Moses __``any better``__ through it ?                                                                                                                             |
| **`pcc_eng_07_094.9797_x1518895_02:13-14`**  | Blanche Lincoln faces a challenger from the left , but is he __``any better``__ on the environment ?                                                                                                  |
| **`pcc_eng_17_102.3816_x1638773_10:7-8`**    | Time will tell if it works __``any better``__ than the others .                                                                                                                                       |
| **`pcc_eng_00_100.1384_x1603437_57:5-6`**    | Was the latter option __``any better``__ , denotatively and connotatively , than the former ?                                                                                                         |
| **`pcc_eng_13_004.7071_x0059775_071:17-18`** | Nowadays , we survey and poll with computer - driven databases , but are the results __``any better``__ -- really ?                                                                                   |
| **`pcc_eng_00_053.5346_x0849025_216:4-5`**   | Does life get __``any better``__ than that - going to London , having your own show you made with your best friend about dinosaurs and then having a pizza truck whipping out pizzas all night long ? |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_better_ex.md`


### 4. _any clearer_


#### 4.1. "any clearer" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                            |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19991230_0115_37:3-5-6`**            | it does n't get __``any clearer``__ the deeper you look into how we keep track of time .                                                                                               |
| **`pcc_eng_20_083.9583_x1340490_035:34-35-36`** | I get Young Adult and I get Adult Fiction but I ca n't quite get my head around this new concept ; I could n't before I read this book and I 'm not __``any clearer``__ about it now . |
| **`pcc_eng_24_078.6373_x1255862_34:10-12-13`**  | In buzzwords that debase the English language but could not be __``any clearer``__ , what if I am unable to toilet myself ?                                                            |
| **`pcc_eng_16_080.9214_x1293532_28:07-09-10`**  | By the end , we wo n't be __``any clearer``__ on if he did or did n't but USADA , Tyler Hamilton , Mc Quaid , and everyone else will be made to look like utter bastards .             |
| **`pcc_eng_29_008.4338_x0120147_17:5-7-8`**     | But his reply could n't be __``any clearer``__ that there is definitely more theme park excitement in store for Harry Potter fans .                                                    |
| **`pcc_eng_17_002.1012_x0017705_19:07-09-10`**  | The message , here , could not be __``any clearer``__ .                                                                                                                                |
| **`nyt_eng_20001010_0063_22:4-6-7`**            | the choice could n't be __``any clearer``__ .                                                                                                                                          |
| **`nyt_eng_19970212_0747_28:5-6-7`**            | `` But it is not __``any clearer``__ to shareholders than it was on Tuesday how this maximization of value is going to be achieved . ''                                                |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_clearer_ex.md`


#### 4.2. "any clearer" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                      |
|:---------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_069.7553_x1110797_08:12-13`**  | But it is difficult to see how those countries could be __``any clearer``__ than they were last year at the disastrous Copenhagen climate summit - the problem is that the British government is n't listening . |
| **`pcc_eng_04_017.0435_x0258946_18:4-5`**    | Could that be __``any clearer``__ ?                                                                                                                                                                              |
| **`pcc_eng_00_034.5311_x0541732_166:22-23`** | And , in this novel where the characters are blatantly two dimensional , one cannot see that aspect of the period __``any clearer``__ .                                                                          |
| **`pcc_eng_18_058.7431_x0934845_05:7-8`**    | It 's hard to make it __``any clearer``__ than this .                                                                                                                                                            |
| **`pcc_eng_02_107.10294_x1730796_33:18-19`** | Yet at the end of the day , can we really say that Joan 's forecast is __``any clearer``__ than the rest of the characters ?                                                                                     |
| **`pcc_eng_01_002.8201_x0029401_11:5-6`**    | He cannot possibly be __``any clearer``__ as he spent endless hours for many months assuring Israel and its supporters , while condemning Palestinians without any reservation or remorse .                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_clearer_ex.md`


### 5. _any worse_


#### 5.1. "any worse" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                            |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_03_035.4837_x0558528_117:12-16-17`** | The tower in Siloam fell on certain men of Jerusalem but not because they were __``any worse``__ than their neighbors .                                                                                                                                                                                                                                                |
| **`pcc_eng_07_059.8326_x0950967_61:5-6-7`**     | Well , you are not __``any worse``__ off .                                                                                                                                                                                                                                                                                                                             |
| **`pcc_eng_18_003.5557_x0041458_07:14-15-16`**  | Now you might think this is dumb , but really , it 's not __``any worse``__ than any other Koopaling backstory .                                                                                                                                                                                                                                                       |
| **`pcc_eng_29_081.7335_x1304040_70:22-24-25`**  | We may conclude that those who suffered as a result of the cyclone in Darwin or the accident at Hobart were not necessarily __``any worse``__ than their fellow citizens , but the point that Jesus underlined was that we all deserve judgement and we will all receive it unless we repent of our godlessness : " Unless we repent , we will all likewise perish " . |
| **`pcc_eng_09_031.3370_x0491111_70:21-23-24`**  | A friend told me they bought some steaks here and loved them so I thought what the heck , ca n't be __``any worse``__ .                                                                                                                                                                                                                                                |
| **`pcc_eng_24_026.9018_x0418780_06:2-3-4`**     | But not __``any worse``__ than you 'd expect for the price .                                                                                                                                                                                                                                                                                                           |
| **`pcc_eng_07_104.6619_x1675689_06:22-24-25`**  | I have n't seen the original film , I do n't know if it 's any better , but it can hardly be __``any worse``__ .                                                                                                                                                                                                                                                       |
| **`apw_eng_20020210_0764_5:4-6-7`**             | `` It ca n't be __``any worse``__ _ I already know .                                                                                                                                                                                                                                                                                                                   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_worse_ex.md`


#### 5.2. "any worse" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                |
|:---------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_17_098.4251_x1574680_040:14-15`** | If you have crossed the line , turn around before it hurts you __``any worse``__ .                                                                                                         |
| **`pcc_eng_10_029.7444_x0464611_16:28-29`**  | I was in so much pain that we agreed we needed to call 999 as I was becoming increasingly short of breath and if the cramps got __``any worse``__ , I was afraid I was going to pass out . |
| **`pcc_eng_19_103.7603_x1661412_071:6-7`**   | You mean , is it __``any worse``__ than usual ? "                                                                                                                                          |
| **`pcc_eng_13_009.3800_x0135182_20:6-7`**    | Damn , could this get __``any worse``__ ?                                                                                                                                                  |
| **`pcc_eng_14_057.3773_x0911279_04:22-23`**  | However , some of the issues are natural , and man can only have a part in preventing them from getting __``any worse``__ .                                                                |
| **`pcc_eng_26_075.6340_x1206382_41:12-13`**  | It is far from certain that the situation would have been __``any worse``__ for Israel if it did not control the western part of the Golan .                                               |
| **`pcc_eng_18_108.3436_x1739116_17:8-9`**    | Only Michael Cera could make this concept __``any worse``__ .                                                                                                                              |
| **`pcc_eng_24_025.4573_x0395370_21:8-9`**    | One wonders , if it could get __``any worse``__ ?                                                                                                                                          |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_worse_ex.md`


### 6. _any younger_


#### 6.1. "any younger" under _negative_ polarity


|                                             | `token_str`                                                                                                                                                                             |
|:--------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20071130_0032_23:3-5-6`**        | i 'm not getting __``any younger``__ , and it takes longer for me to heal , but I expect to play . ''                                                                                   |
| **`nyt_eng_20000503_0370_28:09-10-11`**     | -lrb- Kenneth Branagh , at 35 , seemed hardly __``any younger``__ , despite his trim platinum haircut and the enthusiastic swashbuckling moves unleashed for the climactic duel . -RRB- |
| **`pcc_eng_23_044.4479_x0702045_3:7-8-9`**  | She got no boobs , she ain't __``any younger``__ but she has a beautiful arm pit !                                                                                                      |
| **`apw_eng_20080703_0375_25:5-7-8`**        | `` My husband is not getting __``any younger``__ -- nor am I for that matter -- but he is of course under more stress , '' she said .                                                   |
| **`pcc_eng_07_057.3884_x0911250_34:3-5-6`** | Please do n't be __``any younger``__ then x or older then x .                                                                                                                           |
| **`nyt_eng_20071220_0141_7:12-14-15`**      | `` It 's so nice to see you know you 're not getting __``any younger``__ . ''                                                                                                           |
| **`nyt_eng_20070731_0050_58:1-7-8`**        | none of these guys is getting __``any younger``__ .                                                                                                                                     |
| **`nyt_eng_20050103_0014_41:09-11-12`**     | he will turn 64 next summer , is n't getting __``any younger``__ , and only Manning , not the Giants , has gotten any better .                                                          |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_younger_ex.md`


#### 6.2. "any younger" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                    |
|:--------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_048.0576_x0760344_24:31-32`** | It was challenging for my guys , I would probably use bigger objects ( like dried garbanzos , mancala stones or large wood beads ) and bigger containers for kids __``any younger``__ or for anyone that wanted to focus more on math and less on fine motor . |
| **`nyt_eng_20060403_0038_12:5-6`**          | if these teams were __``any younger``__ , they 'd have to be burped .                                                                                                                                                                                          |
| **`apw_eng_20061215_0017_31:5-6`**          | if the actors were __``any younger``__ , he said , some of them would n't be able to appear in their roles because of laws barring performers under 17 from engaging in simulated or suggestive sexual activity .                                              |
| **`pcc_eng_25_014.0845_x0211541_03:3-4`**   | For those __``any younger``__ than Obama -- and I guess that includes me , for I am three weeks younger than Obama -- " Sputnik " was the name of the Soviet satellite that won them the first round of the " space race " in 1957 .                           |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_younger_ex.md`


### 7. _any cheaper_


#### 7.1. "any cheaper" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                     |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_036.6992_x0577136_70:27-29-30`**  | Since those back -office and overhead costs are either minimal or folded in with the county 's overall volume , outsourcing it to the chamber might not be __``any cheaper``__ for the county . |
| **`pcc_eng_22_055.2254_x0876246_041:26-27-28`** | This is because it is usually packed with NYU students who , one can only assume , do n't know any better because it is n't __``any cheaper``__ than the good stuff .                           |
| **`pcc_eng_13_044.6918_x0706511_13:4-5-6`**     | - gas was n't __``any cheaper``__ there than it is here                                                                                                                                         |
| **`pcc_eng_21_093.8012_x1499423_09:3-7-8`**     | It 's not like driving is __``any cheaper``__ .                                                                                                                                                 |
| **`pcc_eng_05_018.0536_x0275992_3:10-12-13`**   | Train would take most of the day and probably not be __``any cheaper``__ .                                                                                                                      |
| **`pcc_eng_10_071.7911_x1144431_52:7-8-9`**     | But if the 5 C is not __``any cheaper``__ than the 5 would have been , why did Apple go to all the trouble to make it ?                                                                         |
| **`pcc_eng_20_003.4025_x0038550_56:17-18-19`**  | It does n't look too different from what they sell today , and it sure is n't __``any cheaper``__ , either .                                                                                    |
| **`pcc_eng_09_006.5535_x0089995_18:18-19-20`**  | Ghana 's railway links Accra , Kumasi and Takoradi but the trains are much slower and are n't __``any cheaper``__ than motorised transport .                                                    |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_cheaper_ex.md`


#### 7.2. "any cheaper" under _positive_ polarity


|                                             | `token_str`                                                                                       |
|:--------------------------------------------|:--------------------------------------------------------------------------------------------------|
| **`pcc_eng_11_016.0647_x0243812_10:10-11`** | It is far from certain that it will be __``any cheaper``__ to use phones cross border in 2017 . " |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_cheaper_ex.md`


### 8. _any nicer_


#### 8.1. "any nicer" under _negative_ polarity


|                                                | `token_str`                                                                                                                             |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_05_003.0388_x0033137_06:7-8-9`**    | We in the solar industry are not __``any nicer``__ or better - behaved than professionals in the wind power industry .                  |
| **`pcc_eng_24_033.1236_x0519651_80:3-4-5`**    | They are n't __``any nicer``__ than the cabins we 've seen on other ships .                                                             |
| **`pcc_eng_07_023.8043_x0368677_36:08-10-11`** | When I visit my Mom they could not be __``any nicer``__ and I feel welcome .                                                            |
| **`pcc_eng_29_010.5889_x0155042_15:3-4-5`**    | Nick was n't __``any nicer``__ when he left the eatery .                                                                                |
| **`pcc_eng_00_037.5574_x0590595_81:5-8-9`**    | English paper one could n't have been __``any nicer``__ !                                                                               |
| **`nyt_eng_19990709_0395_77:04-10-11`**        | `` There 's nobody in this world that 's __``any nicer``__ than those people -LRB- the Wildermuths -RRB- , '' Rosie Breen said Friday . |
| **`pcc_eng_08_071.0142_x1133635_29:07-10-11`** | The owner and management company could not have been __``any nicer``__ or more helpful .                                                |
| **`pcc_eng_16_036.4489_x0573735_48:4-5-6`**    | The house was n't __``any nicer``__ than ours , but it did have an indoor toilet .                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_nicer_ex.md`


#### 8.2. "any nicer" under _positive_ polarity


|                                          | `token_str`                                                                                               |
|:-----------------------------------------|:----------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_083.9467_x1340971_6:6-7`** | " Could these views be __``any nicer``__ ? " is probably something Matthew Perry never tires of hearing . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_nicer_ex.md`


### 9. _any brighter_


#### 9.1. "any brighter" under _negative_ polarity


|                                                  | `token_str`                                                                                                                                                                                                                                                                         |
|:-------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_16_031.6261_x0495715_04:3-5-6`**      | Life could n't be __``any brighter``__ for art dealer Christina Daniels .                                                                                                                                                                                                           |
| **`pcc_eng_05_006.4356_x0088370_020:12-13-14`**  | If you use a reading light , make sure it 's not __``any brighter``__ than necessary and does n't shine in your eyes .                                                                                                                                                              |
| **`pcc_eng_20_085.4477_x1364434_12:42-43-44`**   | Actually , I do n't know my hippocampus from my amygdala , but it 's nice to know that all those mornings I 've sat on my butt wrestling silently with my monkey mind might actually keep me lucid -- if not __``any brighter``__ -- long into my crusty old age .                  |
| **`pcc_eng_10_050.3232_x0797563_18:4-6-7`**      | Those lights wo n't be __``any brighter``__ than on Saturday when they kickoff in the Georgia Dome .                                                                                                                                                                                |
| **`pcc_eng_22_107.05830_x1722622_163:13-15-16`** | Despite falling short of a return to the playoffs the future could not be __``any brighter``__ for the Rockets with Steve Francis and Yao Ming forming the foundation of a team that enters a new state of the art arena next season .                                              |
| **`pcc_eng_19_042.2865_x0666602_11:32-33-34`**   | As it stands , Black Berry is already in the red , losing as much as US $ 84 million in the last quarter ; the prediction for the next is n't __``any brighter``__ at this point .                                                                                                  |
| **`apw_eng_20020415_1436_7:22-24-25`**           | it was the second consecutive quarter the company missed Wall Street expectations , and the outlook for the current quarter did not appear __``any brighter``__ , though company officials said they remain confident Handspring will reach profitability in the December quarter . |
| **`nyt_eng_20051010_0255_28:11-12-13`**          | `` If you do n't see a tomorrow that 's not __``any brighter``__ than today , you just give up hope , '' she said .                                                                                                                                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_brighter_ex.md`


#### 9.2. "any brighter" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*any brighter*" found in **positive** hits.

### 10. _any wiser_


#### 10.1. "any wiser" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                              |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_06_028.3691_x0442793_062:38-40-41`** | However , this time I started thinking about why the hell am I clicking on that link and watching a video of this harridan , when I know exactly what to expect , and that I will not be __``any wiser``__ or otherwise improved afterwards ?                                                            |
| **`pcc_eng_13_006.3494_x0086333_24:22-26-27`**  | Because the visitor was operating undercover , no fuss was made of the transaction and he went on his merry way without the locals being __``any wiser``__ .                                                                                                                                             |
| **`pcc_eng_23_001.9510_x0015350_086:11-14-15`** | She 'd already packed her things , anyway , so nobody would be __``any wiser``__ .                                                                                                                                                                                                                       |
| **`pcc_eng_29_038.2705_x0601613_11:07-09-10`**  | However sadly I suspect we wo n't be __``any wiser``__ in March than we are now beyond what the newspapers and broadcasters have to share with us .                                                                                                                                                      |
| **`pcc_eng_26_003.4857_x0040043_23:25-26-27`**  | If , however , during this slow-down there commences vigorous education from government skeptics -- those who know that people working for government are n't __``any wiser``__ or more virtuous than are those working in the marker place -- then perhaps some serious benefit can be reaped from it . |
| **`pcc_eng_24_107.00037_x1715095_04:19-20-21`** | This is the seventh year this survey has been conducted , and , alas , the business is n't __``any wiser``__ to the agile way of doing things .                                                                                                                                                          |
| **`pcc_eng_02_002.5659_x0025351_109:13-16-17`** | The tinted windows allowed Brennan to be driven straight past the reporters without them being __``any wiser``__ .                                                                                                                                                                                       |
| **`pcc_eng_05_033.1232_x0520358_31:33-35-36`**  | If we already believe in emergence or Darwinism ( " competition among the resulting patterns " ) , we will find the new ideas compatible with our existing beliefs but we wo n't be __``any wiser``__ about consciousness .                                                                              |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_wiser_ex.md`


#### 10.2. "any wiser" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*any wiser*" found in **positive** hits.

### 11. _any more_


#### 11.1. "any more" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                         |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_039.1889_x0617478_18:59-60-61`**  | You do n't get it for free just by accident of birth ; it comes with responsibilities as well as rights , and if you cannot sit under an alethiometer and honestly declare that , yes , you do intend to honor the Contract and the Charter and all their implications ( something that your homegrown radical could not __``any more``__ than a would - be immigrant one ) , no citizen- shareholdership for you . |
| **`pcc_eng_08_073.9692_x1181488_090:20-21-22`** | Burton is finally getting his shot , but this is a huge deal for a tight end who was never __``any more``__ than third on the depth chart in Philadelphia .                                                                                                                                                                                                                                                         |
| **`pcc_eng_28_041.5014_x0655075_14:5-6-7`**     | It is n't , not __``any more``__ than you are when you get an X-ray .                                                                                                                                                                                                                                                                                                                                               |
| **`pcc_eng_26_092.8620_x1485719_286:08-10-11`** | This means that simply being online is n't enough __``any more``__ ; you need to be mobile ready .                                                                                                                                                                                                                                                                                                                  |
| **`pcc_eng_08_101.4328_x1626219_22:16-17-18`**  | Redick and Dudley wo n't get embarrassed by those types of stars ( at least not __``any more``__ than anyone else would ) , but they wo n't be able to impose their will defensively , either .                                                                                                                                                                                                                     |
| **`pcc_eng_15_094.2713_x1507533_036:23-25-26`** | The picture above of a same sex wedding looks legitimate and contains many elements of a real wedding , but it can never be __``any more``__ than play acting for those involved .                                                                                                                                                                                                                                  |
| **`pcc_eng_01_038.6433_x0608369_091:22-23-24`** | They 're in favor of minimal government intervention , sure - say , a carbon tax - but actual libertarians are n't __``any more``__ in favor of letting polluters run rampant than they are in letting murders walk free .                                                                                                                                                                                          |
| **`apw_eng_20020313_0521_5:14-15-16`**          | `` Based on preservation of the body , I would suggest it is not __``any more``__ than a day or so , '' Peerwani said .                                                                                                                                                                                                                                                                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_more_ex.md`


#### 11.2. "any more" under _positive_ polarity


|                                             | `token_str`                                                                                                                                            |
|:--------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19970707_0558_17:09-10`**        | he added , `` Are they helping us __``any more``__ than financially ? No way . ''                                                                      |
| **`pcc_eng_27_080.0571_x1278460_160:1-2`**  | __``Any more``__ than that and I 'd be cheating . "                                                                                                    |
| **`pcc_eng_29_032.6339_x0510582_05:26-27`** | -- NASA Even S. Fred Singer , famous for his stretching of data to deny global warming , is having a hard time using 1998 __``any more``__ ; he has to |
| **`pcc_eng_29_090.7140_x1449010_17:6-7`**   | And if that analogy were __``any more``__ on -point , Tommy Chong would be Trump 's running mate .                                                     |
| **`pcc_eng_07_076.8048_x1224907_37:1-2`**   | __``Any more``__ than that , and you 've got an identity crisis on your hands .                                                                        |
| **`pcc_eng_28_039.4873_x0622464_12:14-15`** | That was the score when I left and really who needs to know __``any more``__ than that .                                                               |
| **`pcc_eng_14_003.3017_x0037236_28:1-2`**   | __``Any more``__ than this and you will end up damaging the potential yield of your pear crop , or even damage the tree itself .                       |
| **`pcc_eng_21_053.1786_x0843490_50:1-2`**   | __``Any more``__ than that , I 'm too scared to talk about . "                                                                                         |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_more_ex.md`


### 12. _any different_


#### 12.1. "any different" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                       |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19980211_0442_39:7-8-9`**            | `` Well , corporate care is not __``any different``__ , and that 's what we got right now . ''                                                                                    |
| **`pcc_eng_12_066.9236_x1065457_346:08-09-10`** | And , for me , it 's not __``any different``__ than anyone else .                                                                                                                 |
| **`pcc_eng_29_045.5242_x0719300_46:4-5-6`**     | Accounting resumes are not __``any different``__ than every other professional restart .                                                                                          |
| **`pcc_eng_24_076.5730_x1222569_24:09-11-12`**  | Tolzmann said her team 's approach today wo n't be __``any different``__ than the approach for the first 25 games .                                                               |
| **`pcc_eng_17_075.7397_x1207980_20:08-09-10`**  | According to Turnmyre , these rules are n't __``any different``__ than the rules enforced in the past , so the skaters and bikers will not notice any differences .               |
| **`pcc_eng_18_088.6771_x1419845_047:4-5-6`**    | " They were not __``any different``__ than the usual furnaces , only they were much larger , they were about 80 meters in length , and in every two meters there was an opening . |
| **`nyt_eng_19990210_0475_23:5-6-7`**            | `` That shot was n't __``any different``__ than the shots I got in the first half , '' said Young , who missed all eight of his first-half attempts .                             |
| **`nyt_eng_20001205_0319_37:7-8-9`**            | for Mike Mussina , it was n't __``any different``__ than all those hours he 'd spent alone in the basement .                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_different_ex.md`


#### 12.2. "any different" under _positive_ polarity


|                                          | `token_str`                                                                                                                          |
|:-----------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19960517_0674_16:12-13`**     | the children want to know if popcorn carried into space tastes __``any different``__ than the Earth variety .                        |
| **`apw_eng_19950325_0321_39:10-11`**     | there any reason to believe that baseball will look __``any different``__ when opening day arrives in a week ?                       |
| **`nyt_eng_20100923_0083_2:8-9`**        | why should `` Wall Street '' be __``any different``__ ?                                                                              |
| **`nyt_eng_20010807_0098_49:7-8`**       | what makes you think workers are __``any different``__ ?                                                                             |
| **`nyt_eng_20100411_0019_26:10-11`**     | `` We are dubious that this time would be __``any different``__ , but will await the details , '' he said in a statement .           |
| **`pcc_eng_29_084.8245_x1353818_4:5-6`** | Asked if it was __``any different``__ than playing against big league players in Spring Training , Featherston said : " Not at all . |
| **`apw_eng_19961108_0604_8:7-8`**        | so what makes Saturday 's fight __``any different``__ ?                                                                              |
| **`apw_eng_20080312_0015_23:4-5`**       | why is it __``any different``__ from the District of Columbia ? ''                                                                   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_different_ex.md`


### Saving *`any`* Samples

 > in `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/`

1. *any happier*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_happier_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_happier_99ex~2.csv"`

2. *any easier*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_easier_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_easier_99ex~30.csv"`

3. *any better*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_better_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_better_99ex~99.csv"`

4. *any clearer*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_clearer_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_clearer_99ex~6.csv"`

5. *any worse*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_worse_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_worse_99ex~99.csv"`

6. *any younger*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_younger_99ex~66.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_younger_99ex~4.csv"`

7. *any cheaper*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_cheaper_99ex~30.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_cheaper_99ex~1.csv"`

8. *any nicer*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_nicer_99ex~25.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_nicer_99ex~1.csv"`

9. *any brighter*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_brighter_99ex~13.csv"`

10. *any wiser*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_wiser_99ex~16.csv"`

11. *any more*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_more_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_more_99ex~31.csv"`

12. *any different*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/neg-any/neg-any_different_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/any/pos-any/pos-any_different_99ex~47.csv"`


## 5. *remotely*

|                                |   `f` |   `dP1` |   `LRC` |   `P1` |   `G2` |   `f2` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `adj_total` |
|:-------------------------------|------:|--------:|--------:|-------:|-------:|-------:|----------:|------------:|------------:|--------:|-------:|--------------:|
| **NEGany~remotely_true**       |   250 |    0.49 |    3.21 |   0.99 | 318.17 |    253 |    126.50 |      123.50 |        0.49 |    0.00 |   0.00 |        35,146 |
| **NEGany~remotely_close**      |   694 |    0.45 |    3.09 |   0.95 | 729.84 |    729 |    364.50 |      329.50 |        0.47 |    0.00 |   0.00 |        46,485 |
| **NEGmir~remotely_close**      |   218 |    0.44 |    2.13 |   0.94 | 211.77 |    233 |    116.50 |      101.50 |        0.47 |    0.00 |   0.00 |         4,972 |
| **NEGany~remotely_interested** |   330 |    0.40 |    1.97 |   0.90 | 275.36 |    365 |    182.50 |      147.50 |        0.45 |    0.00 |   0.00 |        34,247 |
| **NEGany~remotely_funny**      |   137 |    0.46 |    1.95 |   0.96 | 148.45 |    143 |     71.50 |       65.50 |        0.48 |    0.00 |   0.00 |        15,183 |
| **NEGany~remotely_comparable** |   118 |    0.44 |    1.62 |   0.94 | 119.33 |    125 |     62.50 |       55.50 |        0.47 |    0.00 |   0.00 |         2,443 |
| **NEGany~remotely_ready**      |    58 |    0.50 |    1.49 |   1.00 |  80.41 |     58 |     29.00 |       29.00 |        0.50 |    0.00 |   0.00 |        29,641 |
| **NEGany~remotely_surprising** |    75 |    0.47 |    1.47 |   0.97 |  88.20 |     77 |     38.50 |       36.50 |        0.49 |    0.00 |   0.00 |        18,886 |
| **NEGany~remotely_similar**    |   152 |    0.40 |    1.44 |   0.90 | 127.23 |    168 |     84.00 |       68.00 |        0.45 |    0.00 |   0.00 |        11,228 |
| **NEGmir~remotely_true**       |    61 |    0.48 |    1.43 |   0.98 |  75.72 |     62 |     31.00 |       30.00 |        0.49 |    0.00 |   0.00 |         2,870 |
| **NEGany~remotely_possible**   |   164 |    0.38 |    1.30 |   0.88 | 122.64 |    186 |     93.00 |       71.00 |        0.43 |    0.00 |   0.00 |        30,222 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/ALL-remotely_10mostNEG-bigrams_AMscores.2024-08-07.md`


|                      |       `N` |      `f1` |   `adv_total` |
|:---------------------|----------:|----------:|--------------:|
| **NEGATED_remotely** | 6,347,362 | 3,173,681 |         6,109 |
| **NEGMIR_remotely**  |   583,470 |   291,735 |         1,963 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/remotely_MarginalFreqs_2024-08-07.md`


### 1. _remotely true_


#### 1.1. "remotely true" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                            |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_07_005.0022_x0064666_19:3-4-5`**     | This is n't __``remotely true``__ .                                                                                                                                                                                                    |
| **`pcc_eng_22_061.9397_x0985330_10:1-5-6`**     | None of that is __``remotely true``__ .                                                                                                                                                                                                |
| **`nyt_eng_20070105_0118_23:4-5-6`**            | and it is n't __``remotely true``__ .                                                                                                                                                                                                  |
| **`pcc_eng_27_066.5935_x1060399_19:08-10-11`**  | But to say he was awful is not even __``remotely true``__ .                                                                                                                                                                            |
| **`pcc_eng_08_107.4795_x1724114_012:02-17-18`** | That none of these claims ( with the arguable exception of the second ) is even __``remotely true``__ should n't be too surprising , because the " experts " who give voice to them typically define their expertise institutionally : |
| **`pcc_eng_07_005.0466_x0065388_32:12-14-15`**  | When the fuck did that become the case because it is n't even __``remotely true``__ .                                                                                                                                                  |
| **`pcc_eng_26_044.7963_x0708350_17:6-7-8`**     | A lot of it 's not __``remotely true``__ , so it 's kind of funny .                                                                                                                                                                    |
| **`pcc_eng_17_102.1664_x1635326_03:1-6-7`**     | None of this is even __``remotely true``__ .                                                                                                                                                                                           |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_true_ex.md`


#### 1.2. "remotely true" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                           |
|:--------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_18_012.9535_x0193592_15:5-6`**   | If that trend is __``remotely true``__ , that 's terrible . "                                                                                                         |
| **`pcc_eng_02_097.3407_x1557565_54:11-12`** | You can use facts to prove anything that 's even __``remotely true``__ ! " )                                                                                          |
| **`pcc_eng_16_088.6554_x1418960_07:8-9`**   | But is this assessment fair or even __``remotely true``__ when it comes to The Duchess of Cornwall 's two children from her first marriage to Andrew Parker -Bowles ? |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_true_ex.md`


### 2. _remotely close_


#### 2.1. "remotely close" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_007.1674_x0099469_409:11-13-14`** | The picture sold it , ( even though it was n't even __``remotely close``__ to showing the reality of what were BRINE SHRIMP ! )                                                                                                                                                                                                                                                                                                                                                                                                                          |
| **`pcc_eng_09_041.7494_x0659265_01:3-5-6`**     | Maelstrom , not even __``remotely close``__ to the pop scene .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| **`pcc_eng_01_061.2506_x0974529_08:2-4-5`**     | Like not even __``remotely close``__ to it !                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **`pcc_eng_22_007.9271_x0111811_111:5-6-7`**    | Certainly , Sophia is not __``remotely close``__ to that .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| **`pcc_eng_10_029.5999_x0462314_009:19-21-22`** | The Nuggets have no all-world dynamo in the vein of Le Bron James or Chris Paul , and nothing even __``remotely close``__ .                                                                                                                                                                                                                                                                                                                                                                                                                              |
| **`pcc_eng_22_056.5240_x0897245_073:07-09-10`** | The collection of US data is not even __``remotely close``__ to the claimed " high quality " ( except in a relative sense to that of the global data ) .                                                                                                                                                                                                                                                                                                                                                                                                 |
| **`pcc_eng_00_031.9893_x0500801_210:90-94-95`** | For instance , several years ago I ( futilely ) tried to convince a " stat guru " that Gary Payton did not fit in well with Phil Jackson 's Triangle Offense ; the " stat guru " insisted that on a per touch basis Payton did exactly the same things in the 2003 - 04 season as a Laker that he had done throughout his career , even though anyone who has even an ounce of common sense could easily see that Payton neither did the same things nor was he even __``remotely close``__ to being as effective as he had been earlier in his career . |
| **`pcc_eng_15_093.6272_x1497253_06:29-35-36`**  | Johnson himself broke the news welcoming " Jumanji juniors " Madison Iseman , Ser'Darius Blain , Alex Wolff and Morgan Turner have joined the production , and while none of these actors are even __``remotely close``__ to star power of the above four , the movie will actually see them sharing a unique bond .                                                                                                                                                                                                                                     |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_close_ex.md`


#### 2.2. "remotely close" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                                                                                                                                             |
|:---------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_03_011.7668_x0174028_10:7-8`**    | The only time we saw something __``remotely close``__ was when the government printed money to keep bond rates down so it could finance World War II .                                                                                                                                                                                  |
| **`pcc_eng_20_029.6196_x0462787_28:5-6`**    | Even if he comes __``remotely close``__ , the Phillies will be in business .                                                                                                                                                                                                                                                            |
| **`pcc_eng_24_077.9449_x1244667_54:11-12`**  | He 's -- he is the only candidate who is __``remotely close``__ to the center .                                                                                                                                                                                                                                                         |
| **`pcc_eng_19_031.7496_x0496481_014:49-50`** | Sure , there 's a little preparation involved as we watch the women scout the location and talk through the importance of avoiding slip-ups , but once it 's time to execute the plan , moviegoers will be hard - pressed to explain how these characters are even __``remotely close``__ to being ready for such a dangerous mission . |
| **`pcc_eng_00_022.3843_x0345507_017:11-12`** | It is difficult to imagine any Republican administration surviving anything __``remotely close``__ to Fast and Furious .                                                                                                                                                                                                                |
| **`pcc_eng_14_010.3187_x0150586_06:18-19`**  | Max Payne 1 and 2 took Max 's wife , infant child , and anyone that was __``remotely close``__ to him .                                                                                                                                                                                                                                 |
| **`pcc_eng_11_009.8803_x0143700_12:18-19`**  | Looking at the top of the 140 - pound division there are only two fighters who appear __``remotely close``__ to being able to challenge the Michigan native Mayweather .                                                                                                                                                                |
| **`pcc_eng_00_032.5171_x0509372_41:21-22`**  | There are way too many players of the sport in countries all over the world for your words to be __``remotely close``__ to the truth .                                                                                                                                                                                                  |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_close_ex.md`


### 3. _remotely interested_


#### 3.1. "remotely interested" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                   |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_16_036.1564_x0569012_12:4-5-6`**     | But distributors were n't __``remotely interested``__ in craft beer in those days .                                                                                                                                                                                                                           |
| **`pcc_eng_17_053.3987_x0846571_10:11-15-16`**  | Full disclosure : I do n't watch the show , nor am I even __``remotely interested``__ in it .                                                                                                                                                                                                                 |
| **`pcc_eng_28_011.1458_x0164133_05:15-16-17`**  | After starting grade school , I soon had that lightbulb moment that I was not __``remotely interested``__ in anything pertaining to science .                                                                                                                                                                 |
| **`pcc_eng_19_014.2226_x0213663_23:10-11-12`**  | I 'm pretty healthy , not pregnant , and not __``remotely interested``__ in becoming pregnant anywhere in the foreseeable future -- and , for that matter , I 've got access to birth control methods so that I do n't accidentally become pregnant while this thing is in my system , however long that is . |
| **`pcc_eng_05_002.3054_x0021224_100:3-5-6`**    | I 'm not even __``remotely interested``__ . "                                                                                                                                                                                                                                                                 |
| **`pcc_eng_08_075.7749_x1210583_09:20-21-22`**  | I do n't give TMZ any more attention than I wish to as they mostly report trash I 'm not __``remotely interested``__ in .                                                                                                                                                                                     |
| **`pcc_eng_15_099.2690_x1588186_121:46-47-48`** | In local fuck sluts I had had enough and I 'm currently in my third song , I saw him over visiting with my son when they got kicked out when she sexily sauntered into the reception as part of some award thing I 'm not __``remotely interested``__ in .                                                    |
| **`pcc_eng_24_108.08555_x1745729_23:3-4-5`**    | We are n't __``remotely interested``__ right now .                                                                                                                                                                                                                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_interested_ex.md`


#### 3.2. "remotely interested" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                |
|:--------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_28_047.7543_x0756528_13:14-15`** | Sometimes your university is playing his university and you pretend like you are __``remotely interested``__ in the outcome of the game :                                                                                  |
| **`pcc_eng_01_043.9261_x0693525_01:8-9`**   | Eachus is the only fullback I 'm __``remotely interested``__ in this team keeping .                                                                                                                                        |
| **`pcc_eng_23_103.3933_x1654985_54:28-29`** | Sometimes full of joy , sometimes downright depressing or scary , Unravel is a good , if simple game , and it deserves a look for anyone __``remotely interested``__ in indie titles or puzzle - based platformers .       |
| **`pcc_eng_15_070.5928_x1124137_24:17-18`** | Despite these minor issues , this is a fantastic addition to the strategy genre which anyone __``remotely interested``__ should buy .                                                                                      |
| **`pcc_eng_27_060.9757_x0969337_10:32-33`** | All of these attributes are shown in this seven- issue series that acts as a prelude to David 's 1990s run of Aquaman , the only time I 've ever been __``remotely interested``__ in the character ( sorry , Bill Reed ) . |
| **`pcc_eng_16_095.4783_x1529359_04:26-27`** | More than all of that , though , she uses her track record as the most successful Shark to bait every entrepreneur she 's even __``remotely interested``__ in .                                                            |
| **`pcc_eng_08_085.5488_x1368868_11:09-10`** | This is must-read for anyone who 's even __``remotely interested``__ in working in television .                                                                                                                            |
| **`pcc_eng_12_097.7119_x1562992_19:6-7`**   | Is anyone but me even __``remotely interested``__ ?                                                                                                                                                                        |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_interested_ex.md`


### 4. _remotely funny_


#### 4.1. "remotely funny" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                         |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_034.0465_x0533540_095:14-15-16`** | However the last 2 nights with the adult comedians were truly vulgar and not __``remotely funny``__ and a waste of our time .                                                                                                                       |
| **`pcc_eng_07_057.0140_x0905249_14:12-14-15`**  | And then let 's show that they 're not jokes , not even __``remotely funny``__ , as the Dalek cleanly and efficiently slaughters everyone in sight .                                                                                                |
| **`pcc_eng_25_040.3526_x0637005_48:3-5-6`**     | It 's not even __``remotely funny``__ .                                                                                                                                                                                                             |
| **`nyt_eng_20050103_0141_23:16-18-19`**         | although Poston is appropriately grumpy , Darius McCrary -LRB- `` Family Matters '' -RRB- offers nothing even __``remotely funny``__ as Nate 's friend and co-worker .                                                                              |
| **`pcc_eng_02_038.7814_x0611318_19:3-4-5`**     | There is nothing __``remotely funny``__ or interesting about " Mad Money , " the type of flick that coasts on such hoary cliches as people rolling around in cash and throwing it up in the air at every opportunity .                              |
| **`pcc_eng_16_025.2419_x0392368_08:26-28-29`**  | The last comic standing would be that southern guy that hosts the show and was funny in a sitcom several years ago , but has n't been __``remotely funny``__ since .                                                                                |
| **`apw_eng_20090130_0104_9:27-30-31`**          | even if it had tried to capitalize on the popularity of `` Fargo '' and come out more than a decade ago , it still would n't have been __``remotely funny``__ .                                                                                     |
| **`pcc_eng_16_080.3288_x1283871_3:26-28-29`**   | because atleast to me it looks like she 's legitimately retarded and aside from a few errors with her japanese / pronounciation there 's absolutely nothing even __``remotely funny``__ , even for someone who ' gets ' what she is talking about . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_funny_ex.md`


#### 4.2. "remotely funny" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                            |
|:---------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20071206_0126_10:4-5`**           | and the only __``remotely funny``__ joke is a racist one .                                                                                                                             |
| **`pcc_eng_27_098.0893_x1570404_2:13-14`**   | Please join us for the 2019 Arts In Reach Winter Showcase , __``Remotely Funny``__ !                                                                                                   |
| **`pcc_eng_04_056.4892_x0896164_07:6-7`**    | If you think this is __``remotely funny``__ , there is something seriously wrong with you .                                                                                            |
| **`pcc_eng_00_095.0026_x1520146_43:6-7`**    | How can you find anything __``remotely funny``__ about a marker of a 6 year old ?                                                                                                      |
| **`pcc_eng_12_026.0568_x0405384_04:11-12`**  | We 're on the hunt for anyone who finds him __``remotely funny``__ .                                                                                                                   |
| **`pcc_eng_00_099.0120_x1585219_055:16-17`** | But what is it like when , in addition to your inability to say anything __``remotely funny``__ or interesting to the person you are into , you have a mental health problem as well ? |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_funny_ex.md`


### 5. _remotely comparable_


#### 5.1. "remotely comparable" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                 |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_05_087.5005_x1399656_41:16-18-19`** | It 's a mistake to equate elections with private-sector performance reviews , because they 're not even __``remotely comparable``__ .                                                       |
| **`pcc_eng_10_019.2527_x0295016_57:5-6-7`**    | The two situations are n't __``remotely comparable``__ , and in fact I think they differ in at least six key ways :                                                                         |
| **`pcc_eng_12_039.3369_x0620049_138:7-8-9`**   | My grief is my own and not __``remotely comparable``__ to anyone else 's , so please do n't tell me how it is easy for me because it is n't .                                               |
| **`pcc_eng_21_090.1352_x1440643_46:2-4-5`**    | , not even __``remotely comparable``__ to revive or garrison .                                                                                                                              |
| **`apw_eng_19980204_0856_30:6-7-8`**           | note that we could accomplish nothing __``remotely comparable``__ on the basis of the Court\/rebel scenes .                                                                                 |
| **`pcc_eng_27_030.9843_x0483881_020:1-7-8`**   | Nothing on the present horizon is __``remotely comparable``__ to that .                                                                                                                     |
| **`pcc_eng_18_032.7817_x0514377_30:08-09-10`** | But the freedom of online speech is not __``remotely comparable``__ to Amazon 's unfettered monopoly and stranglehold over the publishing industry .                                        |
| **`pcc_eng_01_032.9426_x0516326_1:22-24-25`**  | What is ironic is that with all the technology and what not that we are blessed with today , Supriya is n't even __``remotely comparable``__ to Silk , when it comes to pure sensuousness . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_comparable_ex.md`


#### 5.2. "remotely comparable" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|:---------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_26_051.1870_x0811429_08:13-14`**  | Ironically , Trump would be lucky indeed to possess an intellect even __``remotely comparable``__ to Thurston's .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| **`pcc_eng_15_101.3041_x1621176_02:14-15`**  | Still , it 's hard to think of another Chicago director with a __``remotely comparable``__ 2013 - 14 dance card .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| **`pcc_eng_16_097.4883_x1561897_3:15-16`**   | Can you name a musical artist of the past 20 or so years even __``remotely comparable``__ to Charlie Parker , Miles Davis , Elvis , Chuck Berry , Bob Dylan or Janis Joplin ?                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| **`pcc_eng_24_017.3501_x0264045_640:14-15`** | If anyone has heard an orchestral recording made in the last 20 years __``remotely comparable``__ in sonic beauty or believability to the 1958 and 1959 Mercury recordings we 've just heard , I 'd sure like to know what it is .                                                                                                                                                                                                                                                                                                                                                                                             |
| **`pcc_eng_04_002.1108_x0017928_015:38-39`** | And therefore , since most of foreign policy is now driven by economics , and most wars are fought largely by economic means , he disposes of more power than any 12 heads of government of any __``remotely comparable``__ countries .                                                                                                                                                                                                                                                                                                                                                                                        |
| **`pcc_eng_06_011.1493_x0164108_102:18-19`** | The Bering Sea is far too cold and frozen over most of the year to be even __``remotely comparable``__ to the English Channel ?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| **`pcc_eng_16_021.1941_x0327052_05:58-59`**  | I speak as a former , if long- lapsed , member of the Tolkien cult -- one who vividly recalls a 12 - year-old 's impassioned assertion that he had the right to borrow books , namely The Lord of the Rings , from the library 's Adult Section , who searched in vain for anything even __``remotely comparable``__ ( C.S. Lewis was grade-school stuff , Robert E. Howard seemed lame ) , who , a few years later , saved up and sent away to a London bookstore to purchase his own hardbound copies of the trilogy , and who was soon after shocked to see the books appear here as garish , unauthorized Ace paperbacks . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_comparable_ex.md`


### 6. _remotely ready_


#### 6.1. "remotely ready" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                             |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_25_009.2874_x0134183_81:22-23-24`** | He was in his forties when we met ; a year and a half later , when I realized he was n't __``remotely ready``__ to settle down , I broke up with him .                                                                                                  |
| **`pcc_eng_28_070.7189_x1128016_12:26-27-28`** | Despite optimistic bulletins from the front that the war will be won in a matter of weeks , their own leaders concede that they 're not __``remotely ready``__ for a direct assault on Tripoli , which in any case would result in massive casualties . |
| **`pcc_eng_00_064.8894_x1032989_22:3-4-5`**    | I am not __``remotely ready``__ to tackle playing the fiddle in a full - bore week - long camp like Violon Trad - I 'll be going for the guitar classes , mostly .                                                                                      |
| **`pcc_eng_04_043.7097_x0690041_04:11-12-13`** | In that instance they were taken totally by surprise , not __``remotely ready``__ to defend against a sudden and coordinated attack .                                                                                                                   |
| **`pcc_eng_06_102.9020_x1648488_117:3-4-5`**   | I 'm not __``remotely ready``__ to talk about how to talk with kids about sex .                                                                                                                                                                         |
| **`pcc_eng_26_032.3176_x0506188_29:13-14-15`** | Smith seems like a pleasant 14 - year-old , but he 's not __``remotely ready``__ to carry a blockbuster on his own or to deal with a script so silly it even seems to stymie his dad .                                                                  |
| **`nyt_eng_19980427_0151_37:08-09-10`**        | Mark had finally confessed : He was n't __``remotely ready``__ to be a father .                                                                                                                                                                         |
| **`nyt_eng_19970113_0054_2:5-6-7`**            | most of them are not __``remotely ready``__ to give up their car keys , and many are not at all bashful about communicating their specific needs to manufacturers .                                                                                     |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_ready_ex.md`


#### 6.2. "remotely ready" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*remotely ready*" found in **positive** hits.

### 7. _remotely surprising_


#### 7.1. "remotely surprising" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                   |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_04_076.4185_x1218068_035:09-10-11`** | So the sentiment that elected Donald Trump was not __``remotely surprising``__ to me , or to a lot of people who cover politics for a long time .                             |
| **`pcc_eng_01_044.3540_x0700401_18:13-14-15`**  | Race and politics are very strong themes this year , which is not __``remotely surprising``__ .                                                                               |
| **`pcc_eng_00_034.7586_x0545419_23:7-8-9`**     | This is , of course , not __``remotely surprising``__ .                                                                                                                       |
| **`pcc_eng_27_026.2661_x0408312_06:24-25-26`**  | Instead , it blends warm comedy , silly slapstick and a heavy dose of sentiment to tell a story that 's engaging but never __``remotely surprising``__ .                      |
| **`pcc_eng_23_046.4112_x0733740_067:5-6-7`**    | And the plot is never __``remotely surprising``__ either .                                                                                                                    |
| **`pcc_eng_24_087.1076_x1392625_05:14-16-17`**  | Well this is the talk of the office this morning , which is n't even __``remotely surprising``__ .                                                                            |
| **`pcc_eng_09_094.8895_x1519087_27:3-4-5`**     | This is n't __``remotely surprising``__ .                                                                                                                                     |
| **`pcc_eng_02_036.7792_x0579083_05:3-4-5`**     | It 's not __``remotely surprising``__ to learn that James Comey and Sally Yates , who were both fired by Donald Trump , have given cooperative interviews to Robert Mueller . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_surprising_ex.md`


#### 7.2. "remotely surprising" under _positive_ polarity


|                                             | `token_str`                                                                              |
|:--------------------------------------------|:-----------------------------------------------------------------------------------------|
| **`pcc_eng_07_089.5566_x1431152_10:14-15`** | The joke , of course , is that any of this would be __``remotely surprising``__ by now . |
| **`pcc_eng_02_099.2627_x1588779_018:4-5`**  | But was it __``remotely surprising``__ ?                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_surprising_ex.md`


### 8. _remotely similar_


#### 8.1. "remotely similar" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                           |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_03_088.0188_x1409076_04:18-19-20`** | There were a few bus lines in Springfield , Illinois , where I grew up , but nothing __``remotely similar``__ to the CTA .                                                                                                                                                                                                                            |
| **`pcc_eng_28_072.5801_x1157937_11:40-41-42`** | Their second album , " Bigorna " ( subtitled ' The Real History of King Arthur and the Knights of the Round Table ' ) is a concept album , a spoof on the legend of King Arthur - nothing __``remotely similar``__ to Rick Wakeman 's solemn " Myths and Legends " ...                                                                                |
| **`pcc_eng_03_031.6455_x0496391_57:11-12-13`** | If she is very religious or has opinions that are not __``remotely similar``__ to mine , I probably would not .                                                                                                                                                                                                                                       |
| **`pcc_eng_04_078.7667_x1256207_60:13-27-28`** | Being only a fictional animal herself , she is nonetheless concerned that none of the goods for which the opponents had registered their earlier CTMs are __``remotely similar``__ to " live animals " for which , in her humble opinion , registration for live animals should be allowed -- though not for animals that are both pink and English . |
| **`nyt_eng_20050112_0364_60:21-22-23`**        | but if you go deeper than this surface comparison , you can see that being a white middle-class person is not __``remotely similar``__ to what African-Americans had to go through in this country , '' he says .                                                                                                                                     |
| **`nyt_eng_20000202_0371_4:15-16-17`**         | `` It 's really weird , '' she says , `` because flaxseed is not __``remotely similar``__ to an egg . ''                                                                                                                                                                                                                                              |
| **`pcc_eng_06_078.4987_x1253180_06:3-4-5`**    | There 's nothing __``remotely similar``__ .                                                                                                                                                                                                                                                                                                           |
| **`pcc_eng_20_080.0498_x1277185_64:3-4-5`**    | It 's not __``remotely similar``__ to Brady Anderson hitting 50 ( at age 32 ) but only cracking 20 two other times ( 24 and 21 ) .                                                                                                                                                                                                                    |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_similar_ex.md`


#### 8.2. "remotely similar" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                                                                                                                       |
|:--------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_21_016.1683_x0244785_58:30-31`** | If you look at the problems that the world community has had dealing with those countries mentioned above , imagine what it would be like to deal with even __``remotely similar``__ problems in a country of 1.3 billion people .                                                                                                                                |
| **`pcc_eng_17_107.08332_x1727115_10:4-5`**  | The thing even __``remotely similar``__ that I 've ever seen before was an instructor not being able to edit feedback on a quiz question in a question bank .                                                                                                                                                                                                     |
| **`pcc_eng_27_017.3964_x0265339_05:17-18`** | It seemed as though I was miles from the mark of being able to achieve anything __``remotely similar``__ .                                                                                                                                                                                                                                                        |
| **`pcc_eng_00_008.0326_x0113530_23:8-9`**   | The only one that Lee's Hagerstown was __``remotely similar``__ to was York .                                                                                                                                                                                                                                                                                     |
| **`pcc_eng_26_001.2002_x0003192_14:15-16`** | " So please if anyone is out there right now going through something even __``remotely similar``__ to what I went through , please , we all need to speak out about this - yes of course I want justice for the POS that manipulated me , but more importantly this is about saving anyone else who feels like they do n't have a voice , " the actor continued . |
| **`pcc_eng_07_034.7710_x0546184_2:5-6`**    | Will it be even __``remotely similar``__ ?                                                                                                                                                                                                                                                                                                                        |
| **`pcc_eng_19_046.4073_x0732868_12:6-7`**   | If the Gabba pitch is __``remotely similar``__ to the one served up to Victoria , the Bulls duo will be a difficult task for the Western Australia batsmen .                                                                                                                                                                                                      |
| **`pcc_eng_10_081.8697_x1306883_34:26-27`** | Hoifer reported as follows : " A careful search through the available data on Indian languages of Southern California fails to reveal any word even __``remotely similar``__ to the word Santana , big or bad winds . "                                                                                                                                           |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_similar_ex.md`


### 9. _remotely possible_


#### 9.1. "remotely possible" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                 |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_083.0792_x1326128_21:04-09-10`**  | Of course , nothing like that is even __``remotely possible``__ , but North Korean officials were infuriated that Park and Xi even had the discussion .                                                                                                                                                                     |
| **`pcc_eng_16_055.7032_x0885357_51:3-5-6`**     | It 's not even __``remotely possible``__ that all inequality everywhere is due to the extension of patents and copyrights since 1981 .                                                                                                                                                                                      |
| **`pcc_eng_28_013.0626_x0195370_075:3-4-5`**    | This is not __``remotely possible``__ all the time , but you want to do it as much as possible .                                                                                                                                                                                                                            |
| **`pcc_eng_05_081.7947_x1307639_28:1-6-7`**     | None of it is even __``remotely possible``__ in the real world , of course , defying all the laws of physics and common sense , but it does n't matter : at this point you 'll be so pumped to see the victims get their comeuppance you 'll be more than happy to forgive the outrageous consistencies .                   |
| **`pcc_eng_20_037.8719_x0596086_64:11-12-13`**  | The Ameican people are so fed up , it is not __``remotely possible``__ Obama will win this next election .                                                                                                                                                                                                                  |
| **`pcc_eng_13_089.9570_x1437826_114:19-22-23`** | When asked if his father ever mentioned participating with the Provincetown Players , he responded that this could n't be " __``remotely possible``__ . "                                                                                                                                                                   |
| **`nyt_eng_20070405_0297_1:48-49-50`**          | what if housing prices decline by 20 percent ? That would solve Seattle 's affordability problem , right ? Oh , I can see the e-mails , hear the phone messages and read the hand-written notes already : Idiot ! This will not happen , it is not __``remotely possible``__ , and , thank you , never mention this again . |
| **`pcc_eng_11_083.7052_x1338585_063:4-6-7`**    | " It is not even __``remotely possible``__ that Trump will be able to bring back employment in coal or total manufacturing back anywhere near the level of the mid-20th century .                                                                                                                                           |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_possible_ex.md`


#### 9.2. "remotely possible" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                |
|:---------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_05_105.6942_x1693261_73:4-5`**    | If this is __``remotely possible``__ , a simple unit test testing that each animal you support can eat , in the manner of :                                                                                |
| **`pcc_eng_26_036.0887_x0567165_069:23-24`** | The luckiest thing that happened to me , happened long before any of the things that now make my life seemed even __``remotely possible``__ .                                                              |
| **`pcc_eng_08_095.7630_x1534201_59:19-20`**  | Remote desktop sharing is a powerful new feature which , when perfected , could make pc ANYWHERE and __``Remotely Possible``__ unnecessary .                                                               |
| **`pcc_eng_09_062.7332_x0998735_32:18-19`**  | My practice is to run a reality check on ALL dream material , asking whether it is __``remotely possible``__ that something seen in a dream will manifest in the future , literally or symbolically .      |
| **`pcc_eng_08_032.9087_x0516806_065:4-5`**   | Is that even __``remotely possible``__ ?                                                                                                                                                                   |
| **`pcc_eng_17_055.3431_x0877988_07:20-21`**  | We have been critical of some of the ways Republicans have described the plan , but is this even __``remotely possible``__ ?                                                                               |
| **`pcc_eng_27_085.3371_x1363919_03:19-20`**  | He 's a frantic , over-worked , middle - managery kind of vacuum cleaner , if that 's __``remotely possible``__ , and one that 's locked in constant , hectic mediation between populace and environment . |
| **`pcc_eng_20_056.8341_x0901750_04:12-13`**  | " It 's the only thing that has made our lives __``remotely possible``__ and affordable and sort of possible to raise kids , " says Lopa Pal , 36 .                                                        |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_possible_ex.md`


### Saving *`remotely`* Samples

 > in `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/`

1. *remotely true*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_true_99ex~67.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_true_99ex~3.csv"`

2. *remotely close*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_close_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_close_99ex~35.csv"`

3. *remotely interested*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_interested_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_interested_99ex~35.csv"`

4. *remotely funny*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_funny_99ex~46.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_funny_99ex~6.csv"`

5. *remotely comparable*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_comparable_99ex~33.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_comparable_99ex~7.csv"`

6. *remotely ready*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_ready_99ex~16.csv"`

7. *remotely surprising*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_surprising_99ex~21.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_surprising_99ex~2.csv"`

8. *remotely similar*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_similar_99ex~41.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_similar_99ex~16.csv"`

9. *remotely possible*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/neg-remotely/neg-remotely_possible_99ex~45.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/remotely/pos-remotely/pos-remotely_possible_99ex~22.csv"`


## 6. *yet*

|                          |    `f` |   `dP1` |   `LRC` |   `P1` |      `G2` |   `f2` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `adj_total` |
|:-------------------------|-------:|--------:|--------:|-------:|----------:|-------:|----------:|------------:|------------:|--------:|-------:|--------------:|
| **NEGany~yet_clear**     | 10,406 |    0.50 |    8.43 |   1.00 | 14,363.37 | 10,411 |  5,205.50 |    5,200.50 |        0.50 |    0.00 |   0.00 |        83,958 |
| **NEGany~yet_ready**     |  7,501 |    0.50 |    8.06 |   1.00 | 10,344.71 |  7,505 |  3,752.50 |    3,748.50 |        0.50 |    0.00 |   0.00 |        29,641 |
| **NEGany~yet_available** |  7,430 |    0.50 |    8.05 |   1.00 | 10,246.19 |  7,434 |  3,717.00 |    3,713.00 |        0.50 |    0.00 |   0.00 |        81,972 |
| **NEGany~yet_complete**  |  2,174 |    0.50 |    6.70 |   1.00 |  2,998.57 |  2,175 |  1,087.50 |    1,086.50 |        0.50 |    0.00 |   0.00 |         8,263 |
| **NEGany~yet_sure**      |  1,977 |    0.50 |    6.56 |   1.00 |  2,725.53 |  1,978 |    989.00 |      988.00 |        0.50 |    0.00 |   0.00 |       134,058 |
| **NEGany~yet_certain**   |    866 |    0.50 |    5.60 |   1.00 |  1,200.65 |    866 |    433.00 |      433.00 |        0.50 |    0.00 |   0.00 |        11,301 |
| **NEGany~yet_final**     |    640 |    0.50 |    5.16 |   1.00 |    887.29 |    640 |    320.00 |      320.00 |        0.50 |    0.00 |   0.00 |         1,212 |
| **NEGany~yet_public**    |    467 |    0.50 |    4.69 |   1.00 |    647.43 |    467 |    233.50 |      233.50 |        0.50 |    0.00 |   0.00 |         2,592 |
| **NEGany~yet_eligible**  |    448 |    0.50 |    4.63 |   1.00 |    621.09 |    448 |    224.00 |      224.00 |        0.50 |    0.00 |   0.00 |         2,531 |
| **NEGany~yet_ripe**      |    380 |    0.50 |    4.39 |   1.00 |    526.81 |    380 |    190.00 |      190.00 |        0.50 |    0.00 |   0.00 |         1,454 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/ALL-yet_10mostNEG-bigrams_AMscores.2024-08-07.md`


|                 |       `N` |      `f1` |   `adv_total` |
|:----------------|----------:|----------:|--------------:|
| **NEGATED_yet** | 6,347,362 | 3,173,681 |        53,779 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/yet_MarginalFreqs_2024-08-07.md`


### 1. _yet clear_


#### 1.1. "yet clear" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                 |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_036.0294_x0565822_07:3-4-5`**    | It 's not __``yet clear``__ that any of these pieces will ultimately find their way into the Napster-Bertelsmann subscription music service slated to replace today 's file - swapping free-for-all , or even whether this subscription service will see the light of day . |
| **`pcc_eng_07_065.6324_x1044614_112:3-4-5`**   | It is not __``yet clear``__ whether Payton will be disciplined beyond the in- game penalty .                                                                                                                                                                                |
| **`pcc_eng_08_104.0587_x1668675_095:3-4-5`**   | It is not __``yet clear``__ whether grant opportunities will arise from this fund .                                                                                                                                                                                         |
| **`nyt_eng_20001122_0056_2:23-24-25`**         | the court let Gore 's candidacy live to fight another day _ five days , to be precise _ but it is not __``yet clear``__ whether that is enough time for the vice president to overcome George W. Bush 's 930-vote lead .                                                    |
| **`pcc_eng_09_088.3582_x1413556_07:14-15-16`** | Since My Space also operates a number of offices overseas , it 's not __``yet clear``__ how they were affected ( if at all ) .                                                                                                                                              |
| **`apw_eng_20090423_1539_25:3-4-5`**           | it is n't __``yet clear``__ Franck 's broke the law .                                                                                                                                                                                                                       |
| **`pcc_eng_00_002.4530_x0023404_13:24-25-26`** | But would it be the current head of the Federal drug control service Viktor Ivanov and the Federal migration service Konstantin Romodanovsky is not __``yet clear``__ .                                                                                                     |
| **`pcc_eng_09_003.1501_x0034831_21:3-4-5`**    | It is not __``yet clear``__ what charges he will face .                                                                                                                                                                                                                     |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_clear_ex.md`


#### 1.2. "yet clear" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                                                               |
|:--------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_29_019.3165_x0295575_40:14-15`** | I know you 're smart but I 've been smart a lot longer __``yet clear``__ to include inundated ... with the you may be smaller but it 's more along the inside ... very small I liked it so much that I know my email include an African and still does learn from you ... he was like the honesty and ... |
| **`pcc_eng_07_098.0439_x1568384_37:31-32`** | The lyrics , too , were beyond punk 's usual socio-political remonstrations and protestations ; they spoke of personal pressure , crisis , failure , futility ; they were abstract __``yet clear``__ , specific yet universal .                                                                           |
| **`pcc_eng_13_081.2160_x1296604_53:3-4`**   | Be precise __``yet clear``__ while writing .                                                                                                                                                                                                                                                              |
| **`pcc_eng_15_018.9778_x0290343_64:6-7`**   | The choice is stark and __``yet clear``__                                                                                                                                                                                                                                                                 |
| **`pcc_eng_23_100.5478_x1608952_20:29-30`** | The above illustration , offered in colored pencil aboard , does an exceptional job of capturing every one of the above , revealing glass containers that are substantive __``yet clear``__ as well as light .                                                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/pos-yet/pos-yet_clear_ex.md`


### 2. _yet ready_


#### 2.1. "yet ready" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                  |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_07_053.3650_x0846466_06:14-15-16`** | Details of the project are scarce at the moment , with the movie not __``yet ready``__ to hit markets .                                                                                                                                      |
| **`pcc_eng_11_012.4614_x0185189_2:11-12-13`**  | What do you do when you like something but are not __``yet ready``__ to buy it ?                                                                                                                                                             |
| **`pcc_eng_21_099.1097_x1585069_01:5-6-7`**    | While many seniors are not __``yet ready``__ to move to a nursing home , a lot are ready to transition to a retirement community .                                                                                                           |
| **`pcc_eng_05_007.6937_x0108677_23:5-6-7`**    | As our rooms are not __``yet ready``__ , we proceed to retrieve our dive gear and leave our luggage in the games room .                                                                                                                      |
| **`pcc_eng_17_076.9561_x1227450_14:09-10-11`** | Since this is an Alpha , it is not __``yet ready``__ for production use , and is not yet feature - complete ( and is likely to have bugs still ) - we are waiting for your feedback , requests , and testing to help guide our development ! |
| **`pcc_eng_22_001.1793_x0002894_36:5-6-7`**    | One reason I 'm not __``yet ready``__ to junk my laptop running Windows is the lack of applications I 've become dependent on .                                                                                                              |
| **`pcc_eng_18_087.1726_x1395444_21:19-20-21`** | The program allows physicians to apply on behalf of patients for access to drugs and devices that are n't __``yet ready``__ for clinical trials , or that are in trials that the patients do n't qualify for .                               |
| **`pcc_eng_09_088.5408_x1416467_04:09-10-11`** | Psst , people who are over Tinder but not __``yet ready``__ to join the ranks of marriage -crazed e Harmony : Ok Cupid is your new best friend .                                                                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_ready_ex.md`


#### 2.2. "yet ready" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                        |
|:--------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_06_073.7533_x1176826_06:16-17`** | The statue depicts the half-human , half - vampire son of Dracula as an elegant __``yet ready``__ for battle swordsman , with his cape revealing his three shape - shifter forms : mist , bat , and wolf !         |
| **`pcc_eng_10_087.8961_x1404453_25:1-2`**   | __``Yet ready``__ or not , here she comes .                                                                                                                                                                        |
| **`pcc_eng_18_086.8933_x1390951_35:31-32`** | Behind the mirror are two vertical drawers that pull out from either side , allowing for storage of a number of guns or other valuables to be completely concealed and __``yet ready``__ for quick , easy access . |
| **`pcc_eng_09_068.4000_x1090373_09:09-10`** | And it is extremely doubtful that Japan is __``yet ready``__ to take on the responsibilities and risks which would be involved in such an alliance .                                                               |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/pos-yet/pos-yet_ready_ex.md`


### 3. _yet available_


#### 3.1. "yet available" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                           |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_20020207_1583_30:22-23-24`**         | many have done so , and the industry has said it envisions newer and far stronger doors , although they are not __``yet available``__ .                                                                                               |
| **`pcc_eng_29_031.4354_x0491187_058:15-16-17`** | ECONOMYMauritius has one of the strongest economies in Africa ; although final figures are not __``yet available``__ , 2004 GDP at market prices is estimated at $ 6 billion and per capita income at $ 4,900 .                       |
| **`pcc_eng_16_020.6658_x0318585_35:13-14-15`**  | There are still a few cases where I need other features , not __``yet available``__ in KOffice , but they get less and less .                                                                                                         |
| **`pcc_eng_17_050.5820_x0800945_12:4-5-6`**     | The clothes are n't __``yet available``__ to buy , but the team has placed their work online so that Russian supporters can make their own DIY supporters ' strips .                                                                  |
| **`pcc_eng_10_028.3854_x0442620_11:3-4-5`**     | Endless mode not __``yet available``__ .                                                                                                                                                                                              |
| **`pcc_eng_10_024.6814_x0382616_23:4-5-6`**     | While statistics are not __``yet available``__ to show an expected increase in the number of self-reports filed in Michigan , my administration remains very confident that this is the right avenue to take to increase compliance . |
| **`pcc_eng_19_075.1543_x1197946_03:4-5-6`**     | This book is not __``yet available``__ in our membership service due to restrictions in our agreements with the publisher .                                                                                                           |
| **`pcc_eng_13_001.9269_x0014886_02:17-18-19`**  | Unfortunately , the new features are seen on the Android platform for now , and is not __``yet available``__ to the i OS users .                                                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_available_ex.md`


#### 3.2. "yet available" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                                                   |
|:---------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_25_050.0542_x0794230_08:4-5`**    | With little information __``yet available``__ about the film 's intended plot or direction , it 's been speculated by sites , including Forbes , that the movie will be an adaptation of the video adventure game , Great Detective Pikachu . |
| **`pcc_eng_02_037.1757_x0585422_05:37-38`**  | The technology , which allows for short - range wireless communication for activities like purchasing items at retail outlets , has been rumored to be included in the next i Phone but very little information is __``yet available``__ .    |
| **`pcc_eng_26_063.9026_x1017083_7:5-6`**     | Detailed images are net __``yet available``__ , but K'nex has provided a video featuring the two Titans .                                                                                                                                     |
| **`pcc_eng_17_107.07984_x1726553_11:18-19`** | The problem that every serum faces is actually the instability of Vitamin C in aqueous systems and __``yet available``__ as the ascorbic acid ( Vitamin C ) form after application .                                                          |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/pos-yet/pos-yet_available_ex.md`


### 4. _yet complete_


#### 4.1. "yet complete" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                  |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_060.7870_x0966737_06:4-5-6`**     | His treatment is not __``yet complete``__ but Herbert has greatly improved in just two short weeks on the ward .                                                                                                                                                                                             |
| **`pcc_eng_01_064.2600_x1023131_21:36-37-38`**  | A senior official , who is aware of the panel 's thinking , said they would likely not take the new failed test into consideration because the process of evaluating and confirming the sample was not __``yet complete``__ .                                                                                |
| **`pcc_eng_18_089.9291_x1440180_02:26-27-28`**  | It forms as the cell plate splitting an existing cell , one of the first steps in making the new cell wall , but is not __``yet complete``__ on its own .                                                                                                                                                    |
| **`pcc_eng_12_037.6466_x0592751_026:45-46-47`** | The choice of Rahlfs - Hanhart as the base text was a good decision by the editors and I suppose the most practical since , although a reader 's edition based on the Gottingen Septuagint would be brilliant the fact that that edition is not __``yet complete``__ makes it , as base -text , impossible . |
| **`apw_eng_20080712_0246_8:5-6-7`**             | the agreement , if not __``yet complete``__ , signals the start of the final phase of years of on-again , off-again negotiations to get North Korea to abandon its nuclear weapons program .                                                                                                                 |
| **`pcc_eng_11_014.7219_x0221942_14:30-31-32`**  | As the October 2013 term opens at the court , all of these five areas of O'Connor 's legal worldview are represented on the docket , which is still not __``yet complete``__ .                                                                                                                               |
| **`pcc_eng_15_034.0030_x0533677_33:4-5-6`**     | The jigsaw is not __``yet complete``__ , making it difficult to predict with confidence a possible starting lineup for the opening game at West Brom .                                                                                                                                                       |
| **`apw_eng_20080424_0165_6:3-4-5`**             | it was not __``yet complete``__ but was far enough along to demonstrate a resemblance to the North Korean reactor at Yonbyon .                                                                                                                                                                               |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_complete_ex.md`


#### 4.2. "yet complete" under _positive_ polarity


|                                            | `token_str`                                                                |
|:-------------------------------------------|:---------------------------------------------------------------------------|
| **`pcc_eng_26_054.2249_x0860621_117:8-9`** | ( Only three of these proceedings are __``yet complete``__ , of course . ) |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/pos-yet/pos-yet_complete_ex.md`


### 5. _yet sure_


#### 5.1. "yet sure" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                        |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19961008_0368_31:23-24-25`**         | the Packers are one of the reasons the 49ers have been worried about playing to their standard , and why they are n't __``yet sure``__ if they have met that standard .                                                                                            |
| **`pcc_eng_11_085.0898_x1361135_154:10-11-12`** | And the rest of the crowd is confused , not __``yet sure``__ if Jesus is good news or threat , and if they should seek a scapegoat around whom to coalesce and expel .                                                                                             |
| **`pcc_eng_07_105.5092_x1689351_21:29-30-31`**  | But first I will start on the mask , this part is a bit daunting , there is a lot going on shape wise , and I 'm not __``yet sure``__ how to start as a base .                                                                                                     |
| **`pcc_eng_10_074.3995_x1186415_05:15-16-17`**  | A magnetic whiteboard is useful for all those invoices and memos that you are n't __``yet sure``__ how to file .                                                                                                                                                   |
| **`pcc_eng_25_040.4359_x0638326_04:13-14-15`**  | When asked it if was a laser , she said investigators were n't __``yet sure``__ .                                                                                                                                                                                  |
| **`pcc_eng_25_094.8539_x1518859_14:7-8-9`**     | What that word is I 'm not __``yet sure``__ .                                                                                                                                                                                                                      |
| **`pcc_eng_15_014.0403_x0210293_36:3-4-5`**     | She is not __``yet sure``__ what she will make out of it , but it is sure to be unique !                                                                                                                                                                           |
| **`pcc_eng_19_048.5813_x0767876_26:42-43-44`**  | " But barring that , it seems there is some character among planets , that they are similar in some respects but different in crucial ways , that suggest something about their origin , " Burrows says , although he is not __``yet sure``__ what that might be . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_sure_ex.md`


#### 5.2. "yet sure" under _positive_ polarity


|                                             | `token_str`                                                                                                                               |
|:--------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_18_017.3700_x0265218_14:17-18`** | WITHIN THE HOUR WE'VE LEARNED TEN APD OFFICERS ARE NOW ON LEAVE , THOUGH POLICE AREN'T __``YET SURE``__ IF ALL OF THEM FIRED THEIR GUNS . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/pos-yet/pos-yet_sure_ex.md`


### 6. _yet certain_


#### 6.1. "yet certain" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                            |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_17_070.5577_x1123894_20:17-18-19`** | This feature in Apple Watch Series 4 will be a welcome improvement , but it is not __``yet certain``__ if it does not use a different sensor array than the current Apple Watch .                                                                                                                      |
| **`pcc_eng_08_077.6430_x1240917_02:33-34-35`** | How many of the numerous levies of this kind now collected will pass out of the federal fiscal picture entirely -- and how many will be assessed at lower rates -- is not __``yet certain``__ .                                                                                                        |
| **`pcc_eng_02_034.9767_x0549970_16:11-12-13`** | If you are considering a gold mining venture and are not __``yet certain``__ of your deposits , your equipment or your procedures ; we recommend gathering together the most successful , experienced consulting team you can find to assist you in putting your mining operation on the right track . |
| **`pcc_eng_01_070.1078_x1117997_10:15-16-17`** | As far as the subject matter for my imminent masterpiece goes , I 'm not __``yet certain``__ .                                                                                                                                                                                                         |
| **`pcc_eng_01_041.6896_x0657706_05:18-19-20`** | Tournaments could also be exempt from playing their annual fees to the tour , although that is not __``yet certain``__ .                                                                                                                                                                               |
| **`pcc_eng_09_036.7711_x0579038_11:4-5-6`**    | " I am not __``yet certain``__ what the future holds for my career , but I 'll always support my alma mater , and I appreciate the opportunity to help the university out now , " Fulmer said .                                                                                                        |
| **`pcc_eng_09_086.7613_x1387588_04:7-8-9`**    | Coach Peter Laviolette said he was n't __``yet certain``__ whether the three will play Saturday against Dallas at Bridgestone Arena .                                                                                                                                                                  |
| **`pcc_eng_13_030.9899_x0484988_04:21-22-23`** | The exact shape Brexit will take and the terms of any deal the UK will strike with the EU is not __``yet certain``__ .                                                                                                                                                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_certain_ex.md`


#### 6.2. "yet certain" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*yet certain*" found in **positive** hits.

### 7. _yet final_


#### 7.1. "yet final" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                               |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_28_024.8240_x0384640_155:5-6-7`**   | * All models are not __``yet final``__ and are subject to change .*                                                                                                                                       |
| **`pcc_eng_18_004.8751_x0062841_26:31-32-33`** | In fact the Johnson opinion recognizes this when it states : " Gant applies retroactively to this case , since this case is currently on direct review , and is not __``yet final``__ . "                 |
| **`pcc_eng_11_062.5953_x0996878_17:18-19-20`** | Phase II plans for Riverbend Commons , which Three Corners reports are in the works , but not __``yet final``__ .                                                                                         |
| **`pcc_eng_03_088.5052_x1417025_6:18-19-20`**  | Upon peeking inside the 2015 Mercedes AMG GT ( note that the name of the car is not __``yet final``__ ) , you can see a flat-bottomed steering wheel wrapped ... Continue Reading >>                      |
| **`pcc_eng_26_081.2761_x1297853_14:5-6-7`**    | Although the deal is not __``yet final``__ , the news is bright spot in the biofuels industry which has been overshadowed by discussion over electric vehicles as a way to pollute less and cut oil use . |
| **`pcc_eng_19_077.9347_x1242896_014:4-5-6`**   | That conviction is not __``yet final``__ . ( 2 )                                                                                                                                                          |
| **`nyt_eng_19990723_0215_2:4-5-6`**            | their plans are not __``yet final``__ , but officials say the Vineyard is the most certain among the range of options the first family is considering .                                                   |
| **`nyt_eng_19960427_0346_22:5-7-8`**           | Department officials stress that nothing is __``yet final``__ .                                                                                                                                           |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_final_ex.md`


#### 7.2. "yet final" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*yet final*" found in **positive** hits.

### 8. _yet public_


#### 8.1. "yet public" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                      |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_009.7304_x0140795_21:5-6-7`**    | Although its details are not __``yet public``__ , the New York Times reports the replacement plan would significantly scale back the CPP 's scope , focusing on reducing emissions at individual coal plants instead of establishing industry - wide emissions caps .                                                                                            |
| **`pcc_eng_27_005.0005_x0064177_16:10-11-12`** | The size or nature of the first deployment is not __``yet public``__ , though it will almost certainly contain navigation experts to operate the radar and Arctic intelligence equipment .                                                                                                                                                                       |
| **`pcc_eng_18_088.8390_x1422454_04:54-55-56`** | The report from the inspector general also criticizes Comey for not keeping his superiors at the Justice Department , including former Attorney General Loretta Lynch , properly informed about his handling of the investigation , said the person , who spoke to The Associated Press on condition of anonymity because the report is not __``yet public``__ . |
| **`apw_eng_20090728_0093_4:12-13-14`**         | the official spoke on condition of anonymity because the announcement was not __``yet public``__ .                                                                                                                                                                                                                                                               |
| **`nyt_eng_20070820_0141_11:08-09-10`**        | the details of the plea agreement are not __``yet public``__ .                                                                                                                                                                                                                                                                                                   |
| **`nyt_eng_20050907_0185_5:32-33-34`**         | and it left Mirant 's remaining band of Yahoo posters close to giddy , despite the facts that the deal has n't been finalized , could collapse and its terms are n't __``yet public``__ .                                                                                                                                                                        |
| **`pcc_eng_24_021.5524_x0331942_15:5-6-7`**    | Although the warrants are not __``yet public``__ , in both cases it is apparent that judges signed off on search warrants justifying SWAT team raids leading to the arrests .                                                                                                                                                                                    |
| **`pcc_eng_14_090.4049_x1445106_13:11-12-13`** | They declined to be named as the IPO plans were not __``yet public``__ .                                                                                                                                                                                                                                                                                         |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_public_ex.md`


#### 8.2. "yet public" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*yet public*" found in **positive** hits.

### 9. _yet eligible_


#### 9.1. "yet eligible" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                             |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_20081120_0060_26:44-45-46`**        | when Dodd asked the United Auto Workers union 's president , Ron Gettelfinger , this week whether prepackaged bankruptcy backed by federal guarantees was any more palatable , Gettelfinger cited risks to pensions and to retirees who could lose health benefits and are not __``yet eligible``__ for Medicare , the government 's health plan for the elderly .                      |
| **`pcc_eng_28_041.5928_x0656576_25:40-41-42`** | But between that provision and a cut in the total amount for federal money appropriated for subsidies ( or " tax credits " ) , the Congressional Budget Office numbers say some lower - income older Americans who are n't __``yet eligible``__ for Medicare could face out - of - pocket health costs several times as much as they do now .                                           |
| **`pcc_eng_23_099.3009_x1588862_03:11-12-13`** | You must be over 22 - years old , but not __``yet eligible``__ for Age Pension .                                                                                                                                                                                                                                                                                                        |
| **`pcc_eng_23_084.6514_x1351930_28:3-4-5`**    | These are not __``yet eligible``__ to be ranked in the Top 50 because they are not publicly traded .                                                                                                                                                                                                                                                                                    |
| **`pcc_eng_28_079.3821_x1267905_11:22-23-24`** | Though LSU redshirt sophomore DT Travonte Valentine has rejoined the Tigers , the Baton Rouge Advocate is reporting that he is not __``yet eligible``__ to play .                                                                                                                                                                                                                       |
| **`pcc_eng_19_077.4723_x1235410_49:57-58-59`** | At least 19 million early retirees , workers , their spouses , surviving spouses and dependents have had their premiums or cost sharing reduced thanks to the Early Retiree Reinsurance Program , which has provided $ 5 billion in reinsurance payments to employers so they can continue to provide benefits to their retired workers who are not __``yet eligible``__ for Medicare . |
| **`nyt_eng_19960917_0586_15:15-16-17`**        | the Treasury proposes a regulation declaring that rollovers from new employees , including those not __``yet eligible``__ to participate in the new employer 's plan , will not cause plans to run afoul of these nondiscrimination rules .                                                                                                                                             |
| **`pcc_eng_19_051.7773_x0819524_36:11-12-13`** | Because the program is so new , the degree is not __``yet eligible``__ for accreditation .                                                                                                                                                                                                                                                                                              |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_eligible_ex.md`


#### 9.2. "yet eligible" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*yet eligible*" found in **positive** hits.

### 10. _yet ripe_


#### 10.1. "yet ripe" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_19_049.5895_x0784307_16:35-36-37`** | Between 1580 and 1581 , when Browne formed in Norwich the first known church of this order on definite scriptural theory , and October 1585 , when , being convinced that the times were not __``yet ripe``__ for the realization of the perfect polity , and taking a more charitable view of the established Church , he yielded to the pressure brought to bear on him by his kinsman Lord Burghley , so far as partially to conform to parochial public worship as defined by law ( see Browne , Robert ) , the history of Congregationalism is mainly that of Browne and of his writings . |
| **`apw_eng_19980609_0665_10:13-14-15`**        | analysts yesterday said the general conditions in the mass residential market were not __``yet ripe``__ to lure speculators .                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| **`pcc_eng_03_032.7638_x0514457_14:22-23-24`** | Organizer Marco Schreuder , who posted Muller 's cancellation letter in full on his weblog , notes that the time is not __``yet ripe``__ for this matter to be discussed by the content owners , although the clock is ticking .                                                                                                                                                                                                                                                                                                                                                                |
| **`pcc_eng_15_043.3732_x0685068_39:09-10-11`** | * If your bananas are very yellow and not __``yet ripe``__ , preheat oven to 425o .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| **`pcc_eng_12_031.6467_x0496229_08:24-25-26`** | About 2 weeks ago , on one of our many walks to KK Road , we were commenting on the beautiful , though not __``yet ripe``__ , mangoes hanging from a tree in one of the yards .                                                                                                                                                                                                                                                                                                                                                                                                                 |
| **`pcc_eng_22_059.7434_x0949653_11:26-27-28`** | At a meeting in January 1930 , a joint conference of the Sheffield , Manchester and Liverpool and District Federations considered that the time was not __``yet ripe``__ for the formation of a national federation .                                                                                                                                                                                                                                                                                                                                                                           |
| **`nyt_eng_20001202_0103_27:35-36-37`**        | others might believe , as Justice Sandra Day O'Connor suggested , that whatever statutory or constitutional issues might be presented by a dispute over the legitimacy of Florida 's electors , that dispute is not __``yet ripe``__ for adjudication .                                                                                                                                                                                                                                                                                                                                         |
| **`pcc_eng_16_027.5291_x0429412_21:5-6-7`**    | But the conditions were not __``yet ripe``__ for massive outsourcing and offshoring to be profitable for British business and the British government .                                                                                                                                                                                                                                                                                                                                                                                                                                          |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_ripe_ex.md`


#### 10.2. "yet ripe" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*yet ripe*" found in **positive** hits.

### Saving *`yet`* Samples

 > in `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/`

1. *yet clear*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_clear_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/pos-yet/pos-yet_clear_99ex~5.csv"`

2. *yet ready*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_ready_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/pos-yet/pos-yet_ready_99ex~4.csv"`

3. *yet available*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_available_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/pos-yet/pos-yet_available_99ex~4.csv"`

4. *yet complete*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_complete_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/pos-yet/pos-yet_complete_99ex~1.csv"`

5. *yet sure*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_sure_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/pos-yet/pos-yet_sure_99ex~1.csv"`

6. *yet certain*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_certain_99ex~99.csv"`

7. *yet final*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_final_99ex~99.csv"`

8. *yet public*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_public_99ex~99.csv"`

9. *yet eligible*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_eligible_99ex~99.csv"`

10. *yet ripe*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/yet/neg-yet/neg-yet_ripe_99ex~99.csv"`


## 7. *immediately*

|                                    |    `f` |   `dP1` |   `LRC` |   `P1` |      `G2` |   `f2` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `adj_total` |
|:-----------------------------------|-------:|--------:|--------:|-------:|----------:|-------:|----------:|------------:|------------:|--------:|-------:|--------------:|
| **NEGany~immediately_clear**       | 24,416 |    0.50 |    7.74 |   1.00 | 33,183.58 | 24,476 | 12,238.00 |   12,178.00 |        0.50 |    0.01 |   0.01 |        83,958 |
| **NEGany~immediately_available**   | 21,078 |    0.49 |    6.17 |   0.99 | 27,277.70 | 21,281 | 10,640.50 |   10,437.50 |        0.50 |    0.01 |   0.01 |        81,972 |
| **NEGany~immediately_possible**    |  1,000 |    0.50 |    5.57 |   1.00 |  1,372.02 |  1,001 |    500.50 |      499.50 |        0.50 |    0.00 |   0.00 |        30,222 |
| **NEGany~immediately_obvious**     |  2,238 |    0.47 |    4.09 |   0.97 |  2,561.91 |  2,310 |  1,155.00 |    1,083.00 |        0.48 |    0.00 |   0.00 |        22,422 |
| **NEGany~immediately_able**        |    626 |    0.47 |    3.43 |   0.97 |    717.23 |    646 |    323.00 |      303.00 |        0.48 |    0.00 |   0.00 |        22,885 |
| **NEGany~immediately_successful**  |    290 |    0.48 |    3.21 |   0.98 |    358.28 |    295 |    147.50 |      142.50 |        0.49 |    0.00 |   0.00 |        31,408 |
| **NEGany~immediately_sure**        |    138 |    0.49 |    2.63 |   0.99 |    180.84 |    139 |     69.50 |       68.50 |        0.50 |    0.00 |   0.00 |       134,058 |
| **NEGany~immediately_forthcoming** |    133 |    0.49 |    2.39 |   0.99 |    166.33 |    135 |     67.50 |       65.50 |        0.49 |    0.00 |   0.00 |         2,241 |
| **NEGany~immediately_reachable**   |    109 |    0.49 |    2.26 |   0.99 |    141.10 |    110 |     55.00 |       54.00 |        0.50 |    0.00 |   0.00 |           342 |
| **NEGany~immediately_certain**     |     70 |    0.50 |    1.80 |   1.00 |     97.04 |     70 |     35.00 |       35.00 |        0.50 |    0.00 |   0.00 |        11,301 |
| **NEGmir~immediately_available**   |    162 |    0.37 |    1.22 |   0.87 |    112.16 |    187 |     93.50 |       68.50 |        0.42 |    0.00 |   0.00 |         3,078 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/ALL-immediately_10mostNEG-bigrams_AMscores.2024-08-07.md`


|                         |       `N` |      `f1` |   `adv_total` |
|:------------------------|----------:|----------:|--------------:|
| **NEGATED_immediately** | 6,347,362 | 3,173,681 |        57,730 |
| **NEGMIR_immediately**  |   583,470 |   291,735 |           578 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/immediately_MarginalFreqs_2024-08-07.md`


### 1. _immediately clear_


#### 1.1. "immediately clear" under _negative_ polarity


|                                               | `token_str`                                                                                                                                                                                                                       |
|:----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_035.8135_x0563196_6:17-18-19`** | When we see strange features on the floor of one of these depressions , it is not __``immediately clear``__ if it is because of something from deep down being exposed or something relatively young having been deposited here . |
| **`pcc_eng_04_078.6602_x1254527_06:3-4-5`**   | It 's not __``immediately clear``__ if the law applies to the case of a mural , but if it does things could get interesting .                                                                                                     |
| **`apw_eng_20020202_0269_10:3-4-5`**          | it was n't __``immediately clear``__ if she had retained a new lawyer .                                                                                                                                                           |
| **`pcc_eng_03_002.3305_x0021513_4:6-7-8`**    | What these measures are is n't __``immediately clear``__ , and as Forbes stated , " these sorts of things are earlier on in design , so do n't expect much on this front for a little while . "                                   |
| **`apw_eng_19970712_0171_8:3-4-5`**           | it was not __``immediately clear``__ how the power failure was connected to the jail break .                                                                                                                                      |
| **`apw_eng_20031122_0228_11:4-5-6`**          | it was also not __``immediately clear``__ whether Montenegrin authorities would now reopen the case .                                                                                                                             |
| **`apw_eng_20020607_1645_17:11-12-13`**       | the extent of U.S. involvement in the ongoing offensive was n't __``immediately clear``__ .                                                                                                                                       |
| **`apw_eng_20030508_0832_17:11-12-13`**       | Flores said the extent of damage to the plant was not __``immediately clear``__ .                                                                                                                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_clear_ex.md`


#### 1.2. "immediately clear" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                  |
|:--------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_01_050.3731_x0797790_11:8-9`**   | The potential value of the app was __``immediately clear``__ to me , especially considering how much food is wasted and how this contributes to increased energy use and greenhouse gas emissions .                          |
| **`pcc_eng_08_010.3273_x0150781_53:13-14`** | The balance felt right and organic , perhaps because it was almost __``immediately clear``__ that narrative was n't driving the season .                                                                                     |
| **`nyt_eng_20010824_0166_6:29-30`**         | regardless of how you feel about `` Apocalypse Now '' -LRB- I thought it a great film then and an even greater one now -RRB- , it 's __``immediately clear``__ what sets it apart from this season 's puny crop _ ambition . |
| **`pcc_eng_06_058.5155_x0930615_45:09-10`** | But as the second round began it was __``immediately clear``__ it was Firpo , not Dempsey , who was about to be knocked out .                                                                                                |
| **`pcc_eng_24_082.5384_x1318830_20:5-6`**   | And , it was __``immediately clear``__ that place-mats this beautiful belonged on a party table .                                                                                                                            |
| **`pcc_eng_10_097.8182_x1565284_06:3-4`**   | It was __``immediately clear``__ that this beautiful young woman , with her pleasant personality and superb voice , would have a very successful singing career .                                                            |
| **`pcc_eng_25_065.5456_x1045670_06:09-10`** | Our scan of the data makes two points __``immediately clear``__ :                                                                                                                                                            |
| **`nyt_eng_20041020_0402_44:3-4`**          | it 's __``immediately clear``__ why Sotho-speaking tribesmen in the 19th century named it `` Mosi-oa-tunya '' -- `` the smoke that thunders . ''                                                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_clear_ex.md`


### 2. _immediately available_


#### 2.1. "immediately available" under _negative_ polarity


|                                                | `token_str`                                                                                                        |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_20020629_0521_21:09-10-11`**        | comment from the Italian chamber of deputies was not __``immediately available``__ .                               |
| **`pcc_eng_14_002.4237_x0023100_6:10-11-12`**  | Further details on the accident in College Station were n't __``immediately available``__ .                        |
| **`apw_eng_19980406_0840_11:13-14-15`**        | details of Van Ingen Schenau 's funeral arrangements and surviving family were not __``immediately available``__ . |
| **`pcc_eng_28_046.2979_x0732933_49:13-14-15`** | Information about any payments he may receive due to his departure was not __``immediately available``__ .         |
| **`apw_eng_19981108_0806_5:11-12-13`**         | spokesmen for the ministry of interior and the police were not __``immediately available``__ .                     |
| **`apw_eng_19970406_0025_6:5-6-7`**            | U.S. military officials were not __``immediately available``__ for comment .                                       |
| **`apw_eng_19980907_0543_4:6-7-8`**            | Ministry spokeswoman Tova Elinson was not __``immediately available``__ for comment .                              |
| **`apw_eng_19980526_0854_9:7-8-9`**            | the identities of the injured were not __``immediately available``__ .                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_available_ex.md`


#### 2.2. "immediately available" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                                                                                                                             |
|:--------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_05_050.3097_x0797861_14:5-6`**   | While one spot is __``immediately available``__ should the need arise , H.H. Williams Memorial Hospital also has room for two more drops .                                                                                                                                                                                                                              |
| **`pcc_eng_24_105.4022_x1689127_10:24-25`** | As a cow 's blood calcium levels become quickly depleted via the loss of heavy lactation ( blood calcium is the active and __``immediately available``__ form of the mineral as opposed to the calcium deposits in bones ) , her body 's homeostatic mechanisms are n't able to compensate and she ca n't draw on calcium stores from her skeletal system fast enough . |
| **`pcc_eng_22_101.4146_x1622678_16:39-40`** | " And there are a lot of Guard personnel over there with the same uniform needs as any other Soldier , " explained Zabaldo , before adding , " So having these unit crests on - hand and __``immediately available``__ makes us an important resource for these folks . "                                                                                               |
| **`pcc_eng_12_102.9027_x1646888_5:3-4`**    | Sales are __``immediately available``__ in the POS System , creating a completely unified solution allowing for customers to click and collect their orders and customer returns to happen on any channel .                                                                                                                                                             |
| **`pcc_eng_07_041.0153_x0646996_25:7-8`**   | Once memorized , these facts become __``immediately available``__ for use provided the nature and manner of use is understood .                                                                                                                                                                                                                                         |
| **`pcc_eng_18_019.5126_x0299711_10:22-23`** | For example , Daphne presented to the audience the ways blockchain could improve the logistics process just by making shipping information __``immediately available``__ to all stakeholders ( manufacturers , middle-men , and consumers ) via QR codes .                                                                                                              |
| **`pcc_eng_00_027.3616_x0426160_05:09-10`** | Facebook Local Awareness , presented last October and __``immediately available``__ in the USA , is now also active all countries .                                                                                                                                                                                                                                     |
| **`pcc_eng_04_061.3428_x0974966_2:5-6`**    | The sessions below are __``immediately available``__ for download , and they NEVER expire .                                                                                                                                                                                                                                                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_available_ex.md`


### 3. _immediately possible_


#### 3.1. "immediately possible" under _negative_ polarity


|                                             | `token_str`                                                                                                                                                                                            |
|:--------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_19970907_0270_8:17-18-19`**      | there were reports of numerous victims of clashes between troops and protesters , but it was not __``immediately possible``__ to ascertain how many people were killed , either military or civilian . |
| **`pcc_eng_15_011.0648_x0162486_47:3-4-5`** | It was not __``immediately possible``__ to assess the extent of the damage or independently confirm who hit it .                                                                                       |
| **`apw_eng_19980331_0059_5:3-4-5`**         | it was n't __``immediately possible``__ to confirm the report independently .                                                                                                                          |
| **`apw_eng_19980514_0755_4:3-4-5`**         | it was not __``immediately possible``__ to obtain government comment .                                                                                                                                 |
| **`apw_eng_19980421_0416_15:3-4-5`**        | it was not __``immediately possible``__ to determine whether victors in northern Ugandan municipal races were multiparty supporters or Movement people .                                               |
| **`apw_eng_20080623_0511_8:4-5-6`**         | but it was not __``immediately possible``__ to determine whether the Tripoli violence represents an isolated event or a residue of last month 's clashes that preceded the deal .                      |
| **`pcc_eng_24_081.5379_x1302666_06:3-4-5`** | It was not __``immediately possible``__ to verify the eyewitnesses account .                                                                                                                           |
| **`apw_eng_20020720_0252_5:3-4-5`**         | it was not __``immediately possible``__ to reach the company .                                                                                                                                         |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_possible_ex.md`


#### 3.2. "immediately possible" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                 |
|:--------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_19_014.1126_x0211876_33:29-30`** | But we still need political dreams to drive the national debate , to spur engagement , and these dreams must celebrate ideals that are always beyond what is __``immediately possible``__ . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_possible_ex.md`


### 4. _immediately obvious_


#### 4.1. "immediately obvious" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                       |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_21_015.4807_x0233819_130:21-22-23`** | Through viewing the portraits side-by-side , exhibition curators hope to illuminate the links between the two artists , which are not __``immediately obvious``__ .                                               |
| **`pcc_eng_24_024.6830_x0382878_14:16-18-19`**  | The projects that correct , adapt , or extend existing software in ways that may not be __``immediately obvious``__ to the end-user . -                                                                           |
| **`pcc_eng_11_061.5327_x0979620_08:28-31-32`**  | As an instrumentalist with perfect pitch , he is perfectly able to work from the midst of an ensemble to spot inconsistencies and tuning issues that might not always be __``immediately obvious``__ .            |
| **`pcc_eng_02_091.9550_x1470532_86:3-4-5`**     | It is n't __``immediately obvious``__ , to contemporary eyes , what made the art .                                                                                                                                |
| **`pcc_eng_21_073.1671_x1166364_151:7-8-9`**    | These is another benefit that is n't __``immediately obvious``__ , however .                                                                                                                                      |
| **`pcc_eng_01_062.0575_x0987671_134:29-31-32`** | This guide is not a substitute for hard work , but rather an aid " to provide concise explanations of syntactical , clause - level features that may not be __``immediately obvious``__ to the beginner " ( 8 ) . |
| **`pcc_eng_23_087.2389_x1393879_032:4-5-6`**    | The rationale is n't __``immediately obvious``__ to me as , irrespective of the left - hand drive versus right - hand drive issue , I do n't think either would get in the way of the driver .                    |
| **`pcc_eng_17_075.8485_x1209707_085:10-11-12`** | Chiron 's arrival has created a shift that is not __``immediately obvious``__ on the individual level but all the more evident in the world at large by adding a wounded image to masculinity .                   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_obvious_ex.md`


#### 4.2. "immediately obvious" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                              |
|:--------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_076.4914_x1220487_03:5-6`**   | One thing that is __``immediately obvious``__ from the table is that there is substantial variation in unemployment rates across groups .                                                                                                |
| **`pcc_eng_15_062.2549_x0989480_25:4-5`**   | Big life-events were __``immediately obvious``__ , but why did I dip down consistently for that week , spike over there , go bananas for the first two weeks here , and then dropped down to dribbles for the rest of that month ?       |
| **`pcc_eng_06_044.0682_x0696601_17:14-15`** | Sectors such as retail , logistics , and financial services may have more __``immediately obvious``__ opportunities for automation , and are more likely than healthcare to see widespread adoption within the next one to three years . |
| **`pcc_eng_27_071.1509_x1134252_13:15-16`** | The fictitious Flay 's is actually Delphine at the W Hotel in Hollywood -- __``immediately obvious``__ when Ari and Floyd walk in , regardless of the giant Flay 's signage on the wall .                                                |
| **`pcc_eng_08_003.3255_x0037730_3:15-16`**  | Sussex Beds ( tm ) is a true independent family business and this is __``immediately obvious``__ with its warm welcome & relaxed shopping experience .                                                                                   |
| **`pcc_eng_24_036.1843_x0569038_54:8-9`**   | When it 's served , it 's __``immediately obvious``__ that this is not the bulgogi found everywhere else .                                                                                                                               |
| **`pcc_eng_26_059.4619_x0945282_21:7-8`**   | " With Mackie , it is __``immediately obvious``__ from the name what it does .                                                                                                                                                           |
| **`pcc_eng_01_021.7735_x0335945_28:7-8`**   | Improved breast size and shape is __``immediately obvious``__ after surgery and most patients experience increased confidence and satisfaction with the appearance of their breasts .                                                    |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_obvious_ex.md`


### 5. _immediately able_


#### 5.1. "immediately able" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_025.2597_x0392219_17:21-22-23`** | ( Huff Post messaged the user , who had posted about ROF months before Wednesday 's shooting , but was not __``immediately able``__ to confirm his identity . )                                                            |
| **`pcc_eng_18_002.7835_x0029044_7:17-18-19`**  | A spokesman for the army confirmed that there was gunfire in the Adiake barracks but was not __``immediately able``__ to confirm that a mutiny had occurred .                                                              |
| **`pcc_eng_20_017.5635_x0267518_06:3-4-5`**    | Reuters was not __``immediately able``__ to reach city press officers for comment .                                                                                                                                        |
| **`apw_eng_20021222_0283_3:5-6-7`**            | the Associated Press was not __``immediately able``__ to reach Jordanian officials for comment .                                                                                                                           |
| **`pcc_eng_12_013.2382_x0198068_196:3-4-5`**   | Reuters was not __``immediately able``__ to verify the footage .                                                                                                                                                           |
| **`nyt_eng_19970804_0800_32:5-6-7`**           | Guinness Publishing Ltd. was not __``immediately able``__ to identify the new holder of the age record , but the firm is checking other unconfirmed reports , including a woman living in California who said she is 118 . |
| **`pcc_eng_09_030.0637_x0470491_18:3-4-5`**    | Police were n't __``immediately able``__ to identify the gunman and were trying to determine what , if any , relationship he had with the nursing home , Thorp said .                                                      |
| **`pcc_eng_06_100.7497_x1613516_08:6-7-8`**    | The Russian Defence Ministry was not __``immediately able``__ to comment when contacted by Reuters .                                                                                                                       |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_able_ex.md`


#### 5.2. "immediately able" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|:--------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_25_012.9194_x0192801_09:09-10`** | Once we started doing this , we were __``immediately able``__ to start bidding competitively for contracts . "                                                                                                                                                                                                                                                                                                                                                                                                                                |
| **`pcc_eng_12_042.7115_x0674577_5:14-15`**  | The flasks are a unique memento that 's event- appropriate ( riders were __``immediately able``__ to fill their flasks with " cold ' recovery ' beverages " ) and that the recipients would appreciate and be proud to keep .                                                                                                                                                                                                                                                                                                                 |
| **`pcc_eng_01_038.1070_x0599658_70:11-12`** | Time magazine 's rival list of candidates , I was __``immediately able``__ to eliminate Steve Jobs , Bruce Springsteen , George Clooney , Rem Koolhaas , " Brad and Angelina , " Oprah Winfrey , and even Laura Bush 's library - science nominee , the Afghan novelist Khaled Hosseini -- along with several dozen others among the world 's current , media-recognized , " leaders and revolutionaries , heroes and pioneers , scientists and thinkers , artists and entertainers , builders and titans , " to say nothing of their chefs . |
| **`pcc_eng_05_012.9022_x0192982_38:14-15`** | What is known is that , once they hatch , the juveniles are __``immediately able``__ to walk , run and jump .                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| **`pcc_eng_09_001.5600_x0009074_08:27-28`** | " We think with the Nokia brand and the combination of industry leading performance as well as immersive and innovative consumer experience , we will be __``immediately able``__ to make a mark in the high end segment , " Florian Seiche , acting chief executive of HMD Global , told CNBC in an interview ahead of the launch .                                                                                                                                                                                                          |
| **`pcc_eng_23_072.7485_x1159162_23:13-14`** | " This rhythm section has had so much experience that they 're __``immediately able``__ to get down to what you want to do , " he 's quoted as saying in Nat Hentoff 's original liner notes .                                                                                                                                                                                                                                                                                                                                                |
| **`pcc_eng_27_050.6509_x0802420_26:4-5`**   | New members are __``immediately able``__ to participate as equal peers on the network .                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| **`pcc_eng_22_017.4285_x0265002_22:3-4`**   | I was __``immediately able``__ to start throwing without the pain I am usually subject to .                                                                                                                                                                                                                                                                                                                                                                                                                                                   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_able_ex.md`


### 6. _immediately successful_


#### 6.1. "immediately successful" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_006.0149_x0081010_09:11-12-13`**  | Pear Jam's August 1991 released album " Ten " though not __``immediately successful``__ , by late 1992 had reached number two on the Billboard 200 chart . |
| **`pcc_eng_23_009.0502_x0129932_252:10-11-12`** | Hunter tried contacting someone at the center but was not __``immediately successful``__ .                                                                 |
| **`pcc_eng_05_092.0968_x1473834_06:09-10-11`**  | Efforts to reach AZ Patriots for comment were not __``immediately successful``__ .                                                                         |
| **`pcc_eng_01_044.4283_x0701611_18:6-7-8`**     | Efforts to contact him were not __``immediately successful``__ Wednesday .                                                                                 |
| **`pcc_eng_13_096.6423_x1545606_13:15-16-17`**  | Attempts to contact Mayhew , 69 , for comment - or roar - were not __``immediately successful``__ .                                                        |
| **`apw_eng_20090619_1275_21:12-13-14`**         | an attempt to find a listed phone number for Easton was n't __``immediately successful``__ .                                                               |
| **`pcc_eng_27_027.5533_x0428933_25:7-8-9`**     | Attempts to reach town officials were not __``immediately successful``__ Tuesday .                                                                         |
| **`pcc_eng_18_002.8745_x0030500_20:15-16-17`**  | An attempt to reach an EPA spokesman for reaction to Mead 's comments was not __``immediately successful``__ on Friday .                                   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_successful_ex.md`


#### 6.2. "immediately successful" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                |
|:--------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_035.5287_x0559054_14:5-6`**   | Bohn 's series was __``immediately successful``__ and spawned numerous additional series ( with the first and last year of publication for titles in the series , if known ; arranged by year series was first offered ) : |
| **`nyt_eng_19980401_0143_12:7-8`**          | in 1989 , the formula proved __``immediately successful``__ .                                                                                                                                                              |
| **`nyt_eng_19980313_0436_110:2-3`**         | Q. __``Immediately successful``__ ?                                                                                                                                                                                        |
| **`pcc_eng_29_104.6657_x1675109_03:6-7`**   | The Stratos proved to be __``immediately successful``__ in world rally competition .                                                                                                                                       |
| **`pcc_eng_24_102.3009_x1638919_09:10-11`** | Such variety helped attract new readers : Jocoseria was __``immediately successful``__ , and a second edition was printed in the same year as the first .                                                                  |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_successful_ex.md`


### 7. _immediately sure_


#### 7.1. "immediately sure" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                       |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_20090812_0241_8:3-4-5`**            | officials were not __``immediately sure``__ what caused the crash .                                                                                                                                                                                                                               |
| **`pcc_eng_14_087.1863_x1393018_07:4-5-6`**    | If you are n't __``immediately sure``__ what kind of fence will provide your house the greatest visual appeal while still accommodating your budgetary needs , then here 's a complete analysis on choosing a fencing design .                                                                    |
| **`pcc_eng_09_085.1182_x1360927_28:33-34-35`** | By telling us these stories , some of which she borrowed from family and friends , in the second person , Rankine builds her book on shifting sands -- the reader is never __``immediately sure``__ who in the story is black or white , whom to identify with , whom to trust or fear .          |
| **`pcc_eng_08_102.9408_x1650453_07:4-5-6`**    | If you are not __``immediately sure``__ what type of fence will provide your property the best visual appeal while adapting your budgetary needs , then here 's a complete analysis on choosing a fencing layout .                                                                                |
| **`pcc_eng_03_034.2609_x0538764_26:17-18-19`** | When reached Tuesday by this publication , Tina Pelkey , press secretary for Pai , was not __``immediately sure``__ whether any public hearings are scheduled before the Dec. 14 vote .                                                                                                           |
| **`pcc_eng_21_015.8324_x0239430_08:4-5-6`**    | If you are n't __``immediately sure``__ what kind of fence will give your house the best visual appeal while adapting your budgetary needs , then here 's a full analysis on selecting a fence layout .                                                                                           |
| **`nyt_eng_19971205_0024_4:25-26-27`**         | after the Northridge Earthquake in 1994 , it took nearly 30 minutes to pinpoint the epicenter and magnitude , but even then scientists were not __``immediately sure``__ where the shaking had been strongest .                                                                                   |
| **`pcc_eng_09_031.6610_x0496365_05:21-22-23`** | What we have been told is that the administrative burden of making a claim will be reduced - I 'm not __``immediately sure``__ what that really means , albeit we do know that there is a move to look at the way in which larger claimants are required to keep records relating to R&D claims . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_sure_ex.md`


#### 7.2. "immediately sure" under _positive_ polarity


|                                           | `token_str`                                                         |
|:------------------------------------------|:--------------------------------------------------------------------|
| **`pcc_eng_16_081.4859_x1302665_06:7-8`** | If she does , you 're __``immediately sure``__ who Black Alyss is ! |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_sure_ex.md`


### 8. _immediately forthcoming_


#### 8.1. "immediately forthcoming" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                      |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_19_040.0503_x0630071_2:24-26-27`**   | Palestinian Prime Minister Ahmed Qurei also signaled Saturday that he was not opposed to a meeting with Prime Minister Ariel Sharon , although none was __``immediately forthcoming``__ , and confirmed that there were contacts between the two sides .         |
| **`pcc_eng_16_027.8615_x0434864_013:39-40-41`** | He has n't lived up to his father 's expectations , that 's clear , but there 's a more specific history between them , one obviously tied to a girl named Iris , but the details are n't __``immediately forthcoming``__ .                                      |
| **`pcc_eng_04_054.7729_x0868504_11:5-6-7`**     | Hearing that enhancements were not __``immediately forthcoming``__ , one forum member wrote , " I was rather disappointed to see the way the TFTP feature is implemented . "                                                                                     |
| **`pcc_eng_05_034.7526_x0546596_10:7-8-9`**     | When the petting she wants is not __``immediately forthcoming``__ , she complains .                                                                                                                                                                              |
| **`nyt_eng_19961016_0296_6:12-13-14`**          | an explanation for the unexpectedly wide trade deficit in September was n't __``immediately forthcoming``__ from trade officials .                                                                                                                               |
| **`pcc_eng_08_106.7057_x1711593_08:3-4-5`**     | Support was not __``immediately forthcoming``__ from Hall 's former teammates .                                                                                                                                                                                  |
| **`pcc_eng_13_007.1669_x0099461_01:37-39-40`**  | The Air Force 's newest air superiority fighter under development , the F-22 Raptor , has passed an important combat avionics test , but officials within the program say the decision to begin limited production will not be __``immediately forthcoming``__ . |
| **`pcc_eng_21_017.1379_x0260519_24:4-5-6`**     | When it was not __``immediately forthcoming``__ , Fletcher sent on Madjo for Africa- bound Bhasera , giving himself , effectively , four forwards .                                                                                                              |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_forthcoming_ex.md`


#### 8.2. "immediately forthcoming" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                       |
|:--------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_18_032.9117_x0516536_07:29-30`** | " Shortly after Project Healing Waters Fly Fishing 's inception in 2005 , our group was introduced to the Orvis Company and solicited their support , which was __``immediately forthcoming``__ , " said retired U.S. Navy Capt . |
| **`apw_eng_20020211_1473_30:14-15`**        | he was happy to join the initial Somalia mission in January 1993 and __``immediately forthcoming``__ about who his father was .                                                                                                   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_forthcoming_ex.md`


### 9. _immediately reachable_


#### 9.1. "immediately reachable" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                   |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_096.6577_x1546677_08:5-6-7`**    | Hybrid Air Vehicles was not __``immediately reachable``__ by telephone .                                                                                                                      |
| **`pcc_eng_19_072.1259_x1148812_07:4-5-6`**    | Algerian officials were not __``immediately reachable``__ for comment .                                                                                                                       |
| **`pcc_eng_26_040.5565_x0639587_11:12-13-14`** | Semper Fi Management is headed by Frank Palazzolo , who was not __``immediately reachable``__ for comment .                                                                                   |
| **`pcc_eng_27_003.0636_x0033182_06:5-6-7`**    | Ethiopian government officials were not __``immediately reachable``__ for reactions .                                                                                                         |
| **`pcc_eng_11_012.1120_x0179638_09:4-5-6`**    | The club was not __``immediately reachable``__ for comment .                                                                                                                                  |
| **`pcc_eng_07_068.7921_x1095661_30:6-7-8`**    | Ferguson 's old assistant was not __``immediately reachable``__ .                                                                                                                             |
| **`pcc_eng_28_013.3710_x0200419_5:09-10-11`**  | The U.S. Attorney 's Office in Manhattan was not __``immediately reachable``__ for comment .                                                                                                  |
| **`pcc_eng_08_044.1103_x0697589_24:09-10-11`** | Reuters said that officials from Blizzard were " not __``immediately reachable``__ for comment , though its customer support Twitter account said the company 's servers were stabilizing . " |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_reachable_ex.md`


#### 9.2. "immediately reachable" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                 |
|:--------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_058.2919_x0926772_09:32-33`** | It comes equipped with a rear zippered pocket for those days when you 'll be using it as the main outer layer and you want to have your essentials safe and __``immediately reachable``__ . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_reachable_ex.md`


### 10. _immediately certain_


#### 10.1. "immediately certain" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                     |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_20091226_0014_22:20-21-22`**        | a law enforcement source said the explosives may have been strapped to the man 's body but investigators were n't __``immediately certain``__ , partly because of the struggle with other passengers .                                                                                                          |
| **`apw_eng_19971018_0570_9:3-4-5`**            | police were not __``immediately certain``__ if the claim was true .                                                                                                                                                                                                                                             |
| **`pcc_eng_07_028.7656_x0449158_10:18-19-20`** | Brown said district officials had hoped to resume classes this week , and she said it was n't __``immediately certain``__ if the layoff would add days to the end of the school year .                                                                                                                          |
| **`pcc_eng_12_066.5513_x1059416_7:3-4-5`**     | It was n't __``immediately certain``__ that Chase and D'Angelo would be able to return to the franchise , given their price tag and that the parts are more on the level of a cameo , but once the final script and budget were in place , producers went after them once again to ensure their participation . |
| **`pcc_eng_08_072.3121_x1154626_30:3-4-5`**    | It was not __``immediately certain``__ if he would attend the next talks though .                                                                                                                                                                                                                               |
| **`apw_eng_19980723_1334_4:5-6-7`**            | he said it was not __``immediately certain``__ whether the accident happened during landing or take-off .                                                                                                                                                                                                       |
| **`nyt_eng_19960825_0490_3:4-5-6`**            | the cause was not __``immediately certain``__ , but he had been bedridden for three weeks with congestion of the lungs , said a friend , the photographer Cornell Capa , who was the founding director of the International Center of Photography in Manhattan , where Phillips lectured .                      |
| **`apw_eng_20091023_0366_3:12-13-14`**         | Zabit Khan says the exact cause of the blast Friday was not __``immediately certain``__ but that it could have been a remote-controlled bomb .                                                                                                                                                                  |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_certain_ex.md`


#### 10.2. "immediately certain" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*immediately certain*" found in **positive** hits.

### Saving *`immediately`* Samples

 > in `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/`

1. *immediately clear*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_clear_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_clear_99ex~60.csv"`

2. *immediately available*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_available_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_available_99ex~99.csv"`

3. *immediately possible*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_possible_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_possible_99ex~1.csv"`

4. *immediately obvious*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_obvious_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_obvious_99ex~72.csv"`

5. *immediately able*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_able_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_able_99ex~20.csv"`

6. *immediately successful*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_successful_99ex~84.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_successful_99ex~5.csv"`

7. *immediately sure*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_sure_99ex~36.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_sure_99ex~1.csv"`

8. *immediately forthcoming*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_forthcoming_99ex~33.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_forthcoming_99ex~2.csv"`

9. *immediately reachable*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_reachable_99ex~32.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/pos-immediately/pos-immediately_reachable_99ex~1.csv"`

10. *immediately certain*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/immediately/neg-immediately/neg-immediately_certain_99ex~21.csv"`


## 8. *ever*

|                         |   `f` |   `dP1` |   `LRC` |   `P1` |   `G2` |   `f2` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `adj_total` |
|:------------------------|------:|--------:|--------:|-------:|-------:|-------:|----------:|------------:|------------:|--------:|-------:|--------------:|
| **NEGmir~ever_easy**    |   368 |    0.50 |    4.46 |   1.00 | 510.39 |    368 |    184.00 |      184.00 |        0.50 |    0.00 |   0.00 |         7,897 |
| **NEGmir~ever_good**    |   299 |    0.50 |    3.90 |   1.00 | 402.64 |    300 |    150.00 |      149.00 |        0.50 |    0.00 |   0.00 |        13,484 |
| **NEGany~ever_easy**    |   429 |    0.49 |    3.90 |   0.99 | 554.86 |    433 |    216.50 |      212.50 |        0.50 |    0.00 |   0.00 |       109,374 |
| **COM~ever_closer**     |   297 |    0.49 |    3.61 |   0.99 | 390.50 |    299 |    149.50 |      147.50 |        0.50 |    0.00 |   0.00 |         3,488 |
| **NEGmir~ever_perfect** |   206 |    0.50 |    3.59 |   1.00 | 285.65 |    206 |    103.00 |      103.00 |        0.50 |    0.00 |   0.00 |         1,280 |
| **NEGmir~ever_simple**  |   206 |    0.50 |    3.59 |   1.00 | 285.65 |    206 |    103.00 |      103.00 |        0.50 |    0.00 |   0.00 |         7,580 |
| **NEGany~ever_simple**  |   211 |    0.50 |    3.28 |   1.00 | 281.19 |    212 |    106.00 |      105.00 |        0.50 |    0.00 |   0.00 |        47,134 |
| **NEGany~ever_perfect** |   216 |    0.49 |    3.13 |   0.99 | 279.47 |    218 |    109.00 |      107.00 |        0.50 |    0.00 |   0.00 |        12,730 |
| **NEGmir~ever_enough**  |   147 |    0.50 |    3.08 |   1.00 | 203.82 |    147 |     73.50 |       73.50 |        0.50 |    0.00 |   0.00 |         1,334 |
| **NEGmir~ever_certain** |   143 |    0.50 |    3.04 |   1.00 | 198.28 |    143 |     71.50 |       71.50 |        0.50 |    0.00 |   0.00 |         1,284 |
| **COM~ever_greater**    |   169 |    0.49 |    2.94 |   0.99 | 223.41 |    170 |     85.00 |       84.00 |        0.50 |    0.00 |   0.00 |         6,678 |
| **COM~ever_larger**     |   113 |    0.50 |    2.56 |   1.00 | 156.65 |    113 |     56.50 |       56.50 |        0.50 |    0.00 |   0.00 |         7,136 |
| **NEGmir~ever_wrong**   |   102 |    0.50 |    2.52 |   1.00 | 141.42 |    102 |     51.00 |       51.00 |        0.50 |    0.00 |   0.00 |         8,526 |
| **COM~ever_higher**     |   123 |    0.49 |    2.45 |   0.99 | 160.27 |    124 |     62.00 |       61.00 |        0.50 |    0.00 |   0.00 |        13,093 |
| **NEGany~ever_certain** |   147 |    0.48 |    2.40 |   0.98 | 178.54 |    150 |     75.00 |       72.00 |        0.49 |    0.00 |   0.00 |        11,301 |
| **NEGmir~ever_able**    |   136 |    0.48 |    2.37 |   0.98 | 163.78 |    139 |     69.50 |       66.50 |        0.49 |    0.00 |   0.00 |         1,850 |
| **NEGmir~ever_free**    |    69 |    0.50 |    1.90 |   1.00 |  95.66 |     69 |     34.50 |       34.50 |        0.50 |    0.00 |   0.00 |         1,688 |
| **NEGany~ever_sure**    |    87 |    0.49 |    1.90 |   0.99 | 111.05 |     88 |     44.00 |       43.00 |        0.49 |    0.00 |   0.00 |       134,058 |
| **COM~ever_deeper**     |    77 |    0.49 |    1.70 |   0.99 |  97.43 |     78 |     39.00 |       38.00 |        0.49 |    0.00 |   0.00 |         1,823 |
| **NEGmir~ever_boring**  |    57 |    0.50 |    1.59 |   1.00 |  79.02 |     57 |     28.50 |       28.50 |        0.50 |    0.00 |   0.00 |           775 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/ALL-ever_10mostNEG-bigrams_AMscores.2024-08-07.md`


|                  |       `N` |      `f1` |   `adv_total` |
|:-----------------|----------:|----------:|--------------:|
| **NEGMIR_ever**  |   583,470 |   291,735 |         4,776 |
| **NEGATED_ever** | 6,347,362 | 3,173,681 |        10,849 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/ever_MarginalFreqs_2024-08-07.md`


### 1. _ever easy_


#### 1.1. "ever easy" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                      |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_07_102.1027_x1634098_27:19-20-21`**  | If a bereaved mom shares the details with you consider yourself extremely privileged because telling that story is never __``ever easy``__ because losing a child is n't natural .               |
| **`pcc_eng_11_015.6612_x0237322_111:06-10-11`** | But I also know that nothing worth doing is __``ever easy``__ .                                                                                                                                  |
| **`pcc_eng_01_041.5053_x0654711_1015:1-6-7`**   | Nothing in this world is __``ever easy``__ .                                                                                                                                                     |
| **`pcc_eng_15_014.9291_x0224882_49:25-27-28`**  | He 's able to be both the analytical adult who can examine and explain the world in easy terms , while also understanding that nothing is __``ever easy``__ .                                    |
| **`pcc_eng_01_066.0899_x1052767_02:07-09-10`**  | GREEN BAY , Wis . - Nothing 's __``ever easy``__ in the National Football League .                                                                                                               |
| **`pcc_eng_14_094.8833_x1517632_29:2-5-6`**     | " Nothing great is __``ever easy``__ , " Nick says .                                                                                                                                             |
| **`pcc_eng_24_070.8852_x1130442_06:27-29-30`**  | Logic and economics would dictate that the win should propel Marquez into a third bout with Manny Pacquiao , but this is boxing and in boxing nothing is __``ever easy``__ or makes much sense . |
| **`pcc_eng_00_015.1590_x0228629_25:02-09-10`**  | But nothing truly worth doing in life is __``ever easy``__ , " he added .                                                                                                                        |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_easy_ex.md`


#### 1.2. "ever easy" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                |
|:---------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_26_098.8606_x1582054_121:11-12`** | Although it is absurd to say that discovering blockbusters is __``ever easy``__ , it is certainly true that by then drugmakers had plucked much of the low-hanging fruit . |
| **`nyt_eng_19980807_0103_42:1-2`**           | __``ever easy``__ _ and that 's what makes `` Billy 's Hollywood Screen Kiss '' such a smart romantic comedy .                                                             |
| **`pcc_eng_17_053.3523_x0845793_015:10-11`** | When , exactly , is the career of writing __``ever easy``__ ?                                                                                                              |
| **`pcc_eng_06_085.1946_x1361633_41:4-5`**    | But it is __``ever easy``__ to find the genuine option for tons of them in the market .                                                                                    |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_easy_ex.md`


### 2. _ever good_


#### 2.1. "ever good" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                           |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_030.3901_x0475789_09:15-17-18`**  | We pick ourselves apart , never quite satisfied with who we are , because nothing 's __``ever good``__ enough when you 're aiming for something that does n't exist . |
| **`pcc_eng_13_094.5903_x1512516_13:09-10-11`**  | Are you content with your life or is nothing __``ever good``__ enough for you ?                                                                                       |
| **`pcc_eng_25_091.8917_x1470795_124:1-3-4`**    | Nobody is __``EVER good``__ on their first time , I know from experience .                                                                                            |
| **`pcc_eng_24_107.03902_x1721358_037:5-6-7`**   | But idol dances are hardly __``ever good``__ .                                                                                                                        |
| **`pcc_eng_01_093.4349_x1494649_31:1-5-6`**     | Nothing they do is __``ever good``__ enough , and she 's certainly never supportive of their goals .                                                                  |
| **`pcc_eng_09_017.2344_x0263048_21:11-16-17`**  | Abusive , critical , bullying husbands are never pleased ; nothing their wives do is __``ever good``__ enough .                                                       |
| **`pcc_eng_27_032.3782_x0506576_57:1-3-4`**     | Nothing is __``ever good``__ enough , and the demands may escalate .                                                                                                  |
| **`pcc_eng_06_106.9825_x1714475_265:08-09-10`** | Not the stupid little brother who was n't __``ever good``__ enough . "                                                                                                |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_good_ex.md`


#### 2.2. "ever good" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                   |
|:---------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_10_088.1467_x1408431_3:22-23`**   | Watch this scene from ' How to Get Away with Murder , ' Season 4 Episode 4 , ' Was She __``Ever Good``__ at Her Job ? '                                                                                       |
| **`nyt_eng_19940825_0261_29:8-9`**           | he also questioned `` whether it 's __``ever good``__ for the government , even in its role of law enforcer , to send out letters that are abject lies . ''                                                   |
| **`pcc_eng_10_036.8363_x0579747_087:4-5`**   | The only person __``ever good``__ enough was Jesus Christ , who came to Earth and is the Son of God .                                                                                                         |
| **`pcc_eng_17_026.7036_x0415923_38:5-6`**    | But is that answer __``ever good``__ enough ?                                                                                                                                                                 |
| **`pcc_eng_04_107.09835_x1728439_02:09-10`** | The first year that Valentine 's Day was __``ever good``__ for me .                                                                                                                                           |
| **`pcc_eng_03_082.0901_x1313141_3:14-15`**   | How To Get Away With Murder Season 4 Episode 4 ( Was She __``Ever Good``__ at Her Job ? )                                                                                                                     |
| **`pcc_eng_06_024.0486_x0373022_192:36-37`** | As long as I 've known him , he 's been obsessed with sports , and although he played pretty much everything -- basketball , soccer , and his beloved hockey -- he was only __``ever good``__ , never great . |
| **`pcc_eng_03_057.8840_x0921406_11:4-5`**    | She 's only __``ever good``__ in Game of Thrones ...                                                                                                                                                          |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_good_ex.md`


### 3. _ever closer_


#### 3.1. "ever closer" under _negative_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*ever closer*" found in **negative** hits.

#### 3.2. "ever closer" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                                                                                                                                                      |
|:--------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_026.0772_x0405151_5:33-34`**  | This album plots this course which we can see commencing in the world around us as science is worshiped , spirituality condemned , technology becomes a human right , and we creep __``ever closer``__ to creating sentience and sapience to serve us with no free will so that we might devolve back into a philosophically embryonic state and die off like the gods and goddesses before us . |
| **`pcc_eng_19_052.0621_x0824133_02:4-5`**   | As winter rolls __``ever closer``__ it is sometimes nice to forget about the cold days , the wet days , and remember the warm summer nights .                                                                                                                                                                                                                                                    |
| **`pcc_eng_14_080.9110_x1292004_1:09-10`**  | Time leap anime Re : Zero has inched __``ever closer``__ to the truth with its new broadcast , naturally causing more violence to erupt but at least allowing the oft put upon protagonist an opportunity to fight back .                                                                                                                                                                        |
| **`pcc_eng_25_048.9350_x0775926_05:33-34`** | Major central banks such as the U.S. Federal Reserve , the Bank of England and the Bank of Canada have already raised interest rates , while the European Central Bank is moving __``ever closer``__ to unwinding its own ultra-easy monetary policy .                                                                                                                                           |
| **`pcc_eng_10_105.9554_x1697233_40:60-61`** | Officer Samara , as always , you look exceedingly sexy in that space suit , and I appreciate your invitation for me to get out of this cryobed and join you in your bunk for some " good old - fashioned sleep , " but I see that for exactly what it is : an attempt to seduce me __``ever closer``__ to that terrifying alien world .                                                          |
| **`pcc_eng_06_101.4971_x1625712_27:17-18`** | Pep Guardiola saluted his champions - elect for an " outstanding season " after they edged __``ever closer``__ to the title .                                                                                                                                                                                                                                                                    |
| **`pcc_eng_28_017.4678_x0266335_02:21-22`** | On one hand , UK Inc. is having to deal with the uncertainty created as the Brexit countdown clock ticks __``ever closer``__ to zero .                                                                                                                                                                                                                                                           |
| **`pcc_eng_05_023.2745_x0360936_16:11-12`** | There is significant opposition in key European countries to an __``ever closer``__ EU .                                                                                                                                                                                                                                                                                                         |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_closer_ex.md`


### 4. _ever perfect_


#### 4.1. "ever perfect" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                              |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_003.2651_x0036630_35:12-14-15`**  | Also voting to grant the amendment was John Ericson , " Nothing is __``ever perfect``__ , we move forward doing the best job we can .                                                                                    |
| **`pcc_eng_12_035.4607_x0557443_4:15-17-18`**   | I have been looking for this type of wedges for a while noow but nothing is __``ever perfect``__ for me D ; I hope they have this in my local target !                                                                   |
| **`pcc_eng_29_008.1184_x0115022_104:26-28-29`** | In " Katie " she and Freddie are seen sharing affection and drugs at various gatherings ; despite Effy 's remark to Katie that ' nothing 's __``ever perfect``__ ' they are both obviously enjoying their relationship . |
| **`nyt_eng_20000107_0261_16:2-4-5`**            | `` Nothing 's __``ever perfect``__ _ it 's hard to build a business , whether you 're public or you 're private , '' Ms. Polish said .                                                                                   |
| **`pcc_eng_06_022.2337_x0343655_14:1-3-4`**     | Nothing is __``ever perfect``__ but I looked , and looked .                                                                                                                                                              |
| **`pcc_eng_13_085.9993_x1373957_076:1-3-4`**    | Nothing is __``ever perfect``__ , and I think yours is at that stage between " oh gosh why did I think it was a good idea to write this " and " this as good as I can possibly make it " .                               |
| **`pcc_eng_13_099.2824_x1588216_39:10-14-15`**  | We may never find the perfect lover , but nothing in life is __``ever perfect``__ .                                                                                                                                      |
| **`pcc_eng_10_059.6385_x0948134_14:2-4-5`**     | But nothing was __``ever perfect``__ enough to show anyone .                                                                                                                                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_perfect_ex.md`


#### 4.2. "ever perfect" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                             |
|:--------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_25_062.9469_x1003371_09:14-15`** | That final dive sealed perfect 10s across the board , matching the first __``ever perfect``__ 10 which he had clinched during the Red Bull Cliff Diving World Series event in Lebanon . |
| **`pcc_eng_06_084.7299_x1354048_165:5-6`**  | As his wife can __``ever perfect``__ be ; Nor he who owns the least of things , nor he Whom anger chains , can ever pass thro ' Maya 's gates .                                         |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_perfect_ex.md`


### 5. _ever simple_


#### 5.1. "ever simple" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                         |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_05_035.2335_x0554427_16:4-6-7`**    | In lexicography , nothing is __``ever simple``__ .                                                                                                                                                  |
| **`pcc_eng_11_083.1163_x1329077_21:23-25-26`** | In other words , even if we genuinely love someone , the realities of life have a way of reminding us that nothing is __``ever simple``__ ...                                                       |
| **`pcc_eng_11_015.4603_x0234017_08:08-10-11`** | With RIM , it often feels , nothing is __``ever simple``__ .                                                                                                                                        |
| **`pcc_eng_12_066.7344_x1062349_006:2-4-5`**   | But nothing 's __``ever simple``__ when magic is -- literally -- in the air .                                                                                                                       |
| **`pcc_eng_28_072.2803_x1153061_24:4-6-7`**    | Of course , nothing is __``ever simple``__ , and lest we forget , last week Lauter and Mike were starting their own investigation about What Really Happened with Tom Walker and Elizabeth Gaines . |
| **`nyt_eng_20000530_0116_5:2-4-5`**            | but nothing is __``ever simple``__ when it comes to exploring an embryonic artistic form , or , for that matter , stretching the borders of artistic expression in Harlem .                         |
| **`pcc_eng_03_080.6319_x1289486_03:1-3-4`**    | Nothing is __``ever simple``__ .                                                                                                                                                                    |
| **`pcc_eng_16_055.7158_x0885540_05:11-13-14`** | But in this weird , wonderful and anarchic universe , nothing is __``ever simple``__ - not even what colour they should be .                                                                        |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_simple_ex.md`


#### 5.2. "ever simple" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                           |
|:--------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_15_082.1003_x1310689_41:16-17`** | By maintaining such focus and on such a simple task ( as if breathing is __``ever simple``__ ! ) , it creates a mental space that is fully occupied in the present moment only and fully conscious of that one task . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_simple_ex.md`


### 6. _ever enough_


#### 6.1. "ever enough" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                      |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_18_046.7640_x0740682_12:1-5-6`**    | Nothing we do is __``ever enough``__ , and no one dares to disagree with him about anything .                                                                                                                                    |
| **`pcc_eng_22_082.0765_x1310506_33:1-3-4`**    | Nothing is __``ever enough``__ for them .                                                                                                                                                                                        |
| **`pcc_eng_16_085.6227_x1369660_57:10-12-13`** | You could even be one of them that think nothing is __``ever enough``__ ?                                                                                                                                                        |
| **`pcc_eng_11_019.5604_x0300126_06:3-5-6`**    | It is never , __``ever enough``__ for the homosexual lobby .                                                                                                                                                                     |
| **`pcc_eng_11_065.2271_x1039404_05:32-34-35`** | i was worried about you but you never cared about me none you took my money and i knew that you , you could kill someone i gave you everything but nothing was __``ever enough``__ you were always jealous over such crazy stuff |
| **`pcc_eng_04_076.8617_x1225337_259:5-7-8`**   | The ones for whom nothing is __``ever enough``__ , no amount of taxation is enough for you crooks !                                                                                                                              |
| **`pcc_eng_27_008.3888_x0118995_08:29-31-32`** | As the interviewer points out , Mc Donald 's has bent over backwards to accommodate the food faddists in recent years , but , for people like Banzhaf nothing is __``ever enough``__ .                                           |
| **`pcc_eng_28_077.9747_x1245235_072:5-7-8`**   | But hope alone is never , __``ever enough``__ .                                                                                                                                                                                  |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_enough_ex.md`


#### 6.2. "ever enough" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                             |
|:--------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_064.9585_x1034643_195:3-4`**  | Is there __``ever enough``__ time to see it all .                                                                                                                       |
| **`pcc_eng_17_037.1871_x0584795_07:27-28`** | Just looking at pictures of this stupendous rope bridge is enough to make you feel distinctly uncomfortable ; it might be moored down but is that __``ever enough``__ ? |
| **`pcc_eng_00_080.9536_x1292556_14:4-5`**   | But is that __``ever enough``__ recompense ?                                                                                                                            |
| **`pcc_eng_27_076.2549_x1216767_67:4-5`**   | How much is __``ever enough``__ for decision makers .                                                                                                                   |
| **`pcc_eng_19_050.7883_x0803660_32:4-5`**   | But is it __``ever enough``__ ?                                                                                                                                         |
| **`pcc_eng_29_059.7325_x0948904_86:3-4`**   | Is one __``ever enough``__ ?                                                                                                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_enough_ex.md`


### 7. _ever certain_


#### 7.1. "ever certain" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                            |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_26_036.8253_x0579110_12:09-11-12`** | In a market characterized by high risk , nothing is __``ever certain``__ , so only time will tell if the performance of IRBO and other robotics and AI ETFs boom with the industry or if they will continue their current downward trend .                                                                             |
| **`pcc_eng_28_047.0273_x0744787_44:1-3-4`**    | Nothing is __``ever certain``__ in Sitka .                                                                                                                                                                                                                                                                             |
| **`pcc_eng_17_003.8832_x0046510_06:6-8-9`**    | In the cryptocurrency world , nothing is __``ever certain``__ or guaranteed .                                                                                                                                                                                                                                          |
| **`pcc_eng_28_066.9390_x1066899_15:28-30-31`** | The story highlights the fact that people often feel a sense of entitlement and act in their own best interest , but concludes with the message that nothing is __``ever certain``__ and circumstances change .                                                                                                        |
| **`pcc_eng_19_044.2947_x0699007_13:32-33-34`** | The story -- which was also reported by The Intercept , an online magazine on nationals security matters -- paints a classic spy versus spy story where the US agents are n't __``ever certain``__ about who they are dealing with and whether or not they are being baited and played by their Russian counterparts . |
| **`pcc_eng_13_033.5203_x0525793_16:6-8-9`**    | Work is very repetitive and nothing is __``ever certain``__ .                                                                                                                                                                                                                                                          |
| **`pcc_eng_20_035.0161_x0549922_20:27-29-30`** | The story encompasses several moments within this turbulent journey and how each of the characters lives are suddenly turned upside down in a dangerous country where nothing is __``ever certain``__ .                                                                                                                |
| **`pcc_eng_13_092.8113_x1484200_23:2-4-5`**    | While nothing is __``ever certain``__ , we are hopeful that an agreement can be reached for two important reasons :                                                                                                                                                                                                    |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_certain_ex.md`


#### 7.2. "ever certain" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                                                        |
|:--------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_17_018.6885_x0286260_20:15-16`** | The news will be about the drop in HIV / AIDS whenever and if __``ever certain``__ people desist from certain behaviors .                                                                                                                                                                          |
| **`pcc_eng_24_102.9099_x1648709_03:25-26`** | After struggling to find my path in this life , endlessly searching for what would make me happy , the only thing I was __``ever certain``__ of was that I wanted to have children .                                                                                                               |
| **`pcc_eng_14_065.0171_x1034618_15:44-45`** | " Then he went back to Arkham , the terrible witch - haunted old town of his forefathers in New England , and had experiences in the dark , amidst the hoary willows and tottering gambrel roofs , which made him seal for __``ever certain``__ pages in the diary of a wild - minded ancestor . " |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_certain_ex.md`


### 8. _ever greater_


#### 8.1. "ever greater" under _negative_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*ever greater*" found in **negative** hits.

#### 8.2. "ever greater" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                                                |
|:---------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_18_016.1938_x0246164_016:25-26`** | The size of the global economy , he forecast , would increase sevenfold in the following century , and this , in concert with __``ever greater``__ " technical improvements , " would usher in the fifteen - hour week .                   |
| **`pcc_eng_28_091.3130_x1461019_10:34-35`**  | In sum , the report said : " Population aging will lead to profound changes in economic growth prospects for countries around the world ... as governments work to build budgets to face __``ever greater``__ age-related spending needs . |
| **`pcc_eng_22_005.5253_x0073108_35:20-21`**  | And as technology advances and connectivity continues at pace across the world , we face the appalling prospect of __``ever greater``__ numbers of vulnerable children becoming victims .                                                  |
| **`pcc_eng_23_067.5465_x1074968_032:6-7`**   | The EU is about " __``ever greater``__ union " ( the " Project " has always been that ) , your idea that this is just a pose is wrong .                                                                                                    |
| **`pcc_eng_05_083.4183_x1333827_54:8-9`**    | When the possibility of being dismissed becomes __``ever greater``__ in times of crisis , workers are increasingly pushed into accepting ever-worse working conditions .                                                                   |
| **`pcc_eng_03_051.3234_x0815309_033:20-21`** | At this same period Hitler 's U-boats , refuelling now on France 's Atlantic coast , are sinking an __``ever greater``__ tonnage of Allied merchant shipping in the Atlantic .                                                             |
| **`pcc_eng_19_085.4277_x1364188_11:21-22`**  | With a contribution of just 22 Euros a year , you make it possible for these booklets to reach an __``ever greater``__ number of people .                                                                                                  |
| **`nyt_eng_19980628_0222_15:10-11`**         | the parallels between Cigar and Skip Away are becoming __``ever greater``__ .                                                                                                                                                              |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_greater_ex.md`


### 9. _ever larger_


#### 9.1. "ever larger" under _negative_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*ever larger*" found in **negative** hits.

#### 9.2. "ever larger" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                                                                                    |
|:---------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_28_093.6223_x1498480_284:09-10`** | With the U.S. trade deficit with China growing __``ever larger``__ , and the Chinese military growing ever stronger , now is the time to ask whether selling a Chinese " company " such important high technology is a wise idea .                                             |
| **`apw_eng_20080130_0882_15:10-11`**         | amid that climate , mutual fund companies have won __``ever larger``__ numbers of investors with ads citing their high returns of the recent past .                                                                                                                            |
| **`pcc_eng_23_018.4349_x0281293_20:19-20`**  | As nations curb their direct involvement , that means that the UN will be forced to shoulder an __``ever larger``__ burden on security .                                                                                                                                       |
| **`pcc_eng_29_023.9045_x0369687_157:22-23`** | Ironically , while the Left tends to rail against special interests and their role in election campaigns , its support for __``ever larger``__ government inevitably enhances the role of special interests and their role in election campaigns .                             |
| **`pcc_eng_04_045.0764_x0712244_6:8-9`**     | " The bubble is being inflated with __``ever larger``__ amounts of air and at some point it 's going to explode . "                                                                                                                                                            |
| **`pcc_eng_18_077.8289_x1244194_41:10-11`**  | To this day , America 's wealth gap grows __``ever larger``__ .                                                                                                                                                                                                                |
| **`apw_eng_20090603_1074_3:34-35`**          | Sarkozy 's office said the president asked Iran to respect U.N. resolutions and negotiate with the six countries representing the international community , including the United States and France -- or risk `` __``ever larger``__ international isolation in all areas . '' |
| **`nyt_eng_19971231_0243_23:15-16`**         | the changes have allowed banks to offer an expanding array of products and created __``ever larger``__ financial institutions .                                                                                                                                                |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_larger_ex.md`


### 10. _ever wrong_


#### 10.1. "ever wrong" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                      |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_11_098.6038_x1579993_17:1-5-6`**    | Nothing you want is __``ever wrong``__ .                                                                                                                                                                                                                                                                                                                                         |
| **`pcc_eng_15_011.0776_x0162694_035:3-4-5`**   | He was hardly __``ever wrong``__ .                                                                                                                                                                                                                                                                                                                                               |
| **`pcc_eng_15_034.6788_x0544586_12:40-41-42`** | The main advantage of playing games on a computer , whether they be native beasts or some fanciful Eurogame conversion , is the addition of an unsparing , wholly impartial adjudicator who is really good at math and does n't __``ever wrong``__ .                                                                                                                             |
| **`pcc_eng_09_094.3621_x1510615_34:7-8-9`**    | " It feels like there was nothing __``ever wrong``__ with it .                                                                                                                                                                                                                                                                                                                   |
| **`pcc_eng_25_004.5550_x0058042_33:12-14-15`** | But no , Superman has made up his mind and is never , __``ever wrong``__ .                                                                                                                                                                                                                                                                                                       |
| **`pcc_eng_09_034.4973_x0542391_20:70-71-72`** | Even before the church officially defined Mary as God 's mother in the fifth century , the faithful had already acknowledged her divine maternity and called for its recognition , the pope said , noting the case as an example of the " ' sensus fidei ' ( sense of the faith ) of holy people , the faithful of God , who , in their unity , are never __``ever wrong``__ . " |
| **`pcc_eng_29_096.7106_x1546207_01:01-09-10`** | Not that anyone 's opinion on music is __``ever wrong``__ , but man I find music bigots to be annoying .                                                                                                                                                                                                                                                                         |
| **`pcc_eng_12_069.8019_x1111532_27:13-15-16`** | But with proper rest , rehab and painkillers , Lawson looked like nothing was __``ever wrong``__ .                                                                                                                                                                                                                                                                               |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_wrong_ex.md`


#### 10.2. "ever wrong" under _positive_ polarity


|                                           | `token_str`                                                                                                                  |
|:------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_053.2278_x0843930_20:5-6`** | If the bettor is __``ever wrong``__ on 9 consecutive coin flips , it will always break him .                                 |
| **`pcc_eng_14_079.0967_x1262726_73:5-6`** | Boy , was I __``ever wrong``__ .                                                                                             |
| **`pcc_eng_15_055.9439_x0887989_14:3-4`** | Was I __``ever wrong``__ .                                                                                                   |
| **`pcc_eng_08_093.8709_x1503504_35:5-6`** | Damn , was he __``ever wrong``__ .                                                                                           |
| **`pcc_eng_19_075.9128_x1210148_04:3-4`** | Was he __``ever wrong``__ .                                                                                                  |
| **`pcc_eng_00_051.3396_x0813519_61:3-4`** | What was __``ever wrong``__ with I 'll know it when I see it and if it 's sound action it 'll produce something worthwhile . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_wrong_ex.md`


### 11. _ever higher_


#### 11.1. "ever higher" under _negative_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*ever higher*" found in **negative** hits.

#### 11.2. "ever higher" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                                                                                                                        |
|:---------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_06_094.5926_x1513726_102:21-22`** | This is one world , Godis world throughout , whose law-abiding regularities , whose amazing artistries , whose evolution of __``ever higher``__ structures , whose creation of personality , whose endless possibilities of spiritual growth and social progress indicate that it is a spiritual system .          |
| **`pcc_eng_18_078.6918_x1258038_15:5-6`**    | As stirrups were pulled __``ever higher``__ and pant legs dipped lower , some players began favoring a hosiery style that was little more than a tapered vertical stripe of stirrup against a white sani background ( or , for the A's , a yellow background Charlie Finley gave his team yellow sanis in 1966 ) . |
| **`pcc_eng_03_047.0270_x0745696_329:26-27`** | I asked whether Scientology was a caution to Jason 's ambition for the religious opportunity for games - a pyramidial belief -system , founded on __``ever higher``__ levels of adeptness , cynically invented by a bad SF author in California !                                                                  |
| **`pcc_eng_04_054.9555_x0871449_171:8-9`**   | Is it just a pause prior to __``ever higher``__ levels of margin debt outstanding being achieved ?                                                                                                                                                                                                                 |
| **`pcc_eng_27_088.6146_x1417065_026:30-31`** | Or that Wisconsinites are beginning to appreciate how the privileges ( that Bravender calls " rights " above ) the state once granted to public employee unions account for __``ever higher``__ government expenditures ?                                                                                          |
| **`nyt_eng_20020118_0230_7:5-6`**            | after having taken on __``ever higher``__ levels of credit-card and mortgage debt to finance arguably the highest material standard of living the world has ever known , Americans will eventually have to cut back , many economists and writers have argued .                                                    |
| **`pcc_eng_00_079.9320_x1276048_20:33-34`**  | When enough of us Enter this Evolutionary Flow , always Giving Energy to the Higher -Self of Everyone we Meet , we will Build a New Culture where our Bodies Evolve to __``ever Higher``__ Levels of Energy and Perception                                                                                         |
| **`pcc_eng_23_069.8458_x1112340_22:32-33`**  | Aside from the notable case of Free Trade England , most nations in the latter half of the 19th century sought safety from the gales of modern global market competition behind __``ever higher``__ tariff walls , buttressed with government subsidies to domestic industries and imperial expansion .            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_higher_ex.md`


### 12. _ever able_


#### 12.1. "ever able" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_014.5713_x0219749_10:4-5-6`**    | I was just never __``ever able``__ to get into this one .                                                                                                                                                  |
| **`pcc_eng_29_031.6439_x0494520_29:23-24-25`** | For the first month of summer I was working at least two jobs every day -- I became burned out and was n't __``ever able``__ to enjoy time with my friends .                                               |
| **`pcc_eng_21_076.5573_x1221028_11:16-17-18`** | We were former classmate and partner and she 's one precious friend that I will never __``ever able``__ to forget .                                                                                        |
| **`pcc_eng_22_051.2848_x0812424_16:11-12-13`** | He said : " A small country like Switzerland will hardly __``ever able``__ to be the favourite of a World Cup .                                                                                            |
| **`pcc_eng_09_032.0421_x0502471_24:23-29-30`** | However , such was the perceived strategic imperative of shoring up one 's frontiers by controlling part or all of Afghanistan that none of its neighbours have been __``ever able``__ to leave it alone . |
| **`pcc_eng_29_085.3558_x1362536_15:23-25-26`** | Though he wanted those things for himself , he certainly never expected them and every gesture Lexie made for him , he hardly was __``ever able``__ to believe it and accept it graciously .               |
| **`pcc_eng_06_078.6216_x1255179_05:13-14-15`** | Backed by our 100 % Money - Back Guarantee if we 're not __``ever able``__ to produce what you bought from us .                                                                                            |
| **`pcc_eng_15_096.0788_x1536772_25:17-18-19`** | I 've always had the determination and willpower to exercise and eat right , but was never __``EVER able``__ to see such unbelievable results until I found Venus .                                        |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_able_ex.md`


#### 12.2. "ever able" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|:--------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_077.5356_x1236977_6:37-38`**  | Rather , it is my contention that by focusing on cultural differences , the ' multiculturalism ' movement did more to foster balkanization , racism , misunderstanding and mistrust between races than white supremacist groups were __``ever able``__ to accomplish .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| **`pcc_eng_10_048.3694_x0766197_08:27-28`** | I wanted myself to possess the qualities of a man , but how can a woman , designed to be subordinate to a man , be __``ever able``__ to contain these traits ?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| **`pcc_eng_21_040.0541_x0631452_20:14-15`** | Ukoku was obsessed with Koumyou , who was the only one who was __``ever able``__ to get the better of him but refused to fight him straight out .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| **`pcc_eng_11_098.4947_x1578230_095:4-5`**  | If we are __``ever able``__ to inhibit telomerase production in the human body , however , a new problem may develop .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| **`pcc_eng_00_059.9949_x0953788_22:13-14`** | " It seemed like a lifetime or an eternity before I was __``ever able``__ to hold her because I did n't get to hold her when she was first born , so that was the first time I did n't get to hold her and that feeling was ... you ca n't even put into words , you really can't , " she said .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| **`pcc_eng_04_104.5566_x1672872_019:5-6`**  | And that he was __``ever able``__ to erect it again , under another form than that of Gentilism , as he hath done in the antichristian apostasy of the church visible , it was from a neglect and contempt of these gifts , with their due use and improvement , When men began to neglect the attaining of these spiritual gifts , and the exercise of them , in praying , in preaching , in interpretation of the Scripture , in all the administrations and whole worship of the church , betaking themselves wholly to their own abilities and inventions , accommodated unto their ease and secular interest , it was an easy thing for Satan to erect again his kingdom , though not in the old manner , because of the light of the Scripture , which had made an impression on the minds of men which he could not obliterate . |
| **`pcc_eng_18_035.7180_x0561996_28:22-23`** | Democrats , eager to regain majorities in Congress , would be wise to heed the only House speaker in U.S. history __``ever able``__ to lead the chamber to pass -- three times -- national health care , the former Nancy D'Alesandro of Baltimore .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| **`pcc_eng_28_019.5616_x0300215_35:5-6`**   | The most I was __``ever able``__ to achieve was about 3 - and - a- half - portraits per day .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_able_ex.md`


### 13. _ever free_


#### 13.1. "ever free" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                     |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_27_002.7832_x0028690_03:04-13-14`**  | It seems that nothing involving Michael Jackson or his fractious family is __``ever free``__ of controversy , even two years after his death .                                  |
| **`pcc_eng_28_012.7416_x0190111_10:20-24-25`**  | You could add that people and action need willingness , however these three things are the essentials , and none of them are __``ever free``__ .                                |
| **`pcc_eng_26_034.4410_x0540598_091:12-13-14`** | He is terrified of impressive affects in others , and is hardly __``ever free``__ from the dread of falling under hostile influence .                                           |
| **`pcc_eng_03_081.3853_x1301822_40:3-4-5`**     | She 's hardly __``ever free``__ to sit because she 's with her boyfriend 's family and away on all their church group 's outreach things together .                             |
| **`pcc_eng_03_030.4778_x0477292_05:4-6-7`**     | weak mind wo n't be __``ever free``__ from its clutch ; yes so , come out ; breathe the fresh air of the beach an ......                                                        |
| **`pcc_eng_29_099.5497_x1592278_14:01-09-10`**  | Nothing that is worth anything to anyone is __``ever free``__ or easy , and it cannot be a He or a power much superior to him in his mind sophisticated , philosophy appeared . |
| **`pcc_eng_26_018.5686_x0283980_20:10-12-13`**  | You are paying for it , but I suppose nothing is __``ever free``__ . "                                                                                                          |
| **`pcc_eng_29_049.3473_x0780686_5:09-11-12`**   | how even when you 're sneaking in , nothing 's __``ever free``__ .                                                                                                              |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_free_ex.md`


#### 13.2. "ever free" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                      |
|:--------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_21_031.6475_x0495532_096:4-5`**  | Thus we are __``ever free``__ of the sort of feelings that would require lesser men , of weaker faith , to go and hear the Holy Absolution from the mouth of a mere man .                                                        |
| **`pcc_eng_10_016.1307_x0244294_07:22-23`** | Despite the Technomancers being better armed they were outgunned and soon the rebellion fell with a small handful escaping into the __``ever free``__ and never being heard from again .                                         |
| **`pcc_eng_14_063.8300_x1015421_13:10-11`** | Alex Honnold attempts to become the first person to __``ever free``__ solo climb El Capitan .                                                                                                                                    |
| **`pcc_eng_24_034.7262_x0545551_03:39-40`** | On October 28 , they will be able to head to the polls again , this time to vote in a municipal election which its organizer , the United Nations , is billing as Kosovo 's first - __``ever free``__ and fair democratic test . |
| **`pcc_eng_28_090.5265_x1448375_19:4-5`**   | They say nothings __``ever free``__                                                                                                                                                                                              |
| **`pcc_eng_10_051.6414_x0819081_1102:4-5`** | Is a woman __``ever free``__ ? "                                                                                                                                                                                                 |
| **`pcc_eng_01_081.2051_x1296812_62:3-4`**   | Always and __``ever free``__ longer fiction ( books )                                                                                                                                                                            |
| **`pcc_eng_04_004.0082_x0048806_365:5-6`**  | You are more than __``ever free``__ agents .                                                                                                                                                                                     |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_free_ex.md`


### 14. _ever sure_


#### 14.1. "ever sure" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                                                                              |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_087.9226_x1405034_58:4-6-7`**     | Over time , nobody was __``ever sure``__ that the amount of electricity held in the banque was as much as the banque claimed to have stored - or owed to it .                                                                                                                                                                                                                                                            |
| **`pcc_eng_18_030.8542_x0483214_049:30-31-32`** | I was so steeped in scarcity that I never EVER paid full price for anything , so I always ended up with 5 shirts on clearance that I was n't __``ever sure``__ I liked but thought I needed because my other shirts were getting too old or too small .                                                                                                                                                                  |
| **`pcc_eng_19_079.6883_x1271238_31:08-09-10`**  | This is a blog post I was n't __``ever sure``__ I 'd write .                                                                                                                                                                                                                                                                                                                                                             |
| **`pcc_eng_18_091.6320_x1467737_14:3-4-5`**     | He was n't __``ever sure``__ who was doing the talking .                                                                                                                                                                                                                                                                                                                                                                 |
| **`pcc_eng_22_058.1261_x0923450_07:4-5-6`**     | But I 'm not __``ever sure``__ if it is helping me become a better thinker or just a better chaser .                                                                                                                                                                                                                                                                                                                     |
| **`pcc_eng_29_084.0472_x1341322_29:14-15-16`**  | Normally I would refrain from telling her who was coming because I was not __``ever sure``__ who would really show up for her .                                                                                                                                                                                                                                                                                          |
| **`pcc_eng_00_038.2542_x0601790_09:37-38-39`**  | " But in the books , if you read the books , there 's this sort of implicit violence that 's going through them all the time that makes you turn the page because you 're not __``ever sure``__ what 's going to happen next .                                                                                                                                                                                           |
| **`pcc_eng_19_042.9424_x0677131_02:11-12-13`**  | With all the fasting that is happening concurrently I am not __``ever sure``__ when I am supposed to be fasting , I am just waiting for the day to arrive without any planning on my behalf , well maybe a little planning as I have n't bought meat the entire week , just using left overs to make food so that when the 10 day fast does start , I do n't have to throw any meat away that has gotten frost bite on . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_sure_ex.md`


#### 14.2. "ever sure" under _positive_ polarity


|                                            | `token_str`                                  |
|:-------------------------------------------|:---------------------------------------------|
| **`pcc_eng_11_057.3628_x0911893_007:4-5`** | When am I __``ever sure``__ about anything ? |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_sure_ex.md`


### 15. _ever deeper_


#### 15.1. "ever deeper" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                   |
|:------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_006.2279_x0084646_074:11-13-14`** | pull my trigger bang every time you miss I do not deep __``ever deeper``__ inside my finger always strikes does n't lie I know about hiding living in the shadows breaking with the glass every moment getting way too close spending my time getting lost going deep in the darkness keep my secrets never forget No regrets |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_deeper_ex.md`


#### 15.2. "ever deeper" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                                                                                                                                                                          |
|:---------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_005.8730_x0078505_45:17-18`**  | As an undergraduate at Columbia University , he found himself edging away from campus and " __``ever deeper``__ into the darkness of movie houses , not so much in search of a vocation as in flight from the laborious realities of careerism . "                                                                                                                   |
| **`pcc_eng_28_098.4327_x1576135_095:6-7`**   | We do this on an __``ever deeper``__ level until at last the light of Bodhi dawns , and we are transformed in the deepest level of our being .                                                                                                                                                                                                                       |
| **`pcc_eng_27_107.03028_x1719530_06:43-44`** | A Story From the Philadelphia Public Record ( BY PETE MATTHEWS , President , AFSCME District Council 33 / Mayor Michael Nutter 's stubborn and vindictive refusal to settle fair contracts with AFSCME District Councils 33 and 47 is driving city workers __``ever deeper``__ into poverty and negatively impacting the City 's ability to deliver vital services . |
| **`pcc_eng_28_017.7873_x0271459_18:6-7`**    | Yes , Ubuntu is delving __``ever deeper``__ into the cloud market , and as per usual .                                                                                                                                                                                                                                                                               |
| **`pcc_eng_18_065.1748_x1039050_12:16-17`**  | He has also since moved to Glastonbury , and may perhaps be finding himself drawn __``ever deeper``__ into its spell .                                                                                                                                                                                                                                               |
| **`pcc_eng_00_059.2815_x0942116_149:6-7`**   | Instead , his raids went __``ever deeper``__ into enemy country .                                                                                                                                                                                                                                                                                                    |
| **`pcc_eng_13_045.0848_x0712773_21:5-6`**    | As her husband wades __``ever deeper``__ into the bloody mire , Samantha Bond 's once sensual Lady Macbeth can no longer bear to be touched by him , and her extreme mental distress in the sleepwalking scene is pitiable to behold .                                                                                                                               |
| **`pcc_eng_21_020.0775_x0308084_13:26-27`**  | Initially reluctant , she soon starts going along with it , and over the course of one long night , the two underdogs get into __``ever deeper``__ trouble as they cross paths with an unhinged gambling rival ( Andy On ) .                                                                                                                                         |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_deeper_ex.md`


### 16. _ever boring_


#### 16.1. "ever boring" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                     |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_25_013.0080_x0194233_49:2-4-5`**    | Was n't it __``ever boring``__ to play the same five concertos for three years ?                                                                                                                |
| **`pcc_eng_25_004.8609_x0062975_07:1-3-4`**    | Nothing is __``EVER boring``__ with me around ...                                                                                                                                               |
| **`pcc_eng_22_085.9567_x1373236_06:3-5-6`**    | he is never , __``ever boring``__ .                                                                                                                                                             |
| **`pcc_eng_21_076.9634_x1227568_112:3-5-6`**   | It 's never , __``ever boring``__ .                                                                                                                                                             |
| **`pcc_eng_21_090.4400_x1445552_68:26-28-29`** | What I loved most about having her on board was the fact that all of her suggestions had a really original and special touch - nothing was __``ever boring``__ or seen a million times before . |
| **`pcc_eng_29_049.7412_x0787069_15:3-5-6`**    | It 's never , __``ever boring``__ and you never know what to expect .                                                                                                                           |
| **`pcc_eng_val_2.03386_x21696_57:1-5-6`**      | None of it was __``ever boring``__ , and all of it gave us a hunger for more .                                                                                                                  |
| **`pcc_eng_27_064.0342_x1018997_06:6-8-9`**    | Above all , it was never , __``ever boring``__ .                                                                                                                                                |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_boring_ex.md`


#### 16.2. "ever boring" under _positive_ polarity


|                                             | `token_str`                                                             |
|:--------------------------------------------|:------------------------------------------------------------------------|
| **`pcc_eng_12_009.1996_x0132583_01:10-11`** | Who ever said that Bruins vs Canadiens games were __``ever boring``__ ? |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_boring_ex.md`


### Saving *`ever`* Samples

 > in `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/`

1. *ever easy*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_easy_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_easy_99ex~4.csv"`

2. *ever good*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_good_99ex~84.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_good_99ex~12.csv"`

3. *ever closer*

    Full polarity samples saved as...
    1. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_closer_99ex~99.csv"`

4. *ever perfect*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_perfect_99ex~55.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_perfect_99ex~2.csv"`

5. *ever simple*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_simple_99ex~61.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_simple_99ex~1.csv"`

6. *ever enough*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_enough_99ex~46.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_enough_99ex~6.csv"`

7. *ever certain*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_certain_99ex~45.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_certain_99ex~3.csv"`

8. *ever greater*

    Full polarity samples saved as...
    1. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_greater_99ex~99.csv"`

9. *ever larger*

    Full polarity samples saved as...
    1. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_larger_99ex~99.csv"`

10. *ever wrong*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_wrong_99ex~29.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_wrong_99ex~6.csv"`

11. *ever higher*

    Full polarity samples saved as...
    1. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_higher_99ex~99.csv"`

12. *ever able*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_able_99ex~63.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_able_99ex~41.csv"`

13. *ever free*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_free_99ex~22.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_free_99ex~12.csv"`

14. *ever sure*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_sure_99ex~19.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_sure_99ex~1.csv"`

15. *ever deeper*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_deeper_99ex~1.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_deeper_99ex~77.csv"`

16. *ever boring*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/neg-ever/neg-ever_boring_99ex~20.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/ever/pos-ever/pos-ever_boring_99ex~1.csv"`


## 9. *longer*

|                             |   `f` |   `dP1` |   `LRC` |   `P1` |   `G2` |   `f2` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `adj_total` |
|:----------------------------|------:|--------:|--------:|-------:|-------:|-------:|----------:|------------:|------------:|--------:|-------:|--------------:|
| **COM~longer_lasting**      |   151 |    0.47 |    2.32 |   0.97 | 177.73 |    155 |     77.50 |       73.50 |        0.49 |    0.00 |   0.00 |         1,450 |
| **NEGany~longer_available** |    65 |    0.50 |    1.68 |   1.00 |  90.11 |     65 |     32.50 |       32.50 |        0.50 |    0.00 |   0.00 |        81,972 |
| **NEGmir~longer_viable**    |    57 |    0.50 |    1.59 |   1.00 |  79.02 |     57 |     28.50 |       28.50 |        0.50 |    0.00 |   0.00 |           248 |
| **NEGany~longer_viable**    |    61 |    0.50 |    1.57 |   1.00 |  84.56 |     61 |     30.50 |       30.50 |        0.50 |    0.00 |   0.00 |         4,288 |
| **NEGany~longer_pink**      |    51 |    0.50 |    1.27 |   1.00 |  70.70 |     51 |     25.50 |       25.50 |        0.50 |    0.00 |   0.00 |           610 |
| **NEGmir~longer_pink**      |    42 |    0.50 |    1.07 |   1.00 |  58.23 |     42 |     21.00 |       21.00 |        0.50 |    0.00 |   0.00 |            94 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/ALL-longer_10mostNEG-bigrams_AMscores.2024-08-07.md`


|                       |       `N` |      `f1` |   `adv_total` |
|:----------------------|----------:|----------:|--------------:|
| **COMPLEMENT_longer** | 6,347,362 | 3,173,681 |         1,803 |
| **NEGMIR_longer**     |   583,470 |   291,735 |           841 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/longer_MarginalFreqs_2024-08-07.md`


### 1. _longer lasting_


#### 1.1. "longer lasting" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                         |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_081.3811_x1299201_38:18-19-20`** | You are suffering premature ejaculation problem , even your penis erections are big and hard , but without __``longer lasting``__ , big and hard features are meaningless .                                         |
| **`pcc_eng_10_078.9808_x1260411_6:27-28-29`**  | Furthermore , the added duration and power of Optimum batteries only applies to " some devices , " meaning that Optimum batteries are neither more powerful nor __``longer lasting``__ when used in other devices . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/neg-longer/neg-longer_lasting_ex.md`


#### 1.2. "longer lasting" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                                                                                                                                                       |
|:---------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_28_080.8951_x1292326_036:20-21`** | This product also features a rain shield to help protect your fan from the elements , making the product __``longer lasting``__ .                                                                                                                                                                                                                 |
| **`pcc_eng_11_106.1990_x1702793_13:10-11`**  | Ongoing development is aimed at making the additive even __``longer lasting``__ .                                                                                                                                                                                                                                                                 |
| **`pcc_eng_27_028.8865_x0450321_2:6-7`**     | The surface is durable and __``longer lasting``__ than acrylic and can be easily maintained .                                                                                                                                                                                                                                                     |
| **`pcc_eng_07_077.1767_x1230967_04:20-21`**  | We use the highest quality ink to print the font on the laces , which will result in a __``longer lasting``__ print and the color will not run off on your precious sneakers .                                                                                                                                                                    |
| **`pcc_eng_05_009.1072_x0131599_070:31-32`** | This would give us batteries that charge in around 10 minutes , allow us to drive over 600 km at half the cost , and that are lighter weight and __``longer lasting``__ .                                                                                                                                                                         |
| **`pcc_eng_05_025.8617_x0402800_05:19-20`**  | With the new Galaxy Tab S , reputed to be the thinnest so far and equipped with a __``longer lasting``__ battery and Hancom Office Application , work that ordinarily requires a PC or Lap Top and done in physical offices , like creating , editing and viewing word , power point and excel documents is easily done in a mobile environment . |
| **`pcc_eng_15_075.1836_x1198602_14:17-18`**  | Negative symptoms include feeling withdrawn , unmotivated and / or depressed ; these symptoms are usually __``longer lasting``__ .                                                                                                                                                                                                                |
| **`pcc_eng_21_106.8305_x1710315_41:11-12`**  | Tuina has proved to be gentle and of an excellent __``longer lasting``__ benefit in these cases .                                                                                                                                                                                                                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/pos-longer/pos-longer_lasting_ex.md`


### 2. _longer available_


#### 2.1. "longer available" under _negative_ polarity


|                                                | `token_str`                                                                                                     |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_018.3214_x0279754_43:09-10-11`** | Update 1 : The Download Assistant Extension is not __``longer available``__ on the Chrome Store .               |
| **`pcc_eng_01_048.8381_x0773080_01:17-18-19`** | It is often said that it is human nature to only appreciate something when it is not __``longer available``__ . |
| **`pcc_eng_28_070.2237_x1119941_5:13-14-15`**  | Then we asked them what they would choose if that food was not __``longer available``__ .                       |
| **`pcc_eng_05_007.2585_x0101654_58:11-12-13`** | Flat scans of handmade fibre based photographic prints on papers not __``longer available``__ on the market .   |
| **`pcc_eng_24_083.6680_x1337178_1:4-5-6`**     | This pattern is not __``longer available``__ !                                                                  |
| **`pcc_eng_25_094.1694_x1507793_28:17-18-19`** | This is a great system for the 24 and 48 cell blocks which , unfortunately are not __``longer available``__ .   |
| **`pcc_eng_10_025.1543_x0390217_1:09-10-11`**  | The Middle Schools Leadership Program has expired and not __``longer available``__ .                            |
| **`pcc_eng_23_082.5804_x1318338_5:7-8-9`**     | Sorry , the searched product is not __``longer available``__ .                                                  |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/neg-longer/neg-longer_available_ex.md`


#### 2.2. "longer available" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*longer available*" found in **positive** hits.

### 3. _longer viable_


#### 3.1. "longer viable" under _negative_ polarity


|                                         | `token_str`                                                                                                                                                                                                                                                                                          |
|:----------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19990120_0551_7:08-09-10`**  | the 4 percent target for 1999 was no __``longer viable``__ , they argued .                                                                                                                                                                                                                           |
| **`nyt_eng_20051221_0312_40:41-42-43`** | and one thing Boras correctly pointed out in the massive negotiating manifesto he was schlepping around the baseball world the past month or so was that Damon 's value to a team will not automatically cease the minute he is no __``longer viable``__ as either a center fielder or leadoff man . |
| **`apw_eng_19970709_0356_10:45-46-47`** | David Thomas , chief executive of Whitbread PLC , which has sponsored the race since it started in 1973-74 , said the company 's prime business interests _ retail and beer products _ were in Britain and that sponsorship of the worldwide event was no __``longer viable``__ .                    |
| **`nyt_eng_19990929_0329_19:13-14-15`** | Asarco and Cyprus Amax , essentially acknowledging that their original deal was no __``longer viable``__ , postponed shareholder votes scheduled for Thursday on the merger , giving each other until Tuesday to reconsider their options , even ditch merger plans altogether .                     |
| **`nyt_eng_20050221_0028_16:08-09-10`** | Schroeder declared last week that NATO was no __``longer viable``__ as the main venue for trans-Atlantic communications , and recommended that a panel of high-level officials study the problem .                                                                                                   |
| **`apw_eng_20090307_0028_9:19-20-21`**  | he says only that the `` situation '' since the truce was reached means his music business was no __``longer viable``__ .                                                                                                                                                                            |
| **`apw_eng_20090307_0165_9:14-15-16`**  | he says only that the `` situation '' means his music business was no __``longer viable``__ .                                                                                                                                                                                                        |
| **`nyt_eng_20001018_0372_18:29-30-31`** | there are no active repair enzymes in a spore , he said , and radioactivity alone would eventually break the DNA into pieces that would make a spore no __``longer viable``__ .                                                                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/neg-longer/neg-longer_viable_ex.md`


#### 3.2. "longer viable" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*longer viable*" found in **positive** hits.

### 4. _longer pink_


#### 4.1. "longer pink" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20000815_0227_55:13-14-15`**         | stir-fry for 1 to 2 minutes , or until the pork is no __``longer pink``__ .                                                                                                                |
| **`nyt_eng_19990623_0145_82:09-10-11`**         | Simmer about 4 minutes or until chicken is no __``longer pink``__ .                                                                                                                        |
| **`nyt_eng_20050810_0048_12:21-22-23`**         | while crust bakes , in a large skillet over medium heat , cook the beef until it crumbles and is no __``longer pink``__ .                                                                  |
| **`nyt_eng_19991020_0189_2:11-12-13`**          | add half the pork and stir-fry 3 minutes , until no __``longer pink``__ .                                                                                                                  |
| **`pcc_eng_03_033.7772_x0530908_13:25-26-27`**  | When you flip the chicken for the last time , brush with the reserved barbecue sauce mixture and continue cooking until the chicken is not __``longer pink``__ and registers 160 degrees . |
| **`pcc_eng_29_098.1553_x1569691_029:10-11-12`** | Bake in the preheated oven until the chicken is not __``longer pink``__ in the center , and crispy on the outside , about 45 minutes .                                                     |
| **`nyt_eng_19990825_0071_74:10-11-12`**         | add ground beef and cook , stirring , until no __``longer pink``__ , about 5 minutes .                                                                                                     |
| **`nyt_eng_19990609_0159_39:14-15-16`**         | bake in 325-degree oven about 45 minutes or until chicken is tender and no __``longer pink``__ .                                                                                           |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/neg-longer/neg-longer_pink_ex.md`


#### 4.2. "longer pink" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*longer pink*" found in **positive** hits.

### Saving *`longer`* Samples

 > in `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/`

1. *longer lasting*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/neg-longer/neg-longer_lasting_99ex~2.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/pos-longer/pos-longer_lasting_99ex~99.csv"`

2. *longer available*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/neg-longer/neg-longer_available_99ex~23.csv"`

3. *longer viable*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/neg-longer/neg-longer_viable_99ex~13.csv"`

4. *longer pink*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/longer/neg-longer/neg-longer_pink_99ex~9.csv"`


## 10. *particularly*

|                                     |   `f` |   `dP1` |   `LRC` |   `P1` |     `G2` |   `f2` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `adj_total` |
|:------------------------------------|------:|--------:|--------:|-------:|---------:|-------:|----------:|------------:|------------:|--------:|-------:|--------------:|
| **NEGany~particularly_religious**   |   485 |    0.49 |    4.08 |   0.99 |   631.52 |    489 |    244.50 |      240.50 |        0.50 |    0.00 |   0.00 |         3,580 |
| **NEGany~particularly_new**         |   747 |    0.48 |    4.08 |   0.98 |   928.94 |    759 |    379.50 |      367.50 |        0.49 |    0.00 |   0.00 |        21,548 |
| **NEGmir~particularly_new**         |   404 |    0.49 |    4.02 |   0.99 |   529.06 |    407 |    203.50 |      200.50 |        0.50 |    0.00 |   0.00 |         4,393 |
| **NEGany~particularly_surprising**  | 1,069 |    0.47 |    3.78 |   0.97 | 1,231.31 |  1,102 |    551.00 |      518.00 |        0.48 |    0.00 |   0.00 |        18,886 |
| **NEGmir~particularly_wrong**       |   212 |    0.50 |    3.64 |   1.00 |   293.97 |    212 |    106.00 |      106.00 |        0.50 |    0.00 |   0.00 |         8,526 |
| **NEGany~particularly_original**    |   360 |    0.48 |    3.43 |   0.98 |   446.17 |    366 |    183.00 |      177.00 |        0.49 |    0.00 |   0.00 |         4,705 |
| **NEGany~particularly_comfortable** |   291 |    0.49 |    3.32 |   0.99 |   366.62 |    295 |    147.50 |      143.50 |        0.49 |    0.00 |   0.00 |        24,239 |
| **NEGmir~particularly_surprising**  |   166 |    0.49 |    3.02 |   0.99 |   219.33 |    167 |     83.50 |       82.50 |        0.50 |    0.00 |   0.00 |         1,270 |
| **NEGmir~particularly_unusual**     |   170 |    0.49 |    2.87 |   0.99 |   216.70 |    172 |     86.00 |       84.00 |        0.49 |    0.00 |   0.00 |           922 |
| **NEGmir~particularly_close**       |   136 |    0.49 |    2.71 |   0.99 |   178.12 |    137 |     68.50 |       67.50 |        0.50 |    0.00 |   0.00 |         4,972 |
| **COM~particularly_acute**          |   120 |    0.50 |    2.65 |   1.00 |   166.36 |    120 |     60.00 |       60.00 |        0.50 |    0.00 |   0.00 |         1,097 |
| **NEGmir~particularly_remarkable**  |   108 |    0.50 |    2.61 |   1.00 |   149.74 |    108 |     54.00 |       54.00 |        0.50 |    0.00 |   0.00 |         1,062 |
| **NEGany~particularly_sure**        |    76 |    0.50 |    1.93 |   1.00 |   105.36 |     76 |     38.00 |       38.00 |        0.50 |    0.00 |   0.00 |       134,058 |
| **NEGany~particularly_forthcoming** |    72 |    0.50 |    1.84 |   1.00 |    99.81 |     72 |     36.00 |       36.00 |        0.50 |    0.00 |   0.00 |         2,241 |
| **NEGany~particularly_glamorous**   |    71 |    0.50 |    1.82 |   1.00 |    98.43 |     71 |     35.50 |       35.50 |        0.50 |    0.00 |   0.00 |         3,000 |
| **NEGany~particularly_reassuring**  |    70 |    0.50 |    1.80 |   1.00 |    97.04 |     70 |     35.00 |       35.00 |        0.50 |    0.00 |   0.00 |         1,066 |
| **NEGmir~particularly_novel**       |    54 |    0.50 |    1.50 |   1.00 |    74.86 |     54 |     27.00 |       27.00 |        0.50 |    0.00 |   0.00 |           167 |
| **NEGmir~particularly_religious**   |    53 |    0.50 |    1.47 |   1.00 |    73.48 |     53 |     26.50 |       26.50 |        0.50 |    0.00 |   0.00 |           340 |
| **NEGmir~particularly_spectacular** |    45 |    0.50 |    1.19 |   1.00 |    62.39 |     45 |     22.50 |       22.50 |        0.50 |    0.00 |   0.00 |           674 |
| **NEGmir~particularly_shocking**    |    42 |    0.50 |    1.07 |   1.00 |    58.23 |     42 |     21.00 |       21.00 |        0.50 |    0.00 |   0.00 |           549 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/ALL-particularly_10mostNEG-bigrams_AMscores.2024-08-07.md`


|                          |       `N` |      `f1` |   `adv_total` |
|:-------------------------|----------:|----------:|--------------:|
| **NEGATED_particularly** | 6,347,362 | 3,173,681 |        76,722 |
| **NEGMIR_particularly**  |   583,470 |   291,735 |        10,020 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/particularly_MarginalFreqs_2024-08-07.md`


### 1. _particularly new_


#### 1.1. "particularly new" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                           |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_14_018.8183_x0287860_097:1-6-7`**   | None of these behaviors are __``particularly new``__ -- but they 've become remarkably easier for us to partake in .                                                                                  |
| **`pcc_eng_27_051.8294_x0821528_06:4-5-6`**    | These challenges are n't __``particularly new``__ , but things move at the speed of Twitter , and so these challenges have increased and seem to follow a pattern , a professor ( often but not [...] |
| **`pcc_eng_23_031.0959_x0485846_19:20-21-22`** | One of the problems with reading old scifi is that what was new and radical in the 50s is not __``particularly new``__ or radical today .                                                             |
| **`pcc_eng_13_087.6797_x1400902_5:09-10-11`**  | Choreography is not bad , though there is nothing __``particularly new``__ or special going on ( to my untrained eyes ) .                                                                             |
| **`pcc_eng_06_026.1408_x0406862_04:08-09-10`** | The series is over , I have nothing __``particularly new``__ to add , but I 'd like to try to summarize my points and provide an index to the posts in this series .                                  |
| **`pcc_eng_23_035.2532_x0553260_078:4-5-6`**   | But there was nothing __``particularly new``__ in doing that : ecclesiastics had long featured prominently in medieval depictions of hell .                                                           |
| **`pcc_eng_15_046.1740_x0730419_01:1-5-6`**    | None of this is __``particularly new``__ and this habit of acquiring competitors or influencing them from afar is nothing out of the ordinary .                                                       |
| **`pcc_eng_14_009.7002_x0140501_18:4-5-6`**    | " Gamification is not __``particularly new``__ .                                                                                                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_new_ex.md`


#### 1.2. "particularly new" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                                                                                                                   |
|:--------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_15_056.0787_x0890207_20:1-2`**   | __``Particularly new``__ services are scarce , but the necessity for them is massive .                                                                                                                                                                                                                                                                        |
| **`pcc_eng_00_030.8032_x0481723_09:2-3`**   | A __``particularly new``__ standpoint , often put forward in direct contradiction to the current mainstream of evolutionary biology , presents a conception which is bound to the formal theory of dynamic systems and methodologically to the mathematical modelling of regulatory networks .                                                                |
| **`pcc_eng_04_059.0422_x0937511_33:14-15`** | Take note of segmentation info offered by web site customer knowledge evaluation , __``particularly new``__ versus repeat customers , and referral sources .                                                                                                                                                                                                  |
| **`pcc_eng_14_072.0829_x1149194_1:5-6`**    | If you have any __``particularly new``__ equipment that you picked up in Canada you might want to bring the receipt as well so the border guard ca n't accuse of you picking it up in the US and making you pay taxes on it .                                                                                                                                 |
| **`pcc_eng_08_013.8838_x0208306_60:32-33`** | This latest addition to the Casa Magna D. Magnus II line certainly maintains the track record of the previous releases , though I 'm hesitant to say it shows off anything __``particularly new``__ or exciting via this distinct vitola .                                                                                                                    |
| **`pcc_eng_06_089.6953_x1434414_19:31-32`** | Cook sure that your advice for certain would you ought to be able so that you can withstand the criticism of the screen help make sure this tool would be __``particularly new``__ definition no fakes would try to be allowed , keep in mind this would absolutely not be naturally thought of by other one people and / or it should be inherently useful . |
| **`pcc_eng_18_069.9837_x1117175_40:7-8`**   | Twitter is going to be something __``particularly new``__ yet very relationship .                                                                                                                                                                                                                                                                             |
| **`pcc_eng_06_038.4863_x0606292_35:23-24`** | This site is continually being developed and updated in line with customer requirements , with more and more features becoming available , __``particularly new``__ on -line services for Online account holders .                                                                                                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_new_ex.md`


### 2. _particularly religious_


#### 2.1. "particularly religious" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_19_078.4315_x1250922_23:6-7-8`**     | For example if they are n't __``particularly religious``__ then you should avoid a religious message in case they take offence to it or find it uncomfortable                                                                                                                                              |
| **`pcc_eng_07_021.9275_x0338523_18:39-40-41`**  | And then there 's his daddy ( yes , it took some getting used to , but this single mother by choice 's fiance has officially become " daddy " by Noah 's own choosing ) who 's not __``particularly religious``__ , but does happen to be Jewish .                                                         |
| **`pcc_eng_23_005.2133_x0067956_03:06-10-11`**  | The odd thing is , none of them are __``particularly religious``__ , or even all that disciplined , so I wonder why they do it .                                                                                                                                                                           |
| **`pcc_eng_22_054.2767_x0860766_045:04-09-10`** | As adults , neither of the siblings was __``particularly religious``__ , but they felt culturally Jewish - Amy was famously meant to have cooked chicken soup for her bodyguards .                                                                                                                         |
| **`pcc_eng_26_089.5626_x1432130_09:13-14-15`**  | Some religions will perform funeral services even if your loved one was not __``particularly religious``__ , or did not attend worship very often .                                                                                                                                                        |
| **`pcc_eng_17_041.7237_x0657893_170:18-19-20`** | On top of that , such a society would also have a group of men who are n't __``particularly religious``__ and who like to make dirty jokes about naked women , but who are easily whipped up into a frenzy of anger at the thought of all the women out there taking showers that they do n't get to see . |
| **`pcc_eng_08_044.1793_x0698757_35:23-24-25`**  | Though the author 's opinions may not be popular with every reader , and may even be contrary to those who are not __``particularly religious``__ , the story and the execution are truly brilliant .                                                                                                      |
| **`pcc_eng_01_091.5944_x1464856_258:3-4-5`**    | My ( not __``particularly religious``__ ) parents sent me to Catholic school from 1st - 12th grade , solely for the quality of education , though of course I received all of the Catholic guilt and brainwashing that inevitably comes with the package .                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_religious_ex.md`


#### 2.2. "particularly religious" under _positive_ polarity


|                                            | `token_str`                                                                                                                                    |
|:-------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_06_052.9605_x0840597_384:4-5`** | Is your family __``particularly religious``__ ?                                                                                                |
| **`pcc_eng_18_079.5602_x1272095_08:6-7`**  | It appears to be a __``particularly religious``__ community .                                                                                  |
| **`pcc_eng_19_059.4744_x0943839_07:3-4`**  | As one __``particularly religious``__ zealot ( a close relative ) said , " We must not allow Clinton to appoint liberal Supreme Court judges . |
| **`pcc_eng_18_011.9826_x0177810_19:1-2`**  | __``Particularly religious``__ institutions that think they will be compromised . "                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_religious_ex.md`


### 3. _particularly surprising_


#### 3.1. "particularly surprising" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                        |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_23_039.2520_x0618063_10:2-7-8`**    | But none of that should be __``particularly surprising``__ if you 've ever seen a Transformers movie before .                                                                                                      |
| **`pcc_eng_13_018.8202_x0287942_38:09-10-11`** | The actionable consequences for search engine marketing are not __``particularly surprising``__ :                                                                                                                  |
| **`pcc_eng_27_022.0502_x0340328_23:08-09-10`** | From this point of view it is n't __``particularly surprising``__ to find a computer lurking in the complexities of a nonlinear differential equation .                                                            |
| **`pcc_eng_15_103.6342_x1659052_34:01-09-10`** | None of this , of course , is __``particularly surprising``__ .                                                                                                                                                    |
| **`pcc_eng_11_002.7776_x0028870_52:22-23-24`** | Nevertheless , his passionate defense of Leviathan , in all its monstrous , unconstitutional glory , offers a rare , if not __``particularly surprising``__ , glimpse into this faux -moderate President 's mind . |
| **`pcc_eng_15_095.1125_x1521187_05:5-6-7`**    | But this revelation is n't __``particularly surprising``__ .                                                                                                                                                       |
| **`pcc_eng_13_085.8027_x1370712_27:3-4-5`**    | This is n't __``particularly surprising``__ when you consider the last two droughts of similar extent and severity occurred in the 1950s and 1930s - prior to much anthropogenic forcing .                         |
| **`pcc_eng_19_027.1226_x0421635_08:08-09-10`** | He also said that the numbers are n't __``particularly surprising``__ after six years of economic recovery .                                                                                                       |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_surprising_ex.md`


#### 3.2. "particularly surprising" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                   |
|:---------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_10_058.3346_x0927190_16:7-8`**    | The timing , however , is __``particularly surprising``__ because Maccagnan ran the Jets ' draft less than three weeks ago , including taking defensive tackle Quinnen Williams with the No. 3 overall pick . |
| **`pcc_eng_24_096.8854_x1550911_82:2-3`**    | Anything __``particularly surprising``__ ?                                                                                                                                                                    |
| **`pcc_eng_22_100.2336_x1603686_19:6-7`**    | To me , this was __``particularly surprising``__ given the experience on the team .                                                                                                                           |
| **`nyt_eng_20070510_0292_24:4-5`**           | the decision was __``particularly surprising``__ considering the number of guilty pleas and the fact that a major architect of the arrangement scheme was the government 's pivotal witness .                 |
| **`pcc_eng_10_036.0586_x0567013_34:7-8`**    | What makes the account presented here __``particularly surprising``__ and interesting is the fact that it is the product of a group of established , elected politicians .                                    |
| **`pcc_eng_23_015.1459_x0227946_27:4-5`**    | Was there anything __``particularly surprising``__ about developing for Pebble ?                                                                                                                              |
| **`nyt_eng_19960221_0595_3:3-4`**            | it was __``particularly surprising``__ coming on the heels of Russian Vice President Viktor Chernomyrdin 's meetings with Vice President Al Gore , he said .                                                  |
| **`pcc_eng_12_066.6514_x1061013_072:15-16`** | Is there anything that anyone 's said about their writing process that you found __``particularly surprising``__ .                                                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_surprising_ex.md`


### 4. _particularly wrong_


#### 4.1. "particularly wrong" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                        |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_03_034.6677_x0545448_08:3-4-5`**    | There 's nothing __``particularly wrong``__ with canned chickpeas , it 's just soaking your own gives your hummus that extra fresh flavour .                                                                                                       |
| **`pcc_eng_12_063.1209_x1004463_07:3-4-5`**    | There 's nothing __``particularly wrong``__ with writing a story about monsters where the monsters are analogies for the fears and worries we have about growing up .                                                                              |
| **`pcc_eng_07_022.9432_x0354778_07:3-4-5`**    | There is nothing __``particularly wrong``__ about Old Times but if it is a masterpiece , it is one that is in a minor-key .                                                                                                                        |
| **`pcc_eng_27_023.1755_x0358508_25:3-4-5`**    | There is nothing __``particularly wrong``__ with the last point ; it 's just that we should not be fooled into thinking that this year 's budget is somehow radically different from last year 's budget .                                         |
| **`pcc_eng_05_037.6605_x0593581_53:09-10-11`** | That said , of course , there 's nothing __``particularly wrong``__ with taking your cues from both hip hop and The Fall , leaving Key Markets as an exciting , bewildering album that shows off the band 's talents at blazing a new punk trail . |
| **`pcc_eng_10_025.0968_x0389287_39:3-4-5`**    | There 's nothing __``particularly wrong``__ with his latest - it 's earnest , genuine and award -worthy .                                                                                                                                          |
| **`pcc_eng_09_038.6298_x0609008_68:3-4-5`**    | There is nothing __``particularly wrong``__ with this ad , but I do n't really want to have to work so hard .                                                                                                                                      |
| **`pcc_eng_20_091.7058_x1465461_60:3-4-5`**    | There 's nothing __``particularly wrong``__ in the statistics , but there 's nothing particularly right either .                                                                                                                                   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_wrong_ex.md`


#### 4.2. "particularly wrong" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                                                                                                                  |
|:--------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_005.5293_x0073165_098:4-5`**  | This claim is __``particularly wrong``__ with respect to internationalism .                                                                                                                                                                                                                                                                                  |
| **`pcc_eng_10_025.2668_x0392040_005:7-8`**  | On this particular day this seemed __``particularly wrong``__ .                                                                                                                                                                                                                                                                                              |
| **`pcc_eng_02_022.9843_x0355810_29:46-47`** | I 've gotten into the habit of doing this just so I 've got a sense of how my day 's going financially , and also so that when I get the final figure after the ride 's over I can tell if something 's __``particularly wrong``__ with the amount - when it does happen its usually around a toll that was n't added to the fare when it should have been . |
| **`nyt_eng_20021129_0299_17:4-5`**          | but it is __``particularly wrong``__ that parts of the record that are being cited in the parties ' briefs , and that could end up in the court 's decision , are withheld from the public .                                                                                                                                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_wrong_ex.md`


### 5. _particularly original_


#### 5.1. "particularly original" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                          |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_031.4981_x0493136_17:3-8-9`**    | However , none of these things are __``particularly original``__ : everyone who wants to become a business owner knows them .                                                                                                                                                                        |
| **`pcc_eng_13_039.0067_x0614526_08:18-19-20`** | The premise of a group of film students shooting a clandestine porn film on the side is n't __``particularly original``__ , but it is handled with a genuine affection for its subject matter , and enough fresh angles that the overall script did n't feel as though it had all been done before . |
| **`pcc_eng_26_085.8858_x1372655_125:4-5-6`**   | I will say nothing __``particularly original``__ here , so feel free to whistle your own cheerful tune as you read this .                                                                                                                                                                            |
| **`pcc_eng_12_064.9497_x1033756_27:10-11-12`** | The European-esque architecture combined with a steampunk aesthetic is n't __``particularly original``__ , but combined with Kyo Ani 's level of care it becomes something truly captivating .                                                                                                       |
| **`pcc_eng_05_035.2847_x0555274_24:7-8-9`**    | On the downside the plot is n't __``particularly original``__ and it really is just a man shooting , jumping and punching things , but it is done so well that you must argue that it is one of the best we have seen recently .                                                                     |
| **`nyt_eng_20060503_0192_2:4-5-6`**            | its message was not __``particularly original``__ .                                                                                                                                                                                                                                                  |
| **`pcc_eng_26_030.2514_x0472632_70:5-6-7`**    | The main story is n't __``particularly original``__ , borrowing from everything from Thor to Spider - Man to even something like The Lion King .                                                                                                                                                     |
| **`pcc_eng_29_008.2096_x0116513_020:3-4-5`**   | It 's not __``particularly original``__ or insightful , but it is worth commenting on for a number of reasons .                                                                                                                                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_original_ex.md`


#### 5.2. "particularly original" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                     |
|:--------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_18_033.4006_x0524505_21:27-28`** | The book 's explanation of the First Congo War , by identifying an ideological project of liberation as a central driver of the conflict , is __``particularly original``__ .                                                                                   |
| **`pcc_eng_14_028.2203_x0439673_15:12-13`** | But I must admit this particular story presented it in a __``particularly original``__ way - especially with such a non-conformist like Reacher serving as investigator .                                                                                       |
| **`nyt_eng_19980423_0168_39:6-7`**          | the Spade-Segal boss-employee codependency is __``particularly original``__ .                                                                                                                                                                                   |
| **`pcc_eng_09_093.7557_x1500740_5:2-3`**    | A __``particularly original``__ model was that of the ' Turks against Moors ' .                                                                                                                                                                                 |
| **`pcc_eng_11_050.1872_x0795709_14:1-2`**   | __``Particularly original``__ is the layout of some bathrooms , newly renovated , painted with lively pastel colours .                                                                                                                                          |
| **`pcc_eng_29_008.1861_x0116148_18:35-36`** | They pursue Richardson 's ambiguity from his editorial technique , through his attitudes toward paternal , social , religious , and literary contexts , to his reception by women readers and authors -- a __``particularly original``__ and powerful section . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_original_ex.md`


### 6. _particularly comfortable_


#### 6.1. "particularly comfortable" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                               |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_089.8960_x1436844_27:22-24-25`** | For Muslim women who don their natural hair , this could cause unnecessary pressure to conform to hair practices they may not be __``particularly comfortable``__ with .                                                                                                  |
| **`nyt_eng_19970326_0341_14:26-27-28`**        | the actresses , including Cameron Diaz , Kate Winslet , Claire Danes , and Renee Zellweger , all look very shimmery and chesty , if not __``particularly comfortable``__ .                                                                                                |
| **`pcc_eng_06_077.2611_x1233386_17:11-12-13`** | Like most interpreters and intermediaries between cultures , Lin was never __``particularly comfortable``__ in either world , and like Thoreau and the other sages whom he cites , he was an outlier even in the society that he was supposedly introducing to the West . |
| **`pcc_eng_21_028.1554_x0438887_073:5-6-7`**   | My false lashes were n't __``particularly comfortable``__ either .                                                                                                                                                                                                        |
| **`pcc_eng_25_082.4756_x1318885_21:16-17-18`** | Leather tends to make the best impression and looks very stylish , but it is not __``particularly comfortable``__ , especially in hot summer months .                                                                                                                     |
| **`pcc_eng_09_080.7146_x1289509_03:16-17-18`** | Few people take racing bikes on the road with them , and they really are n't __``particularly comfortable``__ for casual rides around town .                                                                                                                              |
| **`nyt_eng_19961011_0079_29:4-5-6`**           | `` I 'm not __``particularly comfortable``__ getting companies listed in every corner of the globe , '' said Tina So , who manages about $ 24 million in China funds at Schroder Investment Management -LRB- Asia -RRB- Ltd. in Hong Kong .                               |
| **`pcc_eng_12_002.7728_x0028664_10:27-29-30`** | History has largely perceived the decision as a moment of weakness on the part of the king , a man of small physical stature who had never been __``particularly comfortable``__ in his role .                                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_comfortable_ex.md`


#### 6.2. "particularly comfortable" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                    |
|:--------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_19980629_0183_70:21-22`**        | `` Over the years I 've sung everything from Monteverdi to Stravinsky , with everything in between ; I feel __``particularly comfortable``__ in French roles . |
| **`pcc_eng_13_058.2834_x0925713_04:24-25`** | Plug it into the headphone jack of the phone and it 'll provide crystal clear audio on both ends and the earpiece is __``particularly comfortable``__ .        |
| **`pcc_eng_15_047.4650_x0751121_15:15-16`** | Jupp Heynckes ' charges were utterly dominant , with Swiss midfielder Xherdan Shaqiri looking __``particularly comfortable``__ in the No. 10 role .            |
| **`pcc_eng_06_068.4326_x1091103_06:6-7`**   | The orange jumpsuit is therefore __``particularly comfortable``__ to wear made of a soft fabric and .                                                          |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_comfortable_ex.md`


### 7. _particularly unusual_


#### 7.1. "particularly unusual" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                       |
|:-----------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`apw_eng_20090326_0084_8:4-5-6`**            | `` There 's nothing __``particularly unusual``__ about that . ''                                                                                                                                                                                                                                                                                                                  |
| **`pcc_eng_13_035.3491_x0555315_16:3-4-5`**    | There was nothing __``particularly unusual``__ about them .                                                                                                                                                                                                                                                                                                                       |
| **`pcc_eng_24_037.8128_x0595011_19:1-2-3`**    | Nothing __``particularly unusual``__ about that - pretty much every beauty contest winner cries .                                                                                                                                                                                                                                                                                 |
| **`pcc_eng_06_073.3840_x1170881_020:4-5-6`**   | So far , nothing __``particularly unusual``__ .                                                                                                                                                                                                                                                                                                                                   |
| **`pcc_eng_28_021.8256_x0336428_021:7-8-9`**   | These extremes are not rare , not __``particularly unusual``__ .                                                                                                                                                                                                                                                                                                                  |
| **`pcc_eng_11_002.8454_x0029975_10:41-42-43`** | Until now the Instagram o Embed endpoint to make it easier for businesses in finding new company profiles , analytics and the introduction of private photo and naturally there buy followers preneur who shared that Twitter linkedin and Google I rarely __``particularly unusual``__ all by themself on the other hand , if equalled followed by being the person who proves . |
| **`pcc_eng_22_053.4087_x0846720_34:10-11-12`** | In Harvey 's defense , what he did was n't __``particularly unusual``__ at the time .                                                                                                                                                                                                                                                                                             |
| **`pcc_eng_08_076.0524_x1215098_24:44-45-46`** | " The language in Judge Royce Lamberth 's order undercut the Democratic presidential contender 's assertion she was allowed to set up a private email server in her home for her work as the country 's top diplomat and that the arrangement was not __``particularly unusual``__ . "                                                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_unusual_ex.md`


#### 7.2. "particularly unusual" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                                                             |
|:--------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_087.4552_x1396663_22:30-31`** | We will make assumptions about the weight of the roof based upon our experience and relevant codes of practice but we would need to know if there is anything __``particularly unusual``__ about the roof that might make it excessively heavy .                                                        |
| **`pcc_eng_01_069.1896_x1103070_4:4-5`**    | The location was __``particularly unusual``__ as it was in a department store of all places .                                                                                                                                                                                                           |
| **`pcc_eng_19_018.1741_x0277094_08:31-32`** | Their bold aesthetic is characterized by frozen scenes of dynamic action often at impressive scale , however it is the delicate details of the everyday life which make their works __``particularly unusual``__ .                                                                                      |
| **`pcc_eng_19_038.2471_x0601017_33:09-10`** | " Stop occasionally if you come across a __``particularly unusual``__ word , but do n't talk about individual words so much that you interrupt the flow of the story , " says Jill Allor , Ph. D. , chair of teaching and learning at the Simmons School at Southern Methodist University , in Dallas . |
| **`pcc_eng_00_063.6863_x1013551_17:3-4`**   | In one __``particularly unusual``__ moment , Trump claimed that Hatch had told him he was a better president than both Abraham Lincoln and George Washington .                                                                                                                                          |
| **`apw_eng_20091021_0077_25:5-6`**          | exhibit photos show a __``particularly unusual``__ outfit : a long skirt unbuttoned to mid-thigh to show knickers underneath .                                                                                                                                                                          |
| **`pcc_eng_20_011.6627_x0171965_09:2-3`**   | A __``particularly unusual``__ version of Dutch Harlem was produced exclusively for the great London store Liberty 's , and further exclusive productions of biscuit barrels were produced for the biscuit manufacturers Mc Vitie and Price .                                                           |
| **`pcc_eng_20_072.5064_x1155277_56:4-5`**   | And it 's __``particularly unusual``__ , if you look at polling , the economic and jobs concerns are always one or two , and they are still are in large measure .                                                                                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_unusual_ex.md`


### 8. _particularly close_


#### 8.1. "particularly close" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_082.5648_x1318433_10:25-26-27`**  | From 2007 through 2009 , no catcher started more games behind the plate or caught as many innings as Martin , and it 's not __``particularly close``__ either .                                                                                            |
| **`pcc_eng_09_043.1783_x0682451_19:5-6-7`**     | Dombrowski and Sabean are n't __``particularly close``__ , and they did n't learn their baseball in the same place .                                                                                                                                       |
| **`pcc_eng_01_043.6747_x0689541_005:39-40-41`** | Clippard was also the game 's most extreme fly - ball pitcher last season ; his 60.6 percent fly - ball rate was the highest among any pitcher that threw at least 20 innings , and it was n't __``particularly close``__ .                                |
| **`pcc_eng_10_025.5441_x0396534_48:3-4-5`**     | He is not __``particularly close``__ to Washington -- which has enhanced his local appeal -- but administration concerns about undermining his position in Syria via a Western embrace ignore the urgency of the situation .                               |
| **`pcc_eng_09_038.0951_x0600379_30:16-17-18`**  | That 's not the worst thing in the world to be , but it 's not __``particularly close``__ to engendering repeat viewing either .                                                                                                                           |
| **`pcc_eng_09_038.3888_x0605091_59:3-4-5`**     | We 're not __``particularly close``__ but I know they do love me and that I 'd be inflicting an awful lot of pain if I told them .                                                                                                                         |
| **`pcc_eng_27_101.3555_x1623413_26:14-15-16`**  | For instance , Ledesma shared stories about his father with whom he was n't __``particularly close``__ , according to " Migrant Sun , " but taught him the value of a hard work ethic , something that he touches on throughout the entirety of his book . |
| **`pcc_eng_02_083.3817_x1331919_37:35-36-37`**  | Ellerbe , one of the team 's top tacklers who is dealing with injuries to his ankle , foot and both of his thumbs , missed Sunday 's game and acknowledged that he was n't __``particularly close``__ to playing .                                         |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_close_ex.md`


#### 8.2. "particularly close" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                      |
|:--------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_007.1337_x0098873_22:5-6`**   | GD also has a __``particularly close``__ relationship with the Pentagon ; 94 % of its contracts come from the US government .                                                                    |
| **`pcc_eng_23_011.9134_x0176084_24:2-3`**   | Pay __``particularly close``__ attention to the final four minutes when JFK talks about tax policies .                                                                                           |
| **`pcc_eng_09_029.1837_x0456329_21:8-9`**   | Simply place them in your garden , __``particularly close``__ to the snail 's favourite plants .                                                                                                 |
| **`pcc_eng_12_085.8645_x1371240_22:14-15`** | Place where Nature is everywhere present , the Ecole de Nancy Museum is __``particularly close``__ of the thematic of these European Heritage Days , " Cultural heritage - natural heritage " !  |
| **`pcc_eng_23_097.1991_x1554749_08:18-19`** | He was a very pretty child with long golden curls and bright blue eyes , and was __``particularly close``__ to the sister nearest to him in age , Princess Sophia , who called him " her son " . |
| **`pcc_eng_20_092.0010_x1470227_6:11-12`**  | A devout Swanson places honor above all else , pays __``particularly close``__ attention to their haircut and sustains the body off a steady supply of beef .                                    |
| **`pcc_eng_19_054.1597_x0857877_13:15-16`** | During this time Bhai Sahib came in contact with many great Jujharoos and became __``particularly close``__ to Shaheed Bhai Jugraj Singh ' Toofan ' .                                            |
| **`nyt_eng_20051105_0035_41:17-18`**        | here , the comparison between the history of American minority groups and those in France seems __``particularly close``__ .                                                                     |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_close_ex.md`


### 9. _particularly acute_


#### 9.1. "particularly acute" under _negative_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*particularly acute*" found in **negative** hits.

#### 9.2. "particularly acute" under _positive_ polarity


|                                            | `token_str`                                                                                                                                                                                                                      |
|:-------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_24_022.2793_x0343737_035:4-5`** | He became a __``particularly acute``__ analyst of the financialization of the U.S. economy , and the growth in inequality that has accompanied it .                                                                              |
| **`pcc_eng_00_009.2005_x0132451_09:5-6`**  | The tensions have become __``particularly acute``__ within the agency 's directorate of operations , which is responsible for global covert operations , the officials said .                                                    |
| **`pcc_eng_13_057.4434_x0912152_15:4-5`**  | These pressures are __``particularly acute``__ in Kazakhstan , which ( outside Russia ) is the largest and most economically developed of the former Soviet republics , with extensive and important mineral resources .         |
| **`pcc_eng_27_076.7944_x1225542_41:6-7`**  | The problem of violence is __``particularly acute``__ among disadvantaged young urban males .                                                                                                                                    |
| **`pcc_eng_14_106.5276_x1705757_28:8-9`**  | The battle between these two outlooks is __``particularly acute``__ for communist union officials .                                                                                                                              |
| **`pcc_eng_22_057.0104_x0905162_06:6-7`**  | The left 's ire was __``particularly acute``__ four years ago , because Brennan had been a high- level CIA official during the Bush Administration .                                                                             |
| **`nyt_eng_20010625_0124_76:4-5`**         | the shortage is __``particularly acute``__ in New England , where waiting times for a patient with a health status similar to Rossetti 's are three times as long as in other parts of the country , particularly in the South . |
| **`pcc_eng_16_098.9383_x1585335_10:4-5`**  | Demand will be __``particularly acute``__ for mid-range candidates with several years of experience , who are willing to accept salaries in the $ 55,000 to $ 65,000 range .                                                     |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_acute_ex.md`


### 10. _particularly remarkable_


#### 10.1. "particularly remarkable" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                          |
|:-----------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_04_088.3946_x1412071_585:1-2-3`**   | Nothing __``particularly remarkable``__ .                                                                                                                                            |
| **`pcc_eng_03_033.2465_x0522325_27:3-4-5`**    | There is nothing __``particularly remarkable``__ about civil servants , he explains in his lectures .                                                                                |
| **`pcc_eng_08_071.9768_x1149198_055:1-2-3`**   | Nothing __``particularly remarkable``__ in the grand scheme of things .                                                                                                              |
| **`pcc_eng_28_012.0738_x0179341_07:08-09-10`** | But was this already a known and not __``particularly remarkable``__ fact ?                                                                                                          |
| **`pcc_eng_15_090.9620_x1454074_19:4-5-6`**    | The method was not __``particularly remarkable``__ -- these sorts of phishing emails are the standard tools of cybercriminals .                                                      |
| **`pcc_eng_00_039.3781_x0619973_113:6-7-8`**   | That was unusual , but not __``particularly remarkable``__ , nor was their anatomical composition - heavier than any deer or horse he 'd seen , but still roughly similar in shape . |
| **`pcc_eng_04_051.4067_x0814326_32:5-6-7`**    | The stories themselves are not __``particularly remarkable``__ , but the writing is straightforward and crisp .                                                                      |
| **`pcc_eng_06_079.3459_x1266836_37:09-10-11`** | At the time , Marco 's essay was not __``particularly remarkable``__ but it would later become significant for what was not in it .                                                  |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_remarkable_ex.md`


#### 10.2. "particularly remarkable" under _positive_ polarity


|                                            | `token_str`                                                                                                                                                                                                                                              |
|:-------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_15_009.5546_x0138138_37:3-4`**  | What 's __``particularly remarkable``__ about the current scheme is the size of its net ; my non - Chinese - speaking editor and some other colleagues have received the calls , which have also been reported in Canada , New Zealand , and Australia . |
| **`pcc_eng_04_056.7118_x0899746_013:8-9`** | What intelligence officials and terrorism experts find __``particularly remarkable``__ in his recent pronouncements is a shift in style from the raw anger and dark imagery of the post - 9/11 days .                                                    |
| **`pcc_eng_13_061.5851_x0979327_28:4-5`**  | The clarity is __``particularly remarkable``__ in a few instances where the actual pattern and weave of the costume fabric can be seen .                                                                                                                 |
| **`nyt_eng_19971223_0673_19:5-6`**         | what made his achievements __``particularly remarkable``__ , though , was that he could barely see .                                                                                                                                                     |
| **`nyt_eng_19961209_0720_23:6-7`**         | the turn of events was __``particularly remarkable``__ for Giuliani , the former prosecutor who has made a career of presenting himself as a maverick who resists the conventions of traditional politics .                                              |
| **`pcc_eng_03_050.5770_x0803060_32:3-4`**  | This is __``particularly remarkable``__ because these two genii are only distantly related , and they live in two completely different parts of the world .                                                                                              |
| **`pcc_eng_12_066.7946_x1063323_20:7-8`**  | " What makes the early results __``particularly remarkable``__ is the fact that the CRPT token sale attracted the attention of token - buyers from every corner of the world .                                                                           |
| **`pcc_eng_17_095.0297_x1519678_02:5-6`**  | CSI 's success is __``particularly remarkable``__ when you consider that the show 's Miami- and New York - based spin-offs have both come and gone while the original trucks on .                                                                        |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_remarkable_ex.md`


### 11. _particularly sure_


#### 11.1. "particularly sure" under _negative_ polarity


|                                              | `token_str`                                                                                                                                                                               |
|:---------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_15_046.9362_x0742689_11:3-4-5`**  | We 're not __``particularly sure``__ why Zdorovetskiy only decided to stalk African Americans .                                                                                           |
| **`pcc_eng_11_063.3971_x1009786_29:1-2-3`**  | Not __``particularly sure``__ why the Royals continue to pursue speedy players who lack on - base skills , but add Billy Hamilton to the roster .                                         |
| **`nyt_eng_19960603_0386_46:16-17-18`**      | DeSisto says the students spent only one night outside , adding , `` I 'm not __``particularly sure``__ that camping in the snow is a bad thing . ''                                      |
| **`pcc_eng_02_005.4518_x0071964_23:4-5-6`**  | If you 're not __``particularly sure``__ about the impact a certain improvement will have once it 's out in the market , testing how valuable this change is is certainly the way to go . |
| **`pcc_eng_02_091.9058_x1469739_044:3-4-5`** | I 'm not __``particularly sure``__ how I would build this infrastructure at this point , but its something that I would like to work on over the next few years .                         |
| **`pcc_eng_00_067.5922_x1076327_016:7-8-9`** | But at this point he 's not __``particularly sure``__ his original hypothesis is even worth testing anymore .                                                                             |
| **`pcc_eng_14_033.6898_x0528149_5:3-4-5`**   | I 'm not __``particularly sure``__ what the full context is for the work above but the images seem to be part of a Japanese television program called Trip to Japan .                     |
| **`pcc_eng_23_008.4908_x0120937_096:6-7-8`** | The truth is I 'm not __``particularly sure``__ why I do n't eat meat anymore .                                                                                                           |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_sure_ex.md`


#### 11.2. "particularly sure" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*particularly sure*" found in **positive** hits.

### 12. _particularly forthcoming_


#### 12.1. "particularly forthcoming" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                         |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_29_003.9387_x0047565_09:1-3-4`**     | Nobody was __``particularly forthcoming``__ about what was discussed , but there was no doubt plenty of discussion about where they go from here .                                                                  |
| **`pcc_eng_02_007.8482_x0110528_18:15-17-18`**  | Matthew tells me he 's always been enamored with stomach hair , but has n't been __``particularly forthcoming``__ about it .                                                                                        |
| **`pcc_eng_11_098.2159_x1573678_117:14-16-17`** | It 's just a little bit slower , and the Trump camp has not been __``particularly forthcoming``__ in terms of providing witnesses and at least some information .                                                   |
| **`pcc_eng_03_098.5024_x1578627_61:4-5-6`**     | When he is n't __``particularly forthcoming``__ , they gauge his eye out .                                                                                                                                          |
| **`nyt_eng_20070515_0045_44:19-20-21`**         | many of the industries and particularly the folks at Interstate Bakeries Corp. -LRB- which owns Hostess -RRB- were n't __``particularly forthcoming``__ .                                                           |
| **`pcc_eng_28_076.2055_x1216497_20:3-4-5`**     | He was not __``particularly forthcoming``__ .                                                                                                                                                                       |
| **`pcc_eng_02_002.5441_x0025004_15:5-6-7`**     | Predictably , they were n't __``particularly forthcoming``__ .                                                                                                                                                      |
| **`pcc_eng_23_039.3177_x0619144_077:29-31-32`** | Doombot shows up to give Victor a new body , but Victor does n't like the buff , weaponized Transformers body that Doombot makes , though Victor is n't being __``particularly forthcoming``__ about his hang-ups . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_forthcoming_ex.md`


#### 12.2. "particularly forthcoming" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*particularly forthcoming*" found in **positive** hits.

### 13. _particularly glamorous_


#### 13.1. "particularly glamorous" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                          |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_01_034.0819_x0534631_37:3-5-6`**     | It may not be __``particularly glamorous``__ to write cheques for replacement cassocks , or new copies of sheet music , or bursary funds , but those few good souls who do this , quietly and unnoticed , are building roads for the future .                                                                        |
| **`pcc_eng_19_013.1987_x0197028_018:28-29-30`** | With Bruce Figgins suspended for the first six games of the year , Lynch looks to be the Bulldogs only true blocking tight end , which while not __``particularly glamorous``__ , it makes the freshman virtually indispensable .                                                                                    |
| **`pcc_eng_14_081.2688_x1297763_171:6-7-8`**    | This process is lengthy and not __``particularly glamorous``__ : indeed , much of the groundwork and infrastructure for the project that this creative diary follows has been in development for nearly two years .                                                                                                  |
| **`pcc_eng_29_009.1046_x0131022_232:3-4-5`**    | It was n't __``particularly glamorous``__ or useful .                                                                                                                                                                                                                                                                |
| **`pcc_eng_20_045.3869_x0717056_14:08-09-10`**  | The routine parts of our business are not __``particularly glamorous``__ , but we should definitely be raising their profile .                                                                                                                                                                                       |
| **`pcc_eng_19_017.1522_x0260520_66:15-16-17`**  | JH : My office is very basic , and I am sad to say not __``particularly glamorous``__ .                                                                                                                                                                                                                              |
| **`pcc_eng_10_024.9418_x0386800_15:08-10-11`**  | Our first week as Sydney -siders has not been __``particularly glamorous``__ .                                                                                                                                                                                                                                       |
| **`pcc_eng_11_084.0994_x1344970_06:53-54-55`**  | Ok , so it does n't * entirely * suck -- you get to meet tons of interesting people , you help build companies , you have the potential to make lots of money -- but there are a lot of parts of it that outsiders do n't think about that are n't __``particularly glamorous``__ and can be downright frustrating . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_glamorous_ex.md`


#### 13.2. "particularly glamorous" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*particularly glamorous*" found in **positive** hits.

### 14. _particularly reassuring_


#### 14.1. "particularly reassuring" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                               |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_10_021.2803_x0327679_19:28-29-30`**  | And in this case , it eased my mind considerably , since the doctor 's breezy " oh , I sure it 's not croup " was n't __``particularly reassuring``__ , funnily enough .  |
| **`pcc_eng_00_035.3393_x0554606_09:24-25-26`**  | The CDC says " more research is needed to assess the effects on human health from exposure to phthalates " , which is not __``particularly reassuring``__ for consumers . |
| **`pcc_eng_05_089.1533_x1426095_11:4-6-7`**     | The estimate was not " __``particularly reassuring``__ " to the committee 's chairman , Smith , who said he 'd look into " possible budgetary assistance . "              |
| **`pcc_eng_15_033.8185_x0530665_37:3-4-5`**     | This is not __``particularly reassuring``__ in some countries , some European political circles , and even some European business circles .                               |
| **`pcc_eng_27_033.4758_x0524519_15:12-13-14`**  | But the Post 's description of how the program works is n't __``particularly reassuring``__ :                                                                             |
| **`pcc_eng_26_007.2316_x0100428_20:06-09-10`**  | That is n't ideal , nor is it __``particularly reassuring``__ , but that 's perhaps the biggest misconception about D'Antoni 's offense .                                 |
| **`pcc_eng_06_075.2916_x1201624_15:10-11-12`**  | It 's perfunctory and unilluminating an , therefore , not __``particularly reassuring``__ .                                                                               |
| **`pcc_eng_04_059.5875_x0946466_346:14-15-16`** | Miss Lesley smiled , and then stopped when she realised her expression was n't __``particularly reassuring``__ .                                                          |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_reassuring_ex.md`


#### 14.2. "particularly reassuring" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*particularly reassuring*" found in **positive** hits.

### 15. _particularly novel_


#### 15.1. "particularly novel" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                   |
|:------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_00_036.7442_x0577427_070:11-12-13`** | The audience went nuts and got two fitting , if not __``particularly novel``__ ( or energizing ) , Schumann ( ? ) encores .                                                                                                                                                                   |
| **`pcc_eng_03_085.6512_x1370701_05:17-18-19`**  | In a city where five-star restaurants are tucked away in nondescript strip malls , there 's nothing __``particularly novel``__ about a high / low culinary mash- up .                                                                                                                         |
| **`pcc_eng_10_049.5724_x0785567_29:6-7-8`**     | In truth , there 's nothing __``particularly novel``__ or disquieting about the scheme Murray 's drawn up , except insofar as the procedural extremism conservatives have deployed in the Obama era is alarming in general .                                                                  |
| **`pcc_eng_25_033.1210_x0519960_021:3-4-5`**    | There 's nothing __``particularly novel``__ about utilizing targeted and sometimes conflicting messages to different audiences , but it would be nice if GOPers stopped squealing like little piggies every time they get called on it , and projecting their dishonesty onto everyone else . |
| **`pcc_eng_20_088.9868_x1421466_013:27-28-29`** | For example , though it is great to see the mainstream media come to condemn the intellectual fallacies inherent with protectionist trade policies , there is nothing __``particularly novel``__ about a president imposing tariffs .                                                         |
| **`pcc_eng_22_002.3096_x0021261_63:6-7-8`**     | Of course , there is nothing __``particularly novel``__ about the relationship between reading and the " self " and its presentation .                                                                                                                                                        |
| **`pcc_eng_08_072.6067_x1159294_09:14-15-16`**  | The idea that ISPs might be interested in hosting cloud services is thus not __``particularly novel``__ , nor is it particularly hard for them to stand a cloud service up today .                                                                                                            |
| **`pcc_eng_27_106.1698_x1701274_23:24-25-26`**  | For a sense of what awaits , our seven courses began with a raw slice of hamachi and asparagus tips in ponzu - not __``particularly novel``__ , but a refreshing start .                                                                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_novel_ex.md`


#### 15.2. "particularly novel" under _positive_ polarity


|                                            | `token_str`                                                                                                                                                                                                                    |
|:-------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_11_003.0853_x0033856_05:2-3`**  | One __``particularly novel``__ tribute is MGo View .                                                                                                                                                                           |
| **`pcc_eng_03_038.0558_x0600129_068:4-5`** | MIRROR provides a __``particularly novel``__ take on Poe's " The Tell - Tale Heart . "                                                                                                                                         |
| **`pcc_eng_13_053.5796_x0849951_44:5-6`**  | The serving method is __``particularly novel``__ : the cooked lamb breast is heaped to one side of a tray used for tea ceremonies , surrounded by petite steamed breads ( called mantou ) and a spicy , savory dipping sauce . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_novel_ex.md`


### 16. _particularly spectacular_


#### 16.1. "particularly spectacular" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                            |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_13_036.1231_x0567775_4:4-5-6`**     | The beach is n't __``particularly spectacular``__ either , but that 's no bother , either .                                                                                                                                                                            |
| **`pcc_eng_28_025.0916_x0389041_14:1-3-4`**    | Neither was __``particularly spectacular``__ , but they were serviceable for my purposes .                                                                                                                                                                             |
| **`pcc_eng_21_015.3176_x0231211_29:3-4-5`**    | There was nothing __``particularly spectacular``__ about the brownies , but they were delicious and made for a well - portioned dessert following something as filling as Graccu 's Homemade Meat Pie .                                                                |
| **`apw_eng_19981117_0725_17:11-12-13`**        | though the Leonids occur every November , they are usually not __``particularly spectacular``__ .                                                                                                                                                                      |
| **`nyt_eng_19990914_0285_24:09-10-11`**        | Marino threw two touchdown passes Monday night , neither __``particularly spectacular``__ , but what made him an easy winner on John Elway appreciation night at Mile High Stadium were the contributions of Jimmy 's new offensive components .                       |
| **`pcc_eng_13_001.7447_x0011941_15:09-10-11`** | Several of their specials are satisfactory , if not __``particularly spectacular``__ .                                                                                                                                                                                 |
| **`pcc_eng_05_088.2605_x1411771_38:07-09-10`** | Last night 's bullpen meltdown was n't even __``particularly spectacular``__ , as far as Mets losses go .                                                                                                                                                              |
| **`pcc_eng_19_014.6613_x0220648_022:4-5-6`**   | Our outcomes are n't __``particularly spectacular``__ either ; while it is true that if you can afford it some of the best care in the world is available in the U.S. , but our national health measured by mortality and morbidity as an average is pretty mediocre . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_spectacular_ex.md`


#### 16.2. "particularly spectacular" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                 |
|:--------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_20_068.2323_x1086223_45:4-5`**   | Its colors are __``particularly spectacular``__ - with an icy blue streak of glacial water , sandwiched between emerald moss-covered rocks and darker volcanic rocks , creating a mesmerizing and very photogenic effect .  |
| **`pcc_eng_13_054.1833_x0859716_17:5-6`**   | The penguin enclosure is __``particularly spectacular``__ , and the large sea bird aviary is populated by king eiders , long-tailed ducks , Arctic terns , ruffs and other Arctic sea birds .                               |
| **`pcc_eng_18_104.5923_x1678356_0138:5-6`** | That autumn was a __``particularly spectacular``__ one in New England .                                                                                                                                                     |
| **`pcc_eng_10_088.2637_x1410326_48:4-5`**   | The terraces are __``particularly spectacular``__ when the rice is ripe , when everything is yellow , and the locals are involved in the harvesting and threshing of maize important to the economy of the entire Vietnam . |
| **`pcc_eng_10_064.1399_x1020665_05:6-7`**   | Enjoyable all year round but __``particularly spectacular``__ during the North - West trade winds when the aqua-blue waters are translucent , balmy and tranquil .                                                          |
| **`nyt_eng_20081129_0041_25:5-6`**          | the show has some __``particularly spectacular``__ turns from established masters , especially the legendary Lino Tagliapietra , whose fame in the glass world matches Chihuly 's .                                         |
| **`pcc_eng_22_036.2301_x0569079_09:4-5`**   | The harbour is __``particularly spectacular``__ with headlands that rear up , rocky and steep , on either side of the river as it widens and flows into the sea .                                                           |
| **`pcc_eng_03_018.5296_x0283463_10:3-4`**   | In a __``particularly spectacular``__ display of corporate delusion , John Deere -- the world 's largest agricultural machinery maker -- told the Copyright Office that farmers do n't own their tractors .                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_spectacular_ex.md`


### 17. _particularly shocking_


#### 17.1. "particularly shocking" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                                        |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_002.9454_x0031448_031:22-23-24`** | Although Fancies and Goodnights is a good book , and the fact that it took home the prize in 1952 is not __``particularly shocking``__ , in retrospect , it just is n't of the same quality as its competition .                                                                                                                                                   |
| **`pcc_eng_05_002.4234_x0023185_09:12-13-14`**  | Viewed in the context of this off-season , the deal is not __``particularly shocking``__ .                                                                                                                                                                                                                                                                         |
| **`pcc_eng_24_022.1488_x0341655_25:55-56-57`**  | Unlike the iconic Eva Herzigova Wonderbra advert of the nineties ( bearing the words " Hello boys " ) which was so exciting that traffic accidents were reported within the proximity of the billboards , Kim Kardashian placing thin black strips over her nipples on an otherwise totally naked photo is now neither new nor __``particularly shocking``__ now . |
| **`pcc_eng_04_073.0900_x1164325_101:06-10-11`** | From a cynical perspective , none of this is __``particularly shocking``__ ; money and American politics have always paired together like , say , gambling and football .                                                                                                                                                                                          |
| **`pcc_eng_10_089.3478_x1428007_036:10-11-12`** | I work in a classroom , but that 's not __``particularly shocking``__ to me .                                                                                                                                                                                                                                                                                      |
| **`pcc_eng_25_032.7305_x0513557_09:08-09-10`**  | Trump 's accusations here about Comey are not __``particularly shocking``__ : Trump already spent the weekend ranting about " slimeball " Comey on Twitter .                                                                                                                                                                                                       |
| **`pcc_eng_03_031.2748_x0490314_19:1-6-7`**     | Nothing about the plot is __``particularly shocking``__ , but its foggy atmosphere and chilling revelations will leave you breathless .                                                                                                                                                                                                                            |
| **`nyt_eng_19961101_0276_52:17-18-19`**         | how that will be accomplished not even he is sure , but Foreman finds this predicament not __``particularly shocking``__ .                                                                                                                                                                                                                                         |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_shocking_ex.md`


#### 17.2. "particularly shocking" under _positive_ polarity


|                                              | `token_str`                                                                                                                                                                                                                                                                                           |
|:---------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_08_093.6343_x1499671_05:25-26`**  | That is why the appearance on nearly every light pole in Montclair Village of flyers depicting a Jewish Oakland Councilmember emblazoned with swastikas was __``particularly shocking``__ to community members .                                                                                      |
| **`pcc_eng_18_060.1327_x0957263_244:50-51`** | RV : People may be most familiar with traffic violations , but your book looks at a whole other range of types of fines and fees that states and localities are now leveeing on people , largely black and brown , largely low-income populations , some of which are __``particularly shocking``__ . |
| **`pcc_eng_24_037.4065_x0588494_11:4-5`**    | But one is __``particularly shocking``__ .                                                                                                                                                                                                                                                            |
| **`pcc_eng_15_026.1452_x0406367_13:12-13`**  | The $ 1,300 bill from AMR , she said , was __``particularly shocking``__ because she knew the trip had been preauthorized by the same medical insurer .                                                                                                                                               |
| **`pcc_eng_12_022.3001_x0344577_04:4-5`**    | The incident was __``particularly shocking``__ to Azza because she was brought up in an environment that respected the army - her father was a general , along with two of her brothers and two of her uncles .                                                                                       |
| **`pcc_eng_02_108.07923_x1743786_28:1-2`**   | __``Particularly shocking``__ to me - it must be a generational reaction - was that they assaulted both the young men and the young women with the same indiscriminate force .                                                                                                                        |
| **`pcc_eng_29_085.9462_x1372112_019:5-6`**   | What made this murder-suicide __``particularly shocking``__ were the subsequent descriptions of Shane Clancy as a gregarious teetotaller whose life revolved around family , study and charity .                                                                                                      |
| **`apw_eng_19980930_1135_15:09-10`**         | Annan added that the reported massacre was `` __``particularly shocking``__ '' since the Yugoslav foreign minister had as recently as Tuesday denied Serb involvement in any atrocities .                                                                                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_shocking_ex.md`


### Saving *`particularly`* Samples

 > in `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/`

1. *particularly new*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_new_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_new_99ex~12.csv"`

2. *particularly religious*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_religious_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_religious_99ex~4.csv"`

3. *particularly surprising*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_surprising_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_surprising_99ex~33.csv"`

4. *particularly wrong*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_wrong_99ex~65.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_wrong_99ex~4.csv"`

5. *particularly original*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_original_99ex~85.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_original_99ex~6.csv"`

6. *particularly comfortable*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_comfortable_99ex~84.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_comfortable_99ex~4.csv"`

7. *particularly unusual*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_unusual_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_unusual_99ex~19.csv"`

8. *particularly close*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_close_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_close_99ex~99.csv"`

9. *particularly acute*

    Full polarity samples saved as...
    1. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_acute_99ex~99.csv"`

10. *particularly remarkable*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_remarkable_99ex~72.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_remarkable_99ex~27.csv"`

11. *particularly sure*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_sure_99ex~17.csv"`

12. *particularly forthcoming*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_forthcoming_99ex~19.csv"`

13. *particularly glamorous*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_glamorous_99ex~19.csv"`

14. *particularly reassuring*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_reassuring_99ex~20.csv"`

15. *particularly novel*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_novel_99ex~37.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_novel_99ex~3.csv"`

16. *particularly spectacular*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_spectacular_99ex~37.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_spectacular_99ex~9.csv"`

17. *particularly shocking*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/neg-particularly/neg-particularly_shocking_99ex~27.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/particularly/pos-particularly/pos-particularly_shocking_99ex~26.csv"`


## 11. *terribly*

|                                 |   `f` |   `dP1` |   `LRC` |   `P1` |     `G2` |   `f2` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `dP2` |   `P2` |   `adj_total` |
|:--------------------------------|------:|--------:|--------:|-------:|---------:|-------:|----------:|------------:|------------:|--------:|-------:|--------------:|
| **NEGany~terribly_surprising**  |   949 |    0.50 |    5.73 |   1.00 | 1,315.74 |    949 |    474.50 |      474.50 |        0.50 |    0.00 |   0.00 |        18,886 |
| **NEGany~terribly_different**   |   366 |    0.50 |    4.10 |   1.00 |   494.98 |    367 |    183.50 |      182.50 |        0.50 |    0.00 |   0.00 |        80,019 |
| **NEGany~terribly_interested**  |   486 |    0.49 |    4.08 |   0.99 |   632.89 |    490 |    245.00 |      241.00 |        0.50 |    0.00 |   0.00 |        34,247 |
| **NEGany~terribly_interesting** |   362 |    0.49 |    3.54 |   0.99 |   455.90 |    367 |    183.50 |      178.50 |        0.49 |    0.00 |   0.00 |        32,662 |
| **NEGany~terribly_original**    |   199 |    0.50 |    3.43 |   1.00 |   275.88 |    199 |     99.50 |       99.50 |        0.50 |    0.00 |   0.00 |         4,705 |
| **NEGany~terribly_good**        |   332 |    0.49 |    3.41 |   0.99 |   415.17 |    337 |    168.50 |      163.50 |        0.49 |    0.00 |   0.00 |       202,048 |
| **NEGany~terribly_helpful**     |   184 |    0.50 |    3.31 |   1.00 |   255.08 |    184 |     92.00 |       92.00 |        0.50 |    0.00 |   0.00 |        17,013 |
| **NEGany~terribly_comfortable** |   129 |    0.50 |    2.77 |   1.00 |   178.83 |    129 |     64.50 |       64.50 |        0.50 |    0.00 |   0.00 |        24,239 |
| **NEGany~terribly_new**         |   113 |    0.50 |    2.56 |   1.00 |   156.65 |    113 |     56.50 |       56.50 |        0.50 |    0.00 |   0.00 |        21,548 |
| **NEGany~terribly_uncommon**    |   103 |    0.50 |    2.42 |   1.00 |   142.79 |    103 |     51.50 |       51.50 |        0.50 |    0.00 |   0.00 |         3,193 |
| **NEGmir~terribly_new**         |    69 |    0.50 |    1.90 |   1.00 |    95.66 |     69 |     34.50 |       34.50 |        0.50 |    0.00 |   0.00 |         4,393 |
| **NEGmir~terribly_surprising**  |    67 |    0.50 |    1.85 |   1.00 |    92.89 |     67 |     33.50 |       33.50 |        0.50 |    0.00 |   0.00 |         1,270 |
| **NEGmir~terribly_exciting**    |    78 |    0.44 |    1.23 |   0.94 |    77.28 |     83 |     41.50 |       36.50 |        0.47 |    0.00 |   0.00 |         4,001 |
| **NEGmir~terribly_original**    |    45 |    0.50 |    1.19 |   1.00 |    62.39 |     45 |     22.50 |       22.50 |        0.50 |    0.00 |   0.00 |           719 |
| **POS~terribly_wrong**          |   340 |    0.31 |    1.16 |   0.81 |   169.53 |    422 |    211.00 |      129.00 |        0.38 |    0.00 |   0.00 |         8,526 |
| **NEGmir~terribly_interesting** |    56 |    0.47 |    1.09 |   0.97 |    63.01 |     58 |     29.00 |       27.00 |        0.48 |    0.00 |   0.00 |         3,893 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/ALL-terribly_10mostNEG-bigrams_AMscores.2024-08-07.md`


|                      |       `N` |      `f1` |   `adv_total` |
|:---------------------|----------:|----------:|--------------:|
| **NEGATED_terribly** | 6,347,362 | 3,173,681 |        19,801 |
| **NEGMIR_terribly**  |   583,470 |   291,735 |         2,196 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/terribly_MarginalFreqs_2024-08-07.md`


### 1. _terribly surprising_


#### 1.1. "terribly surprising" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                                  |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_01_095.1256_x1522014_03:6-7-8`**     | Mind you , this is not __``terribly surprising``__ -- when Canadian customs started seizing our laptops and cameras at the border a couple years back , we kinda figured the snoop was on .                                                                                                                                                  |
| **`pcc_eng_07_106.6523_x1707553_68:20-21-22`**  | As for the Taliban , the fact that their taking shots at NATO through a social media site is n't __``terribly surprising``__ provided their affinity for web- based propaganda , including their site " Islamic Emirate of Afghanistan , " a website that posts messages , battlefield reports and propaganda for the Taliban .              |
| **`pcc_eng_12_061.9454_x0985601_07:28-29-30`**  | Though the Georgia natives are most well -known for their swaggering , powerful Southern rock - indebted sound , their choice to record an acoustic EP is n't __``terribly surprising``__ .                                                                                                                                                  |
| **`pcc_eng_12_035.2706_x0554338_19:43-45-46`**  | And while no studies have been done on our political class , given that psychopaths are deceitful , narcissistic backstabbers who never take blame and do n't get embarrassed , even when caught in a bald - faced lie , it would n't be __``terribly surprising``__ if they were enriched among politicians as well .                       |
| **`pcc_eng_14_002.3265_x0021569_05:08-10-11`**  | Reality echoes this , and that should n't be __``terribly surprising``__ .                                                                                                                                                                                                                                                                   |
| **`pcc_eng_14_038.5376_x0606409_59:6-7-8`**     | Given this , it 's not __``terribly surprising``__ that shooting guards and small forwards , of which there are 19 on the list , but only five in the top 30 , are punished by the metric .                                                                                                                                                  |
| **`pcc_eng_03_084.6071_x1353877_008:10-11-12`** | You know if it 's it 's it 's not __``terribly surprising``__ you know election we have a fiscal session that we .                                                                                                                                                                                                                           |
| **`pcc_eng_28_039.3307_x0619901_10:5-6-7`**     | These findings , while not __``terribly surprising``__ , particularly when considering how the rapid advances in mobile technology have made ' working from anywhere ' a possibility and reality for so many of us , also raise some important issues for organizations or leaders that are supporting or offering telework to their teams . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_surprising_ex.md`


#### 1.2. "terribly surprising" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*terribly surprising*" found in **positive** hits.

### 2. _terribly different_


#### 2.1. "terribly different" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                 |
|:------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20051229_0008_24:34-35-36`**         | still , it should not be lost on the 33,700 workers who walked off the job , breaking New York State law in the process , that they wound up with a contract not __``terribly different``__ than what was on the table before the walkout . |
| **`pcc_eng_06_100.4804_x1609180_06:6-7-8`**     | What is happening now is not __``terribly different``__ , though the regime has changed and the arguments are of a different variety .                                                                                                      |
| **`pcc_eng_23_088.1611_x1408595_19:3-4-5`**     | It is not __``terribly different``__ from many of the other internal conflicts around the world , which forces us to ask once again why the U.S. is involved in Libya 's civil war .                                                        |
| **`pcc_eng_17_071.8342_x1144651_052:7-8-9`**    | D'Antoni 's regular season numbers are not __``terribly different``__ from his counterpart 's : 328-275 ( .544 ) .                                                                                                                          |
| **`pcc_eng_14_090.0848_x1439992_170:13-14-15`** | " The transformation of Sinn Fein into a constitutional Nationalist party , not __``terribly different``__ from the SDLP . "                                                                                                                |
| **`pcc_eng_02_085.6874_x1369116_06:09-10-11`**  | " The new remit is at face value not __``terribly different``__ from the current one , but the devil is in the detail , " said David Tinsley , an economist at BNP Paribas SA who formerly worked at the Bank of England .                  |
| **`pcc_eng_11_013.6913_x0205091_21:16-17-18`**  | 2 . the average cumulative trend rate from 2003 - 2009 was 38 % , not __``terribly different``__ from the overall CPI for similar services ( 6 % per year ) .                                                                               |
| **`pcc_eng_29_048.6231_x0769062_07:14-16-17`**  | I 've kept a similar color scheme , so the new look is n't too __``terribly different``__ .                                                                                                                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_different_ex.md`


#### 2.2. "terribly different" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                |
|:--------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_12_056.2706_x0893490_22:23-24`** | Orza , on criticism of the union position : " Oh , come now , minor-leaguers are being asked to do something __``terribly different``__ than in the past . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_different_ex.md`


### 3. _terribly interested_


#### 3.1. "terribly interested" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                    |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_28_065.2257_x1039269_34:3-4-5`**     | I was n't __``terribly interested``__ by the film when it first came out ; having seen it , I made sure that I 'd catch Catching Fire on the big screen .                                                      |
| **`pcc_eng_26_033.2481_x0521394_08:11-12-13`**  | Bud Selig and the owners who pay his salary were n't __``terribly interested``__ in busting superstars for steroid use , not when new stadiums were going up everywhere and television money kept pouring in . |
| **`pcc_eng_08_076.1877_x1217330_6:12-14-15`**   | Because she 's a child , it makes sense Lee would n't be __``terribly interested``__ in gambling and that she might seem almost invisible in the context of that many adults focused on their own desires .    |
| **`pcc_eng_25_009.0421_x0130317_09:6-7-8`**     | But Aly & AJ are n't __``terribly interested``__ in upholding expectations :                                                                                                                                   |
| **`pcc_eng_15_015.1317_x0228127_050:09-10-11`** | At first , Nate admitted , he was n't __``terribly interested``__ in jiu-jitsu .                                                                                                                               |
| **`pcc_eng_04_070.0403_x1115129_59:09-10-11`**  | And why , most likely , airlines are not __``terribly interested``__ in improving it .                                                                                                                         |
| **`pcc_eng_28_036.9295_x0581033_38:5-6-7`**     | Although most flippers are n't __``terribly interested``__ in being landlords , generating rental income from a botched deal is a solid backup plan .                                                          |
| **`nyt_eng_20000206_0226_24:4-5-6`**            | `` I 'm not __``terribly interested``__ in introspection but I do have deep convictions . ''                                                                                                                   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_interested_ex.md`


#### 3.2. "terribly interested" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                        |
|:--------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_11_059.3260_x0943682_18:6-7`**   | That he has been so __``terribly interested``__ in the church .                                                                                                                                                    |
| **`pcc_eng_05_028.1275_x0439386_33:10-11`** | Perhaps this is the reason why Sachin is so __``terribly interested``__ in them even today at the turn of the twentifirst century and understand why Eliot was so excited about them almost a hundred years back . |
| **`pcc_eng_19_091.4229_x1461538_283:4-5`**  | If I am __``terribly interested``__ in something I give my attention to it , the rest of the time I am not attentive .                                                                                             |
| **`pcc_eng_20_044.0058_x0694803_06:6-7`**   | Right now , JL is __``terribly interested``__ in anything that exists under the sea - whales and sharks in particular .                                                                                            |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_interested_ex.md`


### 4. _terribly interesting_


#### 4.1. "terribly interesting" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                                                                                                                |
|:------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_10_024.2422_x0375450_14:15-16-17`**  | Gere 's slick and handsome , the very visage of corporate America , but not __``terribly interesting``__ .                                                                                                                                                                                                                 |
| **`pcc_eng_19_070.0245_x1114741_11:6-7-8`**     | Some of the material is n't __``terribly interesting``__ , but there 's enough here to keep the listener paying attention , and " Suite " is perhaps the best song the group ever composed .                                                                                                                               |
| **`pcc_eng_24_027.2347_x0424190_11:15-16-17`**  | Every app is going to have a bug or two , and there 's nothing __``terribly interesting``__ about those , but the ads WERE tricky .                                                                                                                                                                                        |
| **`pcc_eng_04_104.5384_x1672604_21:6-8-9`**     | You 'd think this would not be __``terribly interesting``__ , but I assure you that if you take a close look and read the accompanying stories , this exhibit is strangely engrossing .                                                                                                                                    |
| **`pcc_eng_01_031.6063_x0494676_33:41-42-43`**  | Two difficulty levels are available , with " hard " mode offering tougher puzzles along with the usual increases in difficulty - no tutorial , longer hint recharges and so forth - but given that most of the puzzles are n't __``terribly interesting``__ in the first place , I 'm not sure it 'd be worth the bother . |
| **`nyt_eng_19960627_0298_63:15-16-17`**         | Normi Noel is suitably mournful as Wharton , but her thoughts about war are n't __``terribly interesting``__ .                                                                                                                                                                                                             |
| **`pcc_eng_17_104.6921_x1676212_002:53-54-55`** | I can read through only what interests or affects me and get it all done in about 15 minutes , as opposed to having to watch an hour 's worth of news ( local and national / world ) complete with bad hair , bad jokes , human interest stories that are n't __``terribly interesting``__ , and commercials .             |
| **`nyt_eng_20050209_0030_2:44-48-49`**          | Barry Bonds should have been that third rail because he is a considerably better player , but people long ago decided whether to like him or not like him based on a million other factors , all of which you know about and none of which is __``terribly interesting``__ this morning .                                  |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_interesting_ex.md`


#### 4.2. "terribly interesting" under _positive_ polarity


|                                              | `token_str`                                                                                                                            |
|:---------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_21_020.4234_x0313691_17:4-5`**    | I find it __``terribly interesting``__ that Arie decided to have a 1:1 date with pretty much every woman that was left on the show .   |
| **`pcc_eng_01_032.8653_x0515030_17:6-7`**    | Anyways , it 's a __``terribly interesting``__ read : New Yorker 's A Man of Tastes                                                    |
| **`pcc_eng_06_015.1055_x0228065_126:5-6`**   | I found it all __``terribly interesting``__ .                                                                                          |
| **`pcc_eng_06_021.1478_x0325957_16:16-17`**  | They have the world at their fingertips -- a fun , frivolous , and yet __``terribly interesting``__ world in the palm of their hands . |
| **`pcc_eng_16_081.4178_x1301557_096:20-21`** | He had suddenly gone deaf , it appeared , and had found something in the far distance that was __``terribly interesting``__ .          |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_interesting_ex.md`


### 5. _terribly original_


#### 5.1. "terribly original" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_22_084.8728_x1355692_20:11-12-13`**  | But it 's worth pointing out that the movie is n't __``terribly original``__ .                                                                                                             |
| **`pcc_eng_17_083.6372_x1335459_052:6-7-8`**    | Now , the idea was n't __``terribly original``__ when it comes to warfare .                                                                                                                |
| **`pcc_eng_11_083.5065_x1335366_5:5-6-7`**      | Though the name is n't __``terribly original``__ , the fact that there are still a few ways to raft 2 for the price of 1 is .                                                              |
| **`pcc_eng_14_034.9684_x0548704_27:5-6-7`**     | Though the story is n't __``terribly original``__ , it is much better than expected thanks to Black 's voice work and some really sharp writing .                                          |
| **`pcc_eng_07_056.3784_x0895034_38:19-20-21`**  | Even when he appeared on " Dancing with the Stars , " to hilarious effect , Sapp did nothing __``terribly original``__ .                                                                   |
| **`pcc_eng_08_085.8063_x1372984_18:3-4-5`**     | This is n't __``terribly original``__ , but it does take effort to measure oneself .                                                                                                       |
| **`pcc_eng_16_088.0325_x1408847_017:13-14-15`** | In its place , a new identity would form that , while not __``terribly original``__ in the context of the times , was nonetheless a much truer vision of who he was .                      |
| **`nyt_eng_19960205_0510_6:17-18-19`**          | to begin with , the notion of seeing Washington as a model of moral behavior is not __``terribly original``__ ; it was enthusiastically practiced during the first president 's own time . |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_original_ex.md`


#### 5.2. "terribly original" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*terribly original*" found in **positive** hits.

### 6. _terribly good_


#### 6.1. "terribly good" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                   |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_04_059.0083_x0936974_42:3-4-5`**    | I 'm not __``terribly good``__ at stealth though .                                                                                                                                                                                                                            |
| **`pcc_eng_10_023.4060_x0361904_27:10-11-12`** | As we 've established , this Gemini personality is not __``terribly good``__ at providing a solid foundation for itself , being far more interested in the airy and ephemeral areas of cerebral and social interaction .                                                      |
| **`pcc_eng_12_032.1463_x0504256_131:4-6-7`**   | And they 're not even __``terribly good``__ at tearing them down from 35,000 feet .                                                                                                                                                                                           |
| **`nyt_eng_19990303_0306_8:26-27-28`**         | there is a common belief in publishing that , with few exceptions , a writer who can produce more than one book a year is not __``terribly good``__ , so some authors churn out one book a year for one publisher and another under a different name for a second publisher . |
| **`nyt_eng_20000808_0043_34:16-17-18`**        | `` They are stuck in the world of competing on price _ and they 're not __``terribly good``__ at that online right now . ''                                                                                                                                                   |
| **`pcc_eng_26_082.3195_x1314757_47:5-6-7`**    | A tad clumpy , not __``terribly good``__ , had to mix with the sauce to get any real enjoyment out of the dish by itself .                                                                                                                                                    |
| **`pcc_eng_21_099.2001_x1586564_05:4-5-6`**    | What they 're not __``terribly good``__ at , however , is imitating other comedy formats , something they insist on doing time and time and time again .                                                                                                                      |
| **`pcc_eng_17_075.2104_x1199364_23:23-24-25`** | This has to do with nation - building , which people on this side of the Atlantic think the United States is not __``terribly good``__ at . "                                                                                                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_good_ex.md`


#### 6.2. "terribly good" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                        |
|:--------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_15_005.3419_x0070047_33:6-7`**   | Is it because he was __``terribly good``__ looking as she mentions initially or was it because he was brown and as she mentioned she had never been to India and was lonely for over 30 years now , so was it her exotic fantasy ? |
| **`pcc_eng_22_052.8296_x0837300_22:30-31`** | The heart and soul of the city is the French Quarter that has many shops , elegant art galleries , restaurants , bars , jazz club , so a __``terribly good``__ nightlife .                                                         |
| **`pcc_eng_02_042.2241_x0666962_59:4-5`**   | You 're a __``terribly good``__ looking fellow as well , and everything about you is absolute magic .                                                                                                                              |
| **`nyt_eng_19960605_0342_12:34-35`**        | Sir Peter , speaking from England , said : `` I 've known Dulcie and Michael all my theatrical life , so when it came to doing this , it seemed like a __``terribly good``__ idea .                                                |
| **`pcc_eng_13_025.6466_x0398522_07:4-5`**   | He 's just __``terribly good``__ at negotiations .                                                                                                                                                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_good_ex.md`


### 7. _terribly helpful_


#### 7.1. "terribly helpful" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                               |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_02_001.3940_x0006414_06:15-16-17`** | It has been brought to my attention that my description of free verse is not __``terribly helpful``__ .                                                                                                                                                                                                                                   |
| **`pcc_eng_16_087.6310_x1402209_05:22-23-24`** | But Ros Altmann , an independent pensions expert and a former Downing Street adviser , said the information could be " not __``terribly helpful``__ " and " a bit misleading " unless the figures are regularly revised over the course of someone 's retirement to take account of any life changes and backed up by detailed guidance . |
| **`nyt_eng_20070126_0172_19:20-21-22`**        | but that rule needs to be stretched if you are fit and active and dad was not -LRB- and not __``terribly helpful``__ if your parents are still living -RRB- .                                                                                                                                                                             |
| **`pcc_eng_19_056.8642_x0901604_013:3-4-5`**   | This is not __``terribly helpful``__ in the case of a long-lived and long-reigning monarch .                                                                                                                                                                                                                                              |
| **`pcc_eng_16_057.8415_x0920206_16:1-2-3`**    | Not __``terribly helpful``__ .                                                                                                                                                                                                                                                                                                            |
| **`pcc_eng_14_008.0741_x0114194_10:21-22-23`** | We got to hear of a dentist in the nearby large town who actually speaks English and not German ( not __``terribly helpful``__ for us ) and so we decided we really ought to go .                                                                                                                                                         |
| **`pcc_eng_12_072.7363_x1158906_15:3-4-5`**    | It 's not __``terribly helpful``__ to simply report the high death toll without digging deeper .                                                                                                                                                                                                                                          |
| **`pcc_eng_13_031.0721_x0486336_05:19-20-21`** | In its most recent filing issued last week , Intel portrays this number of interviews as excessive and not __``terribly helpful``__ for the matters at hand .                                                                                                                                                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_helpful_ex.md`


#### 7.2. "terribly helpful" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*terribly helpful*" found in **positive** hits.

### 8. _terribly comfortable_


#### 8.1. "terribly comfortable" under _negative_ polarity


|                                                 | `token_str`                                                                                                                                                                                                                    |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_25_007.6093_x0107255_14:21-22-23`**  | Gresham does n't use social media and , according to Kyle Odegard for the official team site , he was not __``terribly comfortable``__ talking about the event at the airport .                                                |
| **`pcc_eng_19_017.6419_x0268452_05:1-2-3`**     | Not __``terribly comfortable``__ for the woman who regularly wears black polyester pants ( we 're getting close to laundry day here ) .                                                                                        |
| **`pcc_eng_04_046.5690_x0736484_14:30-31-32`**  | Got to meet Colin and Terry afterwards , shake their hands , quick pic ( thanks Owen ) and thank them for the music ... very gracious guys if not __``terribly comfortable``__ in the situation .                              |
| **`nyt_eng_19970225_0729_39:08-09-10`**         | the bed , he said , was not __``terribly comfortable``__ , noting that it was so high `` you literally have to step up . ''                                                                                                    |
| **`pcc_eng_20_002.0393_x0016617_10:26-27-28`**  | At the moment she 's scampering around behind me on the couch and trying to skootch her way up into my lap , a position not __``terribly comfortable``__ for me when hunching over my Mac on the coffee table in front of me . |
| **`pcc_eng_16_025.9726_x0404308_059:16-17-18`** | It was clear that personal disclosures were honored at Naropa , and although I was n't __``terribly comfortable``__ with the idea , I decided to ' out myself . '                                                              |
| **`pcc_eng_25_087.2580_x1396052_30:13-14-15`**  | Its a stool Posted by : Swooce Mc Goose from : on Not __``terribly comfortable``__ for more than 15 minutes .                                                                                                                  |
| **`pcc_eng_05_038.9833_x0614842_26:11-12-13`**  | ... Clinton has acknowledged in the past that she is n't __``terribly comfortable``__ speaking in public and , therefore , should avoid doing it .                                                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_comfortable_ex.md`


#### 8.2. "terribly comfortable" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*terribly comfortable*" found in **positive** hits.

### 9. _terribly new_


#### 9.1. "terribly new" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                          |
|:-----------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`nyt_eng_20050125_0046_4:08-09-10`**         | the idea of playing alongside men is n't __``terribly new``__ to a girl who pretended to be a boy in order to play pick-up matches on the street .                   |
| **`pcc_eng_28_076.0342_x1213734_13:10-11-12`** | Unusual shapes and distortion printing of metal sheet is not __``terribly new``__ .                                                                                  |
| **`pcc_eng_25_092.8970_x1487174_10:3-4-5`**    | It 's not __``terribly new``__ , after all .                                                                                                                         |
| **`nyt_eng_19961014_0568_7:1-2-3`**            | nothing __``terribly new``__ there .                                                                                                                                 |
| **`pcc_eng_test_3.09898_x50442_7:5-6-7`**      | Actually , this is n't __``terribly new``__ , as there have previously been stories about slime moulds solving mazes and controlling robots .                        |
| **`nyt_eng_20000302_0121_10:7-8-9`**           | it 's a variation on the not __``terribly new``__ concept of the software emulator , a computer program that makes one computer act as if it were another computer . |
| **`pcc_eng_12_032.5097_x0510113_12:4-5-6`**    | " There was nothing __``terribly new``__ in there .                                                                                                                  |
| **`pcc_eng_19_018.1877_x0277327_13:7-8-9`**    | Most of these healing modalities are n't __``terribly new``__ , but are effective methods that have simply been inaccessible to Americans .                          |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_new_ex.md`


#### 9.2. "terribly new" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*terribly new*" found in **positive** hits.

### 10. _terribly uncommon_


#### 10.1. "terribly uncommon" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                                                                                                                                                                  |
|:-----------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_09_038.8670_x0612840_30:6-7-8`**    | Apple Watch stock shortages are n't __``terribly uncommon``__ , with some bundles and bands occasionally selling out months ahead of a refresh , but given Apple 's commitment to timely band releases and the number of models currently out of stock it 's likely that the company will introduce a similar refresh of Apple Watch bundles and bands in the coming weeks . |
| **`pcc_eng_22_089.4691_x1429976_26:26-27-28`** | The second surprise that fell through was that Bill had set up a police escort from the ceremony to the reception - something that is n't __``terribly uncommon``__ for a high ranking official in a small town .                                                                                                                                                            |
| **`pcc_eng_04_085.5134_x1365481_36:1-7-8`**    | Not that such a sight is __``terribly uncommon``__ in Copenhagen , known as the world 's cycling capital .                                                                                                                                                                                                                                                                   |
| **`pcc_eng_25_011.8947_x0176193_63:5-6-7`**    | Apparently , this is n't __``terribly uncommon``__ in Japan and many Japanese have mastered the art .                                                                                                                                                                                                                                                                        |
| **`pcc_eng_29_039.7301_x0625323_3:09-10-11`**  | Two - headedness , or bicephaly , is not __``terribly uncommon``__ among turtles and snakes , and the twins are reportedly in good health . ( 0:34 )                                                                                                                                                                                                                         |
| **`pcc_eng_14_031.3481_x0490431_40:3-4-5`**    | It 's not __``terribly uncommon``__ for the entire party to be decimated in a seemingly - routine random encounter , especially if they 're just trying to breeze their way through an encounter .                                                                                                                                                                           |
| **`pcc_eng_14_082.8058_x1322266_07:14-15-16`** | While Christians marrying Jews , Jews marrying Hindus and Hindus marrying Buddhists is not __``terribly uncommon``__ in the US and other first world nations , it has not always been this way .                                                                                                                                                                             |
| **`pcc_eng_28_026.9821_x0419639_32:3-4-5`**    | Plums are n't __``terribly uncommon``__ , but not a lot of people are familiar with pluots .                                                                                                                                                                                                                                                                                 |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_uncommon_ex.md`


#### 10.2. "terribly uncommon" under _positive_ polarity

ㄟ( ▔, ▔ )ㄏ No examples of "*terribly uncommon*" found in **positive** hits.

### 11. _terribly exciting_


#### 11.1. "terribly exciting" under _negative_ polarity


|                                                | `token_str`                                                                                                                           |
|:-----------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_11_012.5989_x0187456_62:07-10-11`** | Other than that my year has n't been too __``terribly exciting``__ , which is probably an OK thing .                                  |
| **`pcc_eng_12_002.9577_x0031639_79:6-7-8`**    | It 's rather repetitive , not __``terribly exciting``__ , but makes for a good transition song during live performances .             |
| **`pcc_eng_15_098.8682_x1581771_112:1-3-4`**   | None is __``terribly exciting``__ .                                                                                                   |
| **`pcc_eng_05_099.1957_x1588393_08:08-09-10`** | Our culinary adventure for the week was not __``terribly exciting``__ .                                                               |
| **`pcc_eng_11_087.9274_x1406989_25:25-26-27`** | I made a new entry about Tray 30 , so feel free to pop on over and check it out ( though it 's not __``terribly exciting``__ , haha ) |
| **`pcc_eng_10_028.8858_x0450776_07:09-10-11`** | It 's not awful , it 's just not __``terribly exciting``__ .                                                                          |
| **`pcc_eng_21_097.4945_x1558969_06:1-4-5`**    | Nor is it __``terribly exciting``__ .                                                                                                 |
| **`pcc_eng_14_007.4135_x0103643_11:4-5-6`**    | ( It is not __``terribly exciting``__ , but it gets the job done . )                                                                  |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_exciting_ex.md`


#### 11.2. "terribly exciting" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                       |
|:--------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_26_049.6472_x0786571_029:6-7`**  | I realize this must be __``terribly exciting``__ for you , but , this is n't a race , remember ? "                                                                |
| **`pcc_eng_22_084.1527_x1344137_23:4-5`**   | It all sounds __``terribly exciting``__ .                                                                                                                         |
| **`pcc_eng_12_063.8691_x1016536_60:5-6`**   | It all sounds like __``terribly exciting``__ mathematics , if everything works out .                                                                              |
| **`pcc_eng_21_057.4585_x0912762_05:5-6`**   | The whole thing is __``terribly exciting``__ ; and it gives Mickey an idea .                                                                                      |
| **`pcc_eng_25_099.7887_x1598711_12:17-18`** | And , late in the week another long-sought demolition wrapped up , as seen in this __``terribly exciting``__ before- and - after :                                |
| **`apw_eng_19950909_0814_55:4-5`**          | `` It was __``terribly exciting``__ ; I was fairly flying , '' the pianist says .                                                                                 |
| **`nyt_eng_20071116_0057_36:27-28`**        | `` People can look at their own genomes and find where your and your family fit into human population history , so I think this is __``terribly exciting``__ . '' |
| **`pcc_eng_15_063.6369_x1011812_07:3-4`**   | It 's __``terribly exciting``__ for me , " said Linda , who has been writing , illuminating and painting ketubot ( Jewish marriage contracts ) for many years .   |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_exciting_ex.md`


### 12. _terribly wrong_


#### 12.1. "terribly wrong" under _negative_ polarity


|                                                | `token_str`                                                                                                                                                                                                                         |
|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_27_053.2939_x0845236_12:3-4-5`**    | There 's nothing __``terribly wrong``__ with that view , except that it 's essentially the same song they 've been singing for decades with so far nothing to show for it .                                                         |
| **`pcc_eng_13_042.5807_x0672288_103:3-5-6`**   | LYDEN : Not so __``terribly wrong``__ .                                                                                                                                                                                             |
| **`pcc_eng_16_023.5397_x0364766_03:30-31-32`** | We recently posted a story about Subaru 's renunciation of the Tribeca SUV , a model that was the exact opposite of a sales hit , despite there being nothing __``terribly wrong``__ with it .                                      |
| **`pcc_eng_02_097.6582_x1562714_12:1-2-3`**    | Nothing __``terribly wrong``__ with that .                                                                                                                                                                                          |
| **`pcc_eng_05_082.6438_x1321386_28:3-4-5`**    | There 's nothing __``terribly wrong``__ with the rooms at the Biltmore -- yours will most likely be reasonably clean and reasonably comfortable , and they 're spacious enough .                                                    |
| **`pcc_eng_21_064.1143_x1019955_57:39-41-42`** | " His walks were a little up ( last season ) , " general manager Josh Byrnes told the Arizona Republic , " but I think a lot of his stuff and his supporting statistics were indicators that nothing was __``terribly wrong``__ . " |
| **`pcc_eng_20_090.2569_x1442019_08:3-4-5`**    | There is nothing __``terribly wrong``__ with the story itself .                                                                                                                                                                     |
| **`nyt_eng_19960717_0118_24:3-4-5`**           | there 's nothing __``terribly wrong``__ with Mike Nichols ' scene-by-scene remake of the 1978 French farce _ except the intervening 18 years .                                                                                      |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_wrong_ex.md`


#### 12.2. "terribly wrong" under _positive_ polarity


|                                             | `token_str`                                                                                                                                                                                                                                                                                                                                                                                |
|:--------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`pcc_eng_21_079.7829_x1273214_3:11-12`**  | But like in most of cases like this it went __``terribly wrong``__ and resulted in death of one of the girls .                                                                                                                                                                                                                                                                             |
| **`pcc_eng_26_062.4519_x0993544_03:5-6`**   | But then things went __``terribly wrong``__ .                                                                                                                                                                                                                                                                                                                                              |
| **`pcc_eng_18_011.9339_x0177029_21:5-6`**   | She realized something was __``terribly wrong``__ with their son and brought him back to their Montana home .                                                                                                                                                                                                                                                                              |
| **`pcc_eng_12_035.8589_x0563962_27:14-15`** | A reevaluation of what went right in South Africa , and what went __``terribly wrong``__ in Israel , therefore seems to be completely in order .                                                                                                                                                                                                                                           |
| **`nyt_eng_20041030_0162_44:4-5`**          | something had gone __``terribly wrong``__ .                                                                                                                                                                                                                                                                                                                                                |
| **`pcc_eng_15_068.9727_x1098091_09:35-36`** | I forgive myself that I 've accepted and allowed myself to define myself as inadequate when and as a Spiritual Experience emerge from within and as me , as if I have done something __``terribly wrong``__ , not seeing , realizing and understanding that I should simply see it as it is , another window of opportunity to sort out that which was already existent within and as me . |
| **`pcc_eng_10_093.3210_x1492579_1:6-7`**    | A Yellowstone Bison Photo Goes __``Terribly Wrong``__                                                                                                                                                                                                                                                                                                                                      |
| **`pcc_eng_12_030.1678_x0472299_25:28-29`** | The first great idea in Morrison 's incarnation of the series was playing up the idea that " superheroes " could mean " people who have something __``terribly wrong``__ with their bodies . "                                                                                                                                                                                             |


> saved as:  
> `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_wrong_ex.md`


### Saving *`terribly`* Samples

 > in `/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/`

1. *terribly surprising*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_surprising_99ex~99.csv"`

2. *terribly different*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_different_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_different_99ex~1.csv"`

3. *terribly interested*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_interested_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_interested_99ex~4.csv"`

4. *terribly interesting*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_interesting_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_interesting_99ex~5.csv"`

5. *terribly original*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_original_99ex~58.csv"`

6. *terribly good*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_good_99ex~85.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_good_99ex~5.csv"`

7. *terribly helpful*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_helpful_99ex~63.csv"`

8. *terribly comfortable*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_comfortable_99ex~28.csv"`

9. *terribly new*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_new_99ex~28.csv"`

10. *terribly uncommon*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_uncommon_99ex~21.csv"`

11. *terribly exciting*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_exciting_99ex~99.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_exciting_99ex~11.csv"`

12. *terribly wrong*

    Full polarity samples saved as...
    1. **negative**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/neg-terribly/neg-terribly_wrong_99ex~26.csv"`
    2. **positive**  
       `"/share/compling/projects/sanpi/results/top_AM/NEQ/polar_bigram_examples/terribly/pos-terribly/pos-terribly_wrong_99ex~99.csv"`

